In [1]:
import pandas as pd
import numpy as np
import random
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os

In [2]:
conftfos = pyspark.SparkConf().setAll([
                                       ('spark.executor.memory','20G'),
                                       ('spark.driver.memory','20G'),
                                             
                                      ])

In [3]:
sc = SparkContext(conf=conftfos)
spark = SparkSession(sc)

In [4]:
#将parquet 文件转换成dataframe
def generate_dataframe_spark(filename):
    data = []
    path = '/Users/fang/Documents/Fang/jupyter/FanXiQian/FanXiQian_real/'
    for i in os.listdir(path+filename+'/'):
        if 'parquet' in i:
            temp = spark.read.parquet(path+filename+'/'+i)
            data.append(temp)
    if len(data) == 2:
        return data[0].unin(data[1])
    if len(data) == 1:
        return data[0]
    if len(data) > 2:
        a = data[0]
        for i in range(1,len(data)):
            a = a.union(data[i])
        return a

In [8]:
ori_str = generate_dataframe_spark('ori_str') #可疑案宗表
ori_trx_new = generate_dataframe_spark('ori_trx_new') #交易详情表
ori_retail_new = generate_dataframe_spark('ori_retail_new') #对私客户表
ori_coporate = generate_dataframe_spark('ori_coporate') #对公客户表
ori_account = generate_dataframe_spark('ori_account') #账户(卡)表

In [6]:
ori_trx_new.printSchema()

root
 |-- trx_id: string (nullable = true)
 |-- cust_id: string (nullable = true)
 |-- trx_curr: string (nullable = true)
 |-- flag_cash: string (nullable = true)
 |-- flag_crdr: string (nullable = true)
 |-- trx_amt: string (nullable = true)
 |-- balance_amt: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- target_cust_id: string (nullable = true)
 |-- target_account_id: string (nullable = true)
 |-- trx_code: string (nullable = true)
 |-- target_type: string (nullable = true)
 |-- target_third_pay: string (nullable = true)
 |-- target_offshore: string (nullable = true)
 |-- trx_date: string (nullable = true)
 |-- trx_time: string (nullable = true)



In [7]:
ori_str.printSchema()

root
 |-- ins_date: string (nullable = true)
 |-- label: string (nullable = true)
 |-- rule_id: string (nullable = true)
 |-- p_flag: string (nullable = true)
 |-- trx_id: string (nullable = true)
 |-- ins_id: string (nullable = true)



In [9]:
ori_retail_new.printSchema()

root
 |-- cust_id: string (nullable = true)
 |-- doc_type: string (nullable = true)
 |-- doc_country: string (nullable = true)
 |-- nation: string (nullable = true)
 |-- job: string (nullable = true)
 |-- edu: string (nullable = true)
 |-- marriage: string (nullable = true)
 |-- age: string (nullable = true)



In [10]:
ori_coporate.printSchema()

root
 |-- cust_id: string (nullable = true)
 |-- ind_type: string (nullable = true)
 |-- own_type: string (nullable = true)
 |-- reg_curr: string (nullable = true)
 |-- reg_amt: string (nullable = true)
 |-- legal_age: string (nullable = true)



In [11]:
ori_account.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- offshore: string (nullable = true)
 |-- open_date: string (nullable = true)
 |-- close_date: string (nullable = true)



In [6]:
#sql  所有交易详情数据，包含确定label的交易
ori_trx_new.registerTempTable('ori_trx_new') #注册临时表
ori_trx_new_full = spark.sql(
"""
    select
            trx_id,
            cust_id,
            target_cust_id,
            flag_crdr,
            trx_amt,
            balance_amt,
            trx_date,
            trx_time,
            u_trx_time,

        -- 30min
        count(*) over(w_1800) as cnt_1800,
        collect_list(concat_ws('*',trx_amt,u_trx_time)) over(w_1800) as amt_1800,
        collect_list(concat_ws('*',balance_amt,u_trx_time)) over(w_1800) as balance_amt_1800,
        collect_list(concat_ws('*', u_trx_time)) over(w_1800) as u_time_1800,

        sum(if(flag_crdr=0,1,0)) over(w_1800) as crdr_0_1800,
        sum(if(flag_crdr=1,1,0)) over(w_1800) as crdr_1_1800,

        -- 1h
        count(*) over(w_3600) as cnt_3600,
        collect_list(concat_ws('*',trx_amt,u_trx_time)) over(w_3600) as amt_3600,
        collect_list(concat_ws('*',balance_amt,u_trx_time)) over(w_3600) as balance_amt_3600,
        collect_list(concat_ws('*', u_trx_time)) over(w_3600) as u_time_3600,

        sum(if(flag_crdr=0,1,0)) over(w_3600) as crdr_0_3600,
        sum(if(flag_crdr=1,1,0)) over(w_3600) as crdr_1_3600,

         -- 1天
        count(*) over(w_86400) as cnt_86400,
        collect_list(concat_ws('*',trx_amt,u_trx_time)) over(w_86400) as amt_86400,
        collect_list(concat_ws('*',balance_amt,u_trx_time)) over(w_86400) as balance_amt_86400,
        collect_list(concat_ws('*', u_trx_time)) over(w_86400) as u_time_86400,

        sum(if(flag_crdr=0,1,0)) over(w_86400) as crdr_0_86400,
        sum(if(flag_crdr=1,1,0)) over(w_86400) as crdr_1_86400,

         -- 7天
        count(*) over(w_604800) as cnt_604800,
        collect_list(concat_ws('*',trx_amt,u_trx_time)) over(w_604800) as amt_604800,
        collect_list(concat_ws('*',balance_amt,u_trx_time)) over(w_604800) as balance_amt_604800,
        collect_list(concat_ws('*', u_trx_time)) over(w_604800) as u_time_604800,

        sum(if(flag_crdr=0,1,0)) over(w_604800) as crdr_0_604800,
        sum(if(flag_crdr=1,1,0)) over(w_604800) as crdr_1_604800,

         -- 14天
        count(*) over(w_1209600) as cnt_1209600,
        collect_list(concat_ws('*',trx_amt,u_trx_time)) over(w_1209600) as amt_1209600,
        collect_list(concat_ws('*',balance_amt,u_trx_time)) over(w_1209600) as balance_amt_1209600,
        collect_list(concat_ws('*', u_trx_time)) over(w_1209600) as u_time_1209600,

        sum(if(flag_crdr=0,1,0)) over(w_1209600) as crdr_0_1209600,
        sum(if(flag_crdr=1,1,0)) over(w_1209600) as crdr_1_1209600,

         -- 30天
        count(*) over(w_2592000) as cnt_2592000,
        collect_list(concat_ws('*',trx_amt,u_trx_time)) over(w_2592000) as amt_2592000,
        collect_list(concat_ws('*',balance_amt,u_trx_time)) over(w_2592000) as balance_amt_2592000,
        collect_list(concat_ws('*', u_trx_time)) over(w_2592000) as u_time_2592000,

        sum(if(flag_crdr=0,1,0)) over(w_2592000) as crdr_0_2592000,
        sum(if(flag_crdr=1,1,0)) over(w_2592000) as crdr_1_2592000
        
        

    from(
            select
                trx_id,
                cust_id,
                target_cust_id,
                flag_crdr,
                trx_amt,
                balance_amt,
                trx_date,
                trx_time,
                unix_timestamp(concat(trx_date, ' ', trx_time )) as u_trx_time
            from ori_trx_new

    )
    window
        -- w_300 as (partition by cust_id order by u_trx_time range between 300 preceding and 300 following), --5min
        w_1800 as (partition by cust_id order by u_trx_time range between 1800 preceding and 1800 following), --30min
        w_3600 as (partition by cust_id order by u_trx_time range between 3600 preceding and 3600 following), --1h
        -- w_7200 as (partition by cust_id order by u_trx_time range between 7200 preceding and 7200 following),--2h
        -- w_28800 as (partition by cust_id order by u_trx_time range between 28800 preceding and 28800 following),--8h
        w_86400 as (partition by cust_id order by u_trx_time range between 86400 preceding and 86400 following),--1天
        w_604800 as (partition by cust_id order by u_trx_time range between 604800 preceding and 604800 following),--7天
        w_1209600 as (partition by cust_id order by u_trx_time range between 1209600 preceding and 1209600 following),--14天
        w_2592000 as (partition by cust_id order by u_trx_time range between 2592000 preceding and 2592000 following)--30天

"""

)

In [7]:
ori_trx_new_full.columns

['trx_id',
 'cust_id',
 'target_cust_id',
 'flag_crdr',
 'trx_amt',
 'balance_amt',
 'trx_date',
 'trx_time',
 'u_trx_time',
 'cnt_1800',
 'amt_1800',
 'balance_amt_1800',
 'u_time_1800',
 'crdr_0_1800',
 'crdr_1_1800',
 'cnt_3600',
 'amt_3600',
 'balance_amt_3600',
 'u_time_3600',
 'crdr_0_3600',
 'crdr_1_3600',
 'cnt_86400',
 'amt_86400',
 'balance_amt_86400',
 'u_time_86400',
 'crdr_0_86400',
 'crdr_1_86400',
 'cnt_604800',
 'amt_604800',
 'balance_amt_604800',
 'u_time_604800',
 'crdr_0_604800',
 'crdr_1_604800',
 'cnt_1209600',
 'amt_1209600',
 'balance_amt_1209600',
 'u_time_1209600',
 'crdr_0_1209600',
 'crdr_1_1209600',
 'cnt_2592000',
 'amt_2592000',
 'balance_amt_2592000',
 'u_time_2592000',
 'crdr_0_2592000',
 'crdr_1_2592000']

In [8]:
from pyspark.sql import Row
ax = [Row(
    trx_id='trx_7a3b1bb7-a14a-4f5e-8738-ea7f8941b79a', 
    cust_id='cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272', 
    target_cust_id='cust_persion__a2fd0a79-6d0d-4cfc-b9fa-df0879eca080', 
    flag_crdr='0', 
    trx_amt='12471505.3357', 
    balance_amt='21549400.9814', 
    trx_date='2016-01-01', 
    trx_time='00:24:20', 
    u_trx_time=1451607860, 
    cnt_1800=1, 
    amt_1800=['12471505.3357*1451607860'], 
    balance_amt_1800=['21549400.9814*1451607860'], 
    u_time_1800=['1451607860'], 
    crdr_0_1800=1, 
    crdr_1_1800=0, 
    cnt_3600=1, 
    amt_3600=['12471505.3357*1451607860'], 
    balance_amt_3600=['21549400.9814*1451607860'], u_time_3600=['1451607860'], crdr_0_3600=1, crdr_1_3600=0, cnt_86400=24, amt_86400=['12471505.3357*1451607860', '78809477.6749*1451611951', '67317155.4321*1451614183', '67317155.4321*1451614183', '97672665.4793*1451616403', '97672665.4793*1451616403', '72264109.8265*1451623350', '89669411.1102*1451625058', '88556416.6689*1451642296', '88556416.6689*1451642296', '91545230.2289*1451646892', '91545230.2289*1451646892', '77146413.4146*1451647352', '22321592.7839*1451654044', '72215878.7465*1451656253', '72215878.7465*1451656253', '90251522.0652*1451656320', '73038389.1455*1451657744', '73038389.1455*1451657744', '73038389.1455*1451657744', '67834526.2523*1451684342', '67834526.2523*1451684342', '1785326.45646*1451687279', '1785326.45646*1451687279'], balance_amt_86400=['21549400.9814*1451607860', '60401992.2174*1451611951', '70215158.8144*1451614183', '70215158.8144*1451614183', '99374367.9318*1451616403', '99374367.9318*1451616403', '91843278.6796*1451623350', '44828311.9331*1451625058', '11100036.1124*1451642296', '11100036.1124*1451642296', '839759.974617*1451646892', '839759.974617*1451646892', '90755785.0398*1451647352', '66951727.7947*1451654044', '26063083.9318*1451656253', '26063083.9318*1451656253', '29791574.9576*1451656320', '42899408.762*1451657744', '42899408.762*1451657744', '42899408.762*1451657744', '77082947.4708*1451684342', '77082947.4708*1451684342', '22563996.1981*1451687279', '22563996.1981*1451687279'], u_time_86400=['1451607860', '1451611951', '1451614183', '1451614183', '1451616403', '1451616403', '1451623350', '1451625058', '1451642296', '1451642296', '1451646892', '1451646892', '1451647352', '1451654044', '1451656253', '1451656253', '1451656320', '1451657744', '1451657744', '1451657744', '1451684342', '1451684342', '1451687279', '1451687279'], crdr_0_86400=12, crdr_1_86400=12, cnt_604800=249, amt_604800=['12471505.3357*1451607860', '78809477.6749*1451611951', '67317155.4321*1451614183', '67317155.4321*1451614183', '97672665.4793*1451616403', '97672665.4793*1451616403', '72264109.8265*1451623350', '89669411.1102*1451625058', '88556416.6689*1451642296', '88556416.6689*1451642296', '91545230.2289*1451646892', '91545230.2289*1451646892', '77146413.4146*1451647352', '22321592.7839*1451654044', '72215878.7465*1451656253', '72215878.7465*1451656253', '90251522.0652*1451656320', '73038389.1455*1451657744', '73038389.1455*1451657744', '73038389.1455*1451657744', '67834526.2523*1451684342', '67834526.2523*1451684342', '1785326.45646*1451687279', '1785326.45646*1451687279', '17136968.1802*1451697649', '17136968.1802*1451697649', '3374289.69795*1451700694', '3374289.69795*1451700694', '3374289.69795*1451700694', '7060522.08945*1451704165', '273818.318591*1451710949', '273818.318591*1451710949', '50458366.6304*1451713915', '89525055.1045*1451714043', '89525055.1045*1451714043', '89525055.1045*1451714043', '89525055.1045*1451714043', '83372957.6784*1451727555', '37589086.961*1451730738', '18852484.0632*1451735657', '18852484.0632*1451735657', '36053624.0149*1451737630', '34918676.4457*1451739437', '34918676.4457*1451739437', '34918676.4457*1451739437', '35853187.1554*1451744337', '49440869.4862*1451744398', '19196263.37*1451752723', '1784056.51043*1451754280', '1784056.51043*1451754280', '1784056.51043*1451754280', '1784056.51043*1451754280', '43363610.5017*1451762811', '43363610.5017*1451762811', '43363610.5017*1451762811', '52683411.1171*1451763169', '17191435.568*1451763638', '3563459.35301*1451778692', '33761845.3375*1451783625', '33761845.3375*1451783625', '33761845.3375*1451783625', '24876786.8751*1451786023', '23463873.3936*1451798070', '23463873.3936*1451798070', '43653135.5523*1451798752', '43653135.5523*1451798752', '29008839.1511*1451800725', '59186620.7815*1451803874', '59186620.7815*1451803874', '28345059.2314*1451803879', '70065803.4736*1451813728', '35702005.5204*1451816310', '35702005.5204*1451816310', '35702005.5204*1451816310', '13132259.2355*1451816738', '13132259.2355*1451816738', '13132259.2355*1451816738', '33002083.7483*1451837092', '60233551.7856*1451842707', '15751307.8532*1451844588', '15751307.8532*1451844588', '15751307.8532*1451844588', '77070101.1854*1451849291', '77070101.1854*1451849291', '86941290.3878*1451858456', '69744118.3656*1451860532', '69744118.3656*1451860532', '95828115.1999*1451863272', '38887644.1068*1451866109', '38887644.1068*1451866109', '38887644.1068*1451866109', '79275078.3014*1451869294', '6905548.52172*1451871549', '6905548.52172*1451871549', '34428688.5526*1451874436', '34428688.5526*1451874436', '34428688.5526*1451874436', '12938394.5478*1451881008', '12938394.5478*1451881008', '86436996.7443*1451893789', '86436996.7443*1451893789', '64657142.258*1451897630', '64657142.258*1451897630', '64657142.258*1451897630', '74486802.2976*1451905921', '74486802.2976*1451905921', '74486802.2976*1451905921', '18721749.0222*1451906236', '11325939.4693*1451906285', '25065557.914*1451910519', '25065557.914*1451910519', '25065557.914*1451910519', '25065557.914*1451910519', '22835110.0577*1451910908', '32251480.2938*1451921545', '34106307.8046*1451921618', '34106307.8046*1451921618', '48228385.3959*1451924664', '48838735.9788*1451937070', '48838735.9788*1451937070', '51118025.2697*1451937986', '53804986.1216*1451940275', '53804986.1216*1451940275', '53804986.1216*1451940275', '41781499.7678*1451940407', '41781499.7678*1451940407', '46975494.5064*1451947798', '46975494.5064*1451947798', '46975494.5064*1451947798', '95000641.5339*1451948169', '95000641.5339*1451948169', '95000641.5339*1451948169', '93165958.6984*1451949746', '93165958.6984*1451949746', '46824110.3968*1451950245', '46824110.3968*1451950245', '55588098.1674*1451954552', '60626970.398*1451954918', '17475790.8939*1451955575', '17475790.8939*1451955575', '17475790.8939*1451955575', '46481601.6725*1451965797', '46481601.6725*1451965797', '1357944.09872*1451966880', '1357944.09872*1451966880', '42678397.402*1451969401', '42678397.402*1451969401', '80798066.3313*1451970969', '65109648.6329*1451972489', '28307366.505*1451982108', '2248485.43819*1451982880', '2248485.43819*1451982880', '68772700.1787*1451986832', '9094888.5806*1451990236', '44923134.0054*1451994509', '44923134.0054*1451994509', '73652993.2372*1451995791', '88602665.39*1451996117', '88602665.39*1451996117', '88602665.39*1451996117', '51125324.7342*1451997801', '51125324.7342*1451997801', '27293598.7805*1452000769', '31182233.709*1452007063', '38557045.5662*1452008988', '38557045.5662*1452008988', '38557045.5662*1452008988', '36527483.0752*1452010149', '54765591.5953*1452010591', '54765591.5953*1452010591', '54765591.5953*1452010591', '39610214.4729*1452013132', '63073189.1827*1452028034', '93820344.5575*1452041765', '32478322.5091*1452047038', '87426316.3732*1452054902', '87426316.3732*1452054902', '87426316.3732*1452054902', '87426316.3732*1452054902', '8160560.52548*1452056349', '8160560.52548*1452056349', '27534271.8782*1452058953', '27534271.8782*1452058953', '1764485.37125*1452067003', '1764485.37125*1452067003', '1764485.37125*1452067003', '1764485.37125*1452067003', '89794309.9256*1452069224', '31905725.2591*1452073276', '31905725.2591*1452073276', '47030293.3045*1452076829', '47030293.3045*1452076829', '47030293.3045*1452076829', '47030293.3045*1452076829', '4883487.86866*1452080674', '4883487.86866*1452080674', '95153652.9705*1452099865', '68991183.7462*1452113567', '68991183.7462*1452113567', '10013694.0752*1452119398', '22757413.6641*1452119971', '22757413.6641*1452119971', '38984712.0692*1452125095', '38984712.0692*1452125095', '27001775.7996*1452125623', '27001775.7996*1452125623', '27001775.7996*1452125623', '68721710.0389*1452128987', '30451616.1309*1452129494', '43640357.817*1452134995', '43640357.817*1452134995', '42025696.9398*1452140012', '42025696.9398*1452140012', '57982742.5497*1452141507', '57982742.5497*1452141507', '57982742.5497*1452141507', '76269481.7453*1452142949', '21088702.7842*1452142954', '21088702.7842*1452142954', '21088702.7842*1452142954', '494012.891686*1452144735', '53332591.8915*1452150718', '92323689.7424*1452156026', '43797371.5834*1452164189', '61820440.5636*1452167835', '61820440.5636*1452167835', '28770485.9873*1452168758', '38574126.9816*1452174023', '38574126.9816*1452174023', '38574126.9816*1452174023', '81974798.4459*1452174336', '81974798.4459*1452174336', '95539075.4215*1452175467', '95539075.4215*1452175467', '95539075.4215*1452175467', '52889443.076*1452180614', '52889443.076*1452180614', '82881264.4604*1452190462', '8418037.73131*1452195076', '8418037.73131*1452195076', '8418037.73131*1452195076', '81139663.628*1452196091', '81139663.628*1452196091', '81139663.628*1452196091', '3324350.4493*1452202758', '16198297.0253*1452206051', '32828024.8871*1452208898', '32828024.8871*1452208898', '32828024.8871*1452208898'], balance_amt_604800=['21549400.9814*1451607860', '60401992.2174*1451611951', '70215158.8144*1451614183', '70215158.8144*1451614183', '99374367.9318*1451616403', '99374367.9318*1451616403', '91843278.6796*1451623350', '44828311.9331*1451625058', '11100036.1124*1451642296', '11100036.1124*1451642296', '839759.974617*1451646892', '839759.974617*1451646892', '90755785.0398*1451647352', '66951727.7947*1451654044', '26063083.9318*1451656253', '26063083.9318*1451656253', '29791574.9576*1451656320', '42899408.762*1451657744', '42899408.762*1451657744', '42899408.762*1451657744', '77082947.4708*1451684342', '77082947.4708*1451684342', '22563996.1981*1451687279', '22563996.1981*1451687279', '51795715.1239*1451697649', '51795715.1239*1451697649', '45527212.31*1451700694', '45527212.31*1451700694', '45527212.31*1451700694', '10682201.3541*1451704165', '54257515.4436*1451710949', '54257515.4436*1451710949', '99518743.8099*1451713915', '87598806.3534*1451714043', '87598806.3534*1451714043', '87598806.3534*1451714043', '87598806.3534*1451714043', '48166326.9459*1451727555', '22783857.6912*1451730738', '24018531.8862*1451735657', '24018531.8862*1451735657', '80830511.9624*1451737630', '94168032.0617*1451739437', '94168032.0617*1451739437', '94168032.0617*1451739437', '65497964.3675*1451744337', '48162626.1282*1451744398', '25558409.3094*1451752723', '46826589.7538*1451754280', '46826589.7538*1451754280', '46826589.7538*1451754280', '46826589.7538*1451754280', '32798096.704*1451762811', '32798096.704*1451762811', '32798096.704*1451762811', '39472240.9442*1451763169', '22517648.4138*1451763638', '17352821.9793*1451778692', '51365492.5152*1451783625', '51365492.5152*1451783625', '51365492.5152*1451783625', '68815725.0423*1451786023', '23000816.569*1451798070', '23000816.569*1451798070', '55414602.3953*1451798752', '55414602.3953*1451798752', '34506568.0672*1451800725', '4669892.60012*1451803874', '4669892.60012*1451803874', '34371388.6558*1451803879', '93636937.3579*1451813728', '64521093.5878*1451816310', '64521093.5878*1451816310', '64521093.5878*1451816310', '37831009.0785*1451816738', '37831009.0785*1451816738', '37831009.0785*1451816738', '96537698.7096*1451837092', '89316369.3734*1451842707', '46209625.4203*1451844588', '46209625.4203*1451844588', '46209625.4203*1451844588', '7514041.8878*1451849291', '7514041.8878*1451849291', '54672341.8154*1451858456', '96834449.7728*1451860532', '96834449.7728*1451860532', '69145424.2992*1451863272', '36104080.8776*1451866109', '36104080.8776*1451866109', '36104080.8776*1451866109', '41085034.3379*1451869294', '17444936.5291*1451871549', '17444936.5291*1451871549', '15354450.0987*1451874436', '15354450.0987*1451874436', '15354450.0987*1451874436', '4274383.96983*1451881008', '4274383.96983*1451881008', '48174124.149*1451893789', '48174124.149*1451893789', '36858583.3293*1451897630', '36858583.3293*1451897630', '36858583.3293*1451897630', '37597903.879*1451905921', '37597903.879*1451905921', '37597903.879*1451905921', '94829762.865*1451906236', '77684304.352*1451906285', '39112700.965*1451910519', '39112700.965*1451910519', '39112700.965*1451910519', '39112700.965*1451910519', '85430569.4661*1451910908', '19027172.2921*1451921545', '70062509.7754*1451921618', '70062509.7754*1451921618', '91432736.7086*1451924664', '47700578.5858*1451937070', '47700578.5858*1451937070', '6279255.30081*1451937986', '53481792.4888*1451940275', '53481792.4888*1451940275', '53481792.4888*1451940275', '46961417.8475*1451940407', '46961417.8475*1451940407', '52410123.2217*1451947798', '52410123.2217*1451947798', '52410123.2217*1451947798', '30491901.7747*1451948169', '30491901.7747*1451948169', '30491901.7747*1451948169', '1015330.40217*1451949746', '1015330.40217*1451949746', '73075675.6549*1451950245', '73075675.6549*1451950245', '54907992.2002*1451954552', '78446793.1618*1451954918', '80455421.9752*1451955575', '80455421.9752*1451955575', '80455421.9752*1451955575', '61777895.9127*1451965797', '61777895.9127*1451965797', '2972832.64437*1451966880', '2972832.64437*1451966880', '30492662.4652*1451969401', '30492662.4652*1451969401', '90608159.8886*1451970969', '7256928.32997*1451972489', '99425008.0265*1451982108', '45720434.4438*1451982880', '45720434.4438*1451982880', '61424188.5626*1451986832', '8860811.05248*1451990236', '68743640.5353*1451994509', '68743640.5353*1451994509', '97609260.2419*1451995791', '87110435.3697*1451996117', '87110435.3697*1451996117', '87110435.3697*1451996117', '11104671.9594*1451997801', '11104671.9594*1451997801', '54669431.4024*1452000769', '84537046.263*1452007063', '7430176.26206*1452008988', '7430176.26206*1452008988', '7430176.26206*1452008988', '47464333.857*1452010149', '31496576.9687*1452010591', '31496576.9687*1452010591', '31496576.9687*1452010591', '3707140.02928*1452013132', '10084105.1771*1452028034', '50723345.6296*1452041765', '27239091.9753*1452047038', '55974982.1024*1452054902', '55974982.1024*1452054902', '55974982.1024*1452054902', '55974982.1024*1452054902', '76021526.2399*1452056349', '76021526.2399*1452056349', '32928203.3214*1452058953', '32928203.3214*1452058953', '63719691.099*1452067003', '63719691.099*1452067003', '63719691.099*1452067003', '63719691.099*1452067003', '50694184.4346*1452069224', '76384095.2107*1452073276', '76384095.2107*1452073276', '40752421.0325*1452076829', '40752421.0325*1452076829', '40752421.0325*1452076829', '40752421.0325*1452076829', '81741865.0156*1452080674', '81741865.0156*1452080674', '79125920.7319*1452099865', '84435816.85*1452113567', '84435816.85*1452113567', '77583899.5541*1452119398', '43206954.0652*1452119971', '43206954.0652*1452119971', '25207370.228*1452125095', '25207370.228*1452125095', '49091809.3248*1452125623', '49091809.3248*1452125623', '49091809.3248*1452125623', '58145176.0326*1452128987', '81665440.1836*1452129494', '94542951.6769*1452134995', '94542951.6769*1452134995', '30211066.8193*1452140012', '30211066.8193*1452140012', '2864661.88916*1452141507', '2864661.88916*1452141507', '2864661.88916*1452141507', '47921801.6018*1452142949', '2075678.25453*1452142954', '2075678.25453*1452142954', '2075678.25453*1452142954', '23083298.4434*1452144735', '25404975.963*1452150718', '39222387.4433*1452156026', '6901151.34368*1452164189', '16623777.4843*1452167835', '16623777.4843*1452167835', '82974276.0334*1452168758', '62336814.4617*1452174023', '62336814.4617*1452174023', '62336814.4617*1452174023', '72395289.9152*1452174336', '72395289.9152*1452174336', '38059552.3137*1452175467', '38059552.3137*1452175467', '38059552.3137*1452175467', '29942766.2363*1452180614', '29942766.2363*1452180614', '63365608.0821*1452190462', '85456964.0583*1452195076', '85456964.0583*1452195076', '85456964.0583*1452195076', '64451579.2426*1452196091', '64451579.2426*1452196091', '64451579.2426*1452196091', '81539727.7963*1452202758', '13348292.3602*1452206051', '79810098.7262*1452208898', '79810098.7262*1452208898', '79810098.7262*1452208898'], u_time_604800=['1451607860', '1451611951', '1451614183', '1451614183', '1451616403', '1451616403', '1451623350', '1451625058', '1451642296', '1451642296', '1451646892', '1451646892', '1451647352', '1451654044', '1451656253', '1451656253', '1451656320', '1451657744', '1451657744', '1451657744', '1451684342', '1451684342', '1451687279', '1451687279', '1451697649', '1451697649', '1451700694', '1451700694', '1451700694', '1451704165', '1451710949', '1451710949', '1451713915', '1451714043', '1451714043', '1451714043', '1451714043', '1451727555', '1451730738', '1451735657', '1451735657', '1451737630', '1451739437', '1451739437', '1451739437', '1451744337', '1451744398', '1451752723', '1451754280', '1451754280', '1451754280', '1451754280', '1451762811', '1451762811', '1451762811', '1451763169', '1451763638', '1451778692', '1451783625', '1451783625', '1451783625', '1451786023', '1451798070', '1451798070', '1451798752', '1451798752', '1451800725', '1451803874', '1451803874', '1451803879', '1451813728', '1451816310', '1451816310', '1451816310', '1451816738', '1451816738', '1451816738', '1451837092', '1451842707', '1451844588', '1451844588', '1451844588', '1451849291', '1451849291', '1451858456', '1451860532', '1451860532', '1451863272', '1451866109', '1451866109', '1451866109', '1451869294', '1451871549', '1451871549', '1451874436', '1451874436', '1451874436', '1451881008', '1451881008', '1451893789', '1451893789', '1451897630', '1451897630', '1451897630', '1451905921', '1451905921', '1451905921', '1451906236', '1451906285', '1451910519', '1451910519', '1451910519', '1451910519', '1451910908', '1451921545', '1451921618', '1451921618', '1451924664', '1451937070', '1451937070', '1451937986', '1451940275', '1451940275', '1451940275', '1451940407', '1451940407', '1451947798', '1451947798', '1451947798', '1451948169', '1451948169', '1451948169', '1451949746', '1451949746', '1451950245', '1451950245', '1451954552', '1451954918', '1451955575', '1451955575', '1451955575', '1451965797', '1451965797', '1451966880', '1451966880', '1451969401', '1451969401', '1451970969', '1451972489', '1451982108', '1451982880', '1451982880', '1451986832', '1451990236', '1451994509', '1451994509', '1451995791', '1451996117', '1451996117', '1451996117', '1451997801', '1451997801', '1452000769', '1452007063', '1452008988', '1452008988', '1452008988', '1452010149', '1452010591', '1452010591', '1452010591', '1452013132', '1452028034', '1452041765', '1452047038', '1452054902', '1452054902', '1452054902', '1452054902', '1452056349', '1452056349', '1452058953', '1452058953', '1452067003', '1452067003', '1452067003', '1452067003', '1452069224', '1452073276', '1452073276', '1452076829', '1452076829', '1452076829', '1452076829', '1452080674', '1452080674', '1452099865', '1452113567', '1452113567', '1452119398', '1452119971', '1452119971', '1452125095', '1452125095', '1452125623', '1452125623', '1452125623', '1452128987', '1452129494', '1452134995', '1452134995', '1452140012', '1452140012', '1452141507', '1452141507', '1452141507', '1452142949', '1452142954', '1452142954', '1452142954', '1452144735', '1452150718', '1452156026', '1452164189', '1452167835', '1452167835', '1452168758', '1452174023', '1452174023', '1452174023', '1452174336', '1452174336', '1452175467', '1452175467', '1452175467', '1452180614', '1452180614', '1452190462', '1452195076', '1452195076', '1452195076', '1452196091', '1452196091', '1452196091', '1452202758', '1452206051', '1452208898', '1452208898', '1452208898'], crdr_0_604800=149, crdr_1_604800=100, cnt_1209600=525, amt_1209600=['12471505.3357*1451607860', '78809477.6749*1451611951', '67317155.4321*1451614183', '67317155.4321*1451614183', '97672665.4793*1451616403', '97672665.4793*1451616403', '72264109.8265*1451623350', '89669411.1102*1451625058', '88556416.6689*1451642296', '88556416.6689*1451642296', '91545230.2289*1451646892', '91545230.2289*1451646892', '77146413.4146*1451647352', '22321592.7839*1451654044', '72215878.7465*1451656253', '72215878.7465*1451656253', '90251522.0652*1451656320', '73038389.1455*1451657744', '73038389.1455*1451657744', '73038389.1455*1451657744', '67834526.2523*1451684342', '67834526.2523*1451684342', '1785326.45646*1451687279', '1785326.45646*1451687279', '17136968.1802*1451697649', '17136968.1802*1451697649', '3374289.69795*1451700694', '3374289.69795*1451700694', '3374289.69795*1451700694', '7060522.08945*1451704165', '273818.318591*1451710949', '273818.318591*1451710949', '50458366.6304*1451713915', '89525055.1045*1451714043', '89525055.1045*1451714043', '89525055.1045*1451714043', '89525055.1045*1451714043', '83372957.6784*1451727555', '37589086.961*1451730738', '18852484.0632*1451735657', '18852484.0632*1451735657', '36053624.0149*1451737630', '34918676.4457*1451739437', '34918676.4457*1451739437', '34918676.4457*1451739437', '35853187.1554*1451744337', '49440869.4862*1451744398', '19196263.37*1451752723', '1784056.51043*1451754280', '1784056.51043*1451754280', '1784056.51043*1451754280', '1784056.51043*1451754280', '43363610.5017*1451762811', '43363610.5017*1451762811', '43363610.5017*1451762811', '52683411.1171*1451763169', '17191435.568*1451763638', '3563459.35301*1451778692', '33761845.3375*1451783625', '33761845.3375*1451783625', '33761845.3375*1451783625', '24876786.8751*1451786023', '23463873.3936*1451798070', '23463873.3936*1451798070', '43653135.5523*1451798752', '43653135.5523*1451798752', '29008839.1511*1451800725', '59186620.7815*1451803874', '59186620.7815*1451803874', '28345059.2314*1451803879', '70065803.4736*1451813728', '35702005.5204*1451816310', '35702005.5204*1451816310', '35702005.5204*1451816310', '13132259.2355*1451816738', '13132259.2355*1451816738', '13132259.2355*1451816738', '33002083.7483*1451837092', '60233551.7856*1451842707', '15751307.8532*1451844588', '15751307.8532*1451844588', '15751307.8532*1451844588', '77070101.1854*1451849291', '77070101.1854*1451849291', '86941290.3878*1451858456', '69744118.3656*1451860532', '69744118.3656*1451860532', '95828115.1999*1451863272', '38887644.1068*1451866109', '38887644.1068*1451866109', '38887644.1068*1451866109', '79275078.3014*1451869294', '6905548.52172*1451871549', '6905548.52172*1451871549', '34428688.5526*1451874436', '34428688.5526*1451874436', '34428688.5526*1451874436', '12938394.5478*1451881008', '12938394.5478*1451881008', '86436996.7443*1451893789', '86436996.7443*1451893789', '64657142.258*1451897630', '64657142.258*1451897630', '64657142.258*1451897630', '74486802.2976*1451905921', '74486802.2976*1451905921', '74486802.2976*1451905921', '18721749.0222*1451906236', '11325939.4693*1451906285', '25065557.914*1451910519', '25065557.914*1451910519', '25065557.914*1451910519', '25065557.914*1451910519', '22835110.0577*1451910908', '32251480.2938*1451921545', '34106307.8046*1451921618', '34106307.8046*1451921618', '48228385.3959*1451924664', '48838735.9788*1451937070', '48838735.9788*1451937070', '51118025.2697*1451937986', '53804986.1216*1451940275', '53804986.1216*1451940275', '53804986.1216*1451940275', '41781499.7678*1451940407', '41781499.7678*1451940407', '46975494.5064*1451947798', '46975494.5064*1451947798', '46975494.5064*1451947798', '95000641.5339*1451948169', '95000641.5339*1451948169', '95000641.5339*1451948169', '93165958.6984*1451949746', '93165958.6984*1451949746', '46824110.3968*1451950245', '46824110.3968*1451950245', '55588098.1674*1451954552', '60626970.398*1451954918', '17475790.8939*1451955575', '17475790.8939*1451955575', '17475790.8939*1451955575', '46481601.6725*1451965797', '46481601.6725*1451965797', '1357944.09872*1451966880', '1357944.09872*1451966880', '42678397.402*1451969401', '42678397.402*1451969401', '80798066.3313*1451970969', '65109648.6329*1451972489', '28307366.505*1451982108', '2248485.43819*1451982880', '2248485.43819*1451982880', '68772700.1787*1451986832', '9094888.5806*1451990236', '44923134.0054*1451994509', '44923134.0054*1451994509', '73652993.2372*1451995791', '88602665.39*1451996117', '88602665.39*1451996117', '88602665.39*1451996117', '51125324.7342*1451997801', '51125324.7342*1451997801', '27293598.7805*1452000769', '31182233.709*1452007063', '38557045.5662*1452008988', '38557045.5662*1452008988', '38557045.5662*1452008988', '36527483.0752*1452010149', '54765591.5953*1452010591', '54765591.5953*1452010591', '54765591.5953*1452010591', '39610214.4729*1452013132', '63073189.1827*1452028034', '93820344.5575*1452041765', '32478322.5091*1452047038', '87426316.3732*1452054902', '87426316.3732*1452054902', '87426316.3732*1452054902', '87426316.3732*1452054902', '8160560.52548*1452056349', '8160560.52548*1452056349', '27534271.8782*1452058953', '27534271.8782*1452058953', '1764485.37125*1452067003', '1764485.37125*1452067003', '1764485.37125*1452067003', '1764485.37125*1452067003', '89794309.9256*1452069224', '31905725.2591*1452073276', '31905725.2591*1452073276', '47030293.3045*1452076829', '47030293.3045*1452076829', '47030293.3045*1452076829', '47030293.3045*1452076829', '4883487.86866*1452080674', '4883487.86866*1452080674', '95153652.9705*1452099865', '68991183.7462*1452113567', '68991183.7462*1452113567', '10013694.0752*1452119398', '22757413.6641*1452119971', '22757413.6641*1452119971', '38984712.0692*1452125095', '38984712.0692*1452125095', '27001775.7996*1452125623', '27001775.7996*1452125623', '27001775.7996*1452125623', '68721710.0389*1452128987', '30451616.1309*1452129494', '43640357.817*1452134995', '43640357.817*1452134995', '42025696.9398*1452140012', '42025696.9398*1452140012', '57982742.5497*1452141507', '57982742.5497*1452141507', '57982742.5497*1452141507', '76269481.7453*1452142949', '21088702.7842*1452142954', '21088702.7842*1452142954', '21088702.7842*1452142954', '494012.891686*1452144735', '53332591.8915*1452150718', '92323689.7424*1452156026', '43797371.5834*1452164189', '61820440.5636*1452167835', '61820440.5636*1452167835', '28770485.9873*1452168758', '38574126.9816*1452174023', '38574126.9816*1452174023', '38574126.9816*1452174023', '81974798.4459*1452174336', '81974798.4459*1452174336', '95539075.4215*1452175467', '95539075.4215*1452175467', '95539075.4215*1452175467', '52889443.076*1452180614', '52889443.076*1452180614', '82881264.4604*1452190462', '8418037.73131*1452195076', '8418037.73131*1452195076', '8418037.73131*1452195076', '81139663.628*1452196091', '81139663.628*1452196091', '81139663.628*1452196091', '3324350.4493*1452202758', '16198297.0253*1452206051', '32828024.8871*1452208898', '32828024.8871*1452208898', '32828024.8871*1452208898', '39519338.2479*1452214123', '39519338.2479*1452214123', '39519338.2479*1452214123', '89991721.2313*1452216360', '89991721.2313*1452216360', '34900636.8967*1452218280', '79136577.5331*1452219471', '79136577.5331*1452219471', '83683638.2111*1452223485', '50208103.8346*1452227880', '88352795.6703*1452231411', '88352795.6703*1452231411', '7268095.7213*1452240664', '78076249.7288*1452244765', '24246248.4543*1452253151', '56245228.4508*1452260276', '56245228.4508*1452260276', '88054133.9195*1452273534', '27599359.6395*1452280846', '27599359.6395*1452280846', '78893492.4727*1452282201', '96128463.9571*1452287217', '47000955.15*1452287606', '11163625.1241*1452287620', '11163625.1241*1452287620', '11163625.1241*1452287620', '11163625.1241*1452287620', '79118365.6654*1452288867', '64575864.7724*1452288890', '82146649.5574*1452301670', '474263.060471*1452303377', '474263.060471*1452303377', '2876679.81848*1452316951', '24577662.0889*1452324397', '62093151.3611*1452327342', '62093151.3611*1452327342', '83598740.4709*1452333329', '83598740.4709*1452333329', '47586954.4082*1452334132', '47586954.4082*1452334132', '33387559.5508*1452336906', '33387559.5508*1452336906', '89307721.198*1452336993', '89307721.198*1452336993', '4324215.89677*1452337219', '4324215.89677*1452337219', '4324215.89677*1452337219', '44639511.4194*1452341096', '44639511.4194*1452341096', '44639511.4194*1452341096', '99213706.3456*1452343556', '15052537.241*1452345021', '13386742.3384*1452346423', '6558924.30821*1452348638', '3728533.28245*1452351161', '47732414.0443*1452353837', '76258893.8072*1452356249', '76258893.8072*1452356249', '21061324.0631*1452356280', '21061324.0631*1452356280', '21061324.0631*1452356280', '91327408.7351*1452360188', '95997987.1225*1452371127', '60220786.8947*1452372730', '72125373.4605*1452379508', '72125373.4605*1452379508', '72125373.4605*1452379508', '56067042.2541*1452383896', '56067042.2541*1452383896', '56067042.2541*1452383896', '49631360.6637*1452387967', '49631360.6637*1452387967', '66705576.3294*1452394703', '81408940.2247*1452398529', '55156846.4473*1452399973', '44884380.6695*1452405394', '44884380.6695*1452405394', '44884380.6695*1452405394', '46130751.5406*1452412119', '45374060.7508*1452413435', '45374060.7508*1452413435', '98219830.5049*1452420153', '98219830.5049*1452420153', '98219830.5049*1452420153', '59673433.3308*1452424028', '35944553.3003*1452425880', '35944553.3003*1452425880', '35944553.3003*1452425880', '65796453.9248*1452430786', '15805111.6006*1452438361', '15805111.6006*1452438361', '54344825.0706*1452438952', '54344825.0706*1452438952', '54344825.0706*1452438952', '7150146.41317*1452441177', '97204310.791*1452449327', '67146699.3385*1452451397', '67146699.3385*1452451397', '42086775.4655*1452458152', '42086775.4655*1452458152', '42086775.4655*1452458152', '42086775.4655*1452458152', '34065107.8024*1452462825', '86357890.9523*1452463430', '62053316.91*1452468868', '76283855.3999*1452480396', '21694077.713*1452492151', '38682793.9031*1452492711', '9744944.23327*1452495175', '20524051.2489*1452495313', '59400112.7312*1452501428', '59400112.7312*1452501428', '59400112.7312*1452501428', '79793635.291*1452505965', '7994716.35401*1452507859', '76449878.0104*1452525036', '76449878.0104*1452525036', '76449878.0104*1452525036', '35743551.8138*1452528419', '35743551.8138*1452528419', '35743551.8138*1452528419', '35743551.8138*1452528419', '18895235.187*1452531371', '18895235.187*1452531371', '18895235.187*1452531371', '2236692.50435*1452532683', '44963012.8462*1452536032', '64350517.9108*1452536984', '53270059.5622*1452537896', '55597612.4873*1452538235', '35230297.5699*1452539800', '35230297.5699*1452539800', '35230297.5699*1452539800', '66690115.8968*1452543678', '66690115.8968*1452543678', '37919081.5225*1452546192', '37919081.5225*1452546192', '37919081.5225*1452546192', '87082150.215*1452552002', '87082150.215*1452552002', '46036433.1997*1452555711', '98618098.0277*1452559315', '33456725.6204*1452560864', '88461887.4017*1452564626', '98339795.5973*1452566959', '98339795.5973*1452566959', '27251648.7792*1452567830', '27251648.7792*1452567830', '27251648.7792*1452567830', '91284914.4562*1452570571', '91284914.4562*1452570571', '64690112.7234*1452575051', '87053222.6233*1452576922', '72879583.9435*1452578263', '72879583.9435*1452578263', '72879583.9435*1452578263', '25235919.7262*1452581508', '88735224.264*1452581752', '88735224.264*1452581752', '88735224.264*1452581752', '57418741.5018*1452586837', '97974603.5916*1452595587', '97974603.5916*1452595587', '85599014.1729*1452599920', '54442582.8543*1452604122', '54442582.8543*1452604122', '94433488.6669*1452606334', '41989896.2369*1452611559', '26480862.4608*1452615816', '7676448.04519*1452616504', '63383817.9947*1452618393', '63383817.9947*1452618393', '63383817.9947*1452618393', '40067889.8285*1452633689', '40067889.8285*1452633689', '71910773.8536*1452634324', '42574884.9977*1452635199', '14994009.4493*1452635710', '14994009.4493*1452635710', '14994009.4493*1452635710', '93892116.2176*1452648070', '93892116.2176*1452648070', '93892116.2176*1452648070', '71650048.5034*1452650689', '71650048.5034*1452650689', '92564358.7065*1452652212', '64962667.5946*1452653775', '23124051.9048*1452655684', '23124051.9048*1452655684', '86465135.4894*1452659005', '86465135.4894*1452659005', '86465135.4894*1452659005', '37145871.8236*1452660679', '5427654.00078*1452665649', '59721738.9463*1452665946', '59721738.9463*1452665946', '59721738.9463*1452665946', '43357861.011*1452668562', '50177472.8671*1452672902', '50177472.8671*1452672902', '13727352.9834*1452677155', '25797748.3005*1452677206', '18007004.9258*1452680665', '1919177.12206*1452687642', '1919177.12206*1452687642', '1919177.12206*1452687642', '78833900.7855*1452690354', '91687610.5167*1452692188', '91687610.5167*1452692188', '91687610.5167*1452692188', '5226595.70546*1452693028', '8945988.96078*1452694779', '8945988.96078*1452694779', '74020466.3508*1452703682', '74020466.3508*1452703682', '5995672.96466*1452707474', '5995672.96466*1452707474', '5995672.96466*1452707474', '25184576.0823*1452714322', '25184576.0823*1452714322', '25184576.0823*1452714322', '85022200.0119*1452718442', '85022200.0119*1452718442', '85022200.0119*1452718442', '96950148.0491*1452727180', '96950148.0491*1452727180', '70147043.3536*1452732126', '70147043.3536*1452732126', '70147043.3536*1452732126', '39388379.1781*1452736674', '38395984.6584*1452741984', '38395984.6584*1452741984', '38395984.6584*1452741984', '38395984.6584*1452741984', '79812867.3068*1452747674', '79812867.3068*1452747674', '79812867.3068*1452747674', '9439599.56143*1452753019', '9439599.56143*1452753019', '9439599.56143*1452753019', '56778980.3892*1452753785', '85958842.3213*1452755328', '42730607.3656*1452756579', '37324204.148*1452757700', '37324204.148*1452757700', '37324204.148*1452757700', '29140188.6074*1452763603', '29140188.6074*1452763603', '29140188.6074*1452763603', '60956449.5503*1452766808', '60956449.5503*1452766808', '84876364.703*1452768022', '84876364.703*1452768022', '93740288.3243*1452779418', '63997417.8999*1452781794', '27167748.9295*1452782503', '27167748.9295*1452782503', '8613039.52132*1452793336', '42424486.8602*1452794520', '42424486.8602*1452794520', '11580070.643*1452794792', '11580070.643*1452794792', '11580070.643*1452794792', '61458791.0962*1452796793', '40189516.8117*1452798084', '40189516.8117*1452798084', '24075051.1592*1452799626', '79629106.1086*1452801801', '82170803.157*1452804298', '82170803.157*1452804298', '82170803.157*1452804298', '46423891.9383*1452807355', '46423891.9383*1452807355', '55959297.6985*1452815816', '22767199.3886*1452816239', '22767199.3886*1452816239'], balance_amt_1209600=['21549400.9814*1451607860', '60401992.2174*1451611951', '70215158.8144*1451614183', '70215158.8144*1451614183', '99374367.9318*1451616403', '99374367.9318*1451616403', '91843278.6796*1451623350', '44828311.9331*1451625058', '11100036.1124*1451642296', '11100036.1124*1451642296', '839759.974617*1451646892', '839759.974617*1451646892', '90755785.0398*1451647352', '66951727.7947*1451654044', '26063083.9318*1451656253', '26063083.9318*1451656253', '29791574.9576*1451656320', '42899408.762*1451657744', '42899408.762*1451657744', '42899408.762*1451657744', '77082947.4708*1451684342', '77082947.4708*1451684342', '22563996.1981*1451687279', '22563996.1981*1451687279', '51795715.1239*1451697649', '51795715.1239*1451697649', '45527212.31*1451700694', '45527212.31*1451700694', '45527212.31*1451700694', '10682201.3541*1451704165', '54257515.4436*1451710949', '54257515.4436*1451710949', '99518743.8099*1451713915', '87598806.3534*1451714043', '87598806.3534*1451714043', '87598806.3534*1451714043', '87598806.3534*1451714043', '48166326.9459*1451727555', '22783857.6912*1451730738', '24018531.8862*1451735657', '24018531.8862*1451735657', '80830511.9624*1451737630', '94168032.0617*1451739437', '94168032.0617*1451739437', '94168032.0617*1451739437', '65497964.3675*1451744337', '48162626.1282*1451744398', '25558409.3094*1451752723', '46826589.7538*1451754280', '46826589.7538*1451754280', '46826589.7538*1451754280', '46826589.7538*1451754280', '32798096.704*1451762811', '32798096.704*1451762811', '32798096.704*1451762811', '39472240.9442*1451763169', '22517648.4138*1451763638', '17352821.9793*1451778692', '51365492.5152*1451783625', '51365492.5152*1451783625', '51365492.5152*1451783625', '68815725.0423*1451786023', '23000816.569*1451798070', '23000816.569*1451798070', '55414602.3953*1451798752', '55414602.3953*1451798752', '34506568.0672*1451800725', '4669892.60012*1451803874', '4669892.60012*1451803874', '34371388.6558*1451803879', '93636937.3579*1451813728', '64521093.5878*1451816310', '64521093.5878*1451816310', '64521093.5878*1451816310', '37831009.0785*1451816738', '37831009.0785*1451816738', '37831009.0785*1451816738', '96537698.7096*1451837092', '89316369.3734*1451842707', '46209625.4203*1451844588', '46209625.4203*1451844588', '46209625.4203*1451844588', '7514041.8878*1451849291', '7514041.8878*1451849291', '54672341.8154*1451858456', '96834449.7728*1451860532', '96834449.7728*1451860532', '69145424.2992*1451863272', '36104080.8776*1451866109', '36104080.8776*1451866109', '36104080.8776*1451866109', '41085034.3379*1451869294', '17444936.5291*1451871549', '17444936.5291*1451871549', '15354450.0987*1451874436', '15354450.0987*1451874436', '15354450.0987*1451874436', '4274383.96983*1451881008', '4274383.96983*1451881008', '48174124.149*1451893789', '48174124.149*1451893789', '36858583.3293*1451897630', '36858583.3293*1451897630', '36858583.3293*1451897630', '37597903.879*1451905921', '37597903.879*1451905921', '37597903.879*1451905921', '94829762.865*1451906236', '77684304.352*1451906285', '39112700.965*1451910519', '39112700.965*1451910519', '39112700.965*1451910519', '39112700.965*1451910519', '85430569.4661*1451910908', '19027172.2921*1451921545', '70062509.7754*1451921618', '70062509.7754*1451921618', '91432736.7086*1451924664', '47700578.5858*1451937070', '47700578.5858*1451937070', '6279255.30081*1451937986', '53481792.4888*1451940275', '53481792.4888*1451940275', '53481792.4888*1451940275', '46961417.8475*1451940407', '46961417.8475*1451940407', '52410123.2217*1451947798', '52410123.2217*1451947798', '52410123.2217*1451947798', '30491901.7747*1451948169', '30491901.7747*1451948169', '30491901.7747*1451948169', '1015330.40217*1451949746', '1015330.40217*1451949746', '73075675.6549*1451950245', '73075675.6549*1451950245', '54907992.2002*1451954552', '78446793.1618*1451954918', '80455421.9752*1451955575', '80455421.9752*1451955575', '80455421.9752*1451955575', '61777895.9127*1451965797', '61777895.9127*1451965797', '2972832.64437*1451966880', '2972832.64437*1451966880', '30492662.4652*1451969401', '30492662.4652*1451969401', '90608159.8886*1451970969', '7256928.32997*1451972489', '99425008.0265*1451982108', '45720434.4438*1451982880', '45720434.4438*1451982880', '61424188.5626*1451986832', '8860811.05248*1451990236', '68743640.5353*1451994509', '68743640.5353*1451994509', '97609260.2419*1451995791', '87110435.3697*1451996117', '87110435.3697*1451996117', '87110435.3697*1451996117', '11104671.9594*1451997801', '11104671.9594*1451997801', '54669431.4024*1452000769', '84537046.263*1452007063', '7430176.26206*1452008988', '7430176.26206*1452008988', '7430176.26206*1452008988', '47464333.857*1452010149', '31496576.9687*1452010591', '31496576.9687*1452010591', '31496576.9687*1452010591', '3707140.02928*1452013132', '10084105.1771*1452028034', '50723345.6296*1452041765', '27239091.9753*1452047038', '55974982.1024*1452054902', '55974982.1024*1452054902', '55974982.1024*1452054902', '55974982.1024*1452054902', '76021526.2399*1452056349', '76021526.2399*1452056349', '32928203.3214*1452058953', '32928203.3214*1452058953', '63719691.099*1452067003', '63719691.099*1452067003', '63719691.099*1452067003', '63719691.099*1452067003', '50694184.4346*1452069224', '76384095.2107*1452073276', '76384095.2107*1452073276', '40752421.0325*1452076829', '40752421.0325*1452076829', '40752421.0325*1452076829', '40752421.0325*1452076829', '81741865.0156*1452080674', '81741865.0156*1452080674', '79125920.7319*1452099865', '84435816.85*1452113567', '84435816.85*1452113567', '77583899.5541*1452119398', '43206954.0652*1452119971', '43206954.0652*1452119971', '25207370.228*1452125095', '25207370.228*1452125095', '49091809.3248*1452125623', '49091809.3248*1452125623', '49091809.3248*1452125623', '58145176.0326*1452128987', '81665440.1836*1452129494', '94542951.6769*1452134995', '94542951.6769*1452134995', '30211066.8193*1452140012', '30211066.8193*1452140012', '2864661.88916*1452141507', '2864661.88916*1452141507', '2864661.88916*1452141507', '47921801.6018*1452142949', '2075678.25453*1452142954', '2075678.25453*1452142954', '2075678.25453*1452142954', '23083298.4434*1452144735', '25404975.963*1452150718', '39222387.4433*1452156026', '6901151.34368*1452164189', '16623777.4843*1452167835', '16623777.4843*1452167835', '82974276.0334*1452168758', '62336814.4617*1452174023', '62336814.4617*1452174023', '62336814.4617*1452174023', '72395289.9152*1452174336', '72395289.9152*1452174336', '38059552.3137*1452175467', '38059552.3137*1452175467', '38059552.3137*1452175467', '29942766.2363*1452180614', '29942766.2363*1452180614', '63365608.0821*1452190462', '85456964.0583*1452195076', '85456964.0583*1452195076', '85456964.0583*1452195076', '64451579.2426*1452196091', '64451579.2426*1452196091', '64451579.2426*1452196091', '81539727.7963*1452202758', '13348292.3602*1452206051', '79810098.7262*1452208898', '79810098.7262*1452208898', '79810098.7262*1452208898', '7611854.49726*1452214123', '7611854.49726*1452214123', '7611854.49726*1452214123', '68244732.748*1452216360', '68244732.748*1452216360', '40776335.9087*1452218280', '93267936.8825*1452219471', '93267936.8825*1452219471', '91991271.3732*1452223485', '31894925.5808*1452227880', '10264430.0751*1452231411', '10264430.0751*1452231411', '46759144.676*1452240664', '69867176.2749*1452244765', '98419997.4652*1452253151', '36940941.8432*1452260276', '36940941.8432*1452260276', '69769720.209*1452273534', '34403348.9784*1452280846', '34403348.9784*1452280846', '51883951.3099*1452282201', '60494887.8912*1452287217', '69703616.2427*1452287606', '90740086.2578*1452287620', '90740086.2578*1452287620', '90740086.2578*1452287620', '90740086.2578*1452287620', '922673.664184*1452288867', '68278446.7984*1452288890', '53905895.3904*1452301670', '14992686.4035*1452303377', '14992686.4035*1452303377', '70342713.1009*1452316951', '91105468.9167*1452324397', '16588976.8078*1452327342', '16588976.8078*1452327342', '94848978.4181*1452333329', '94848978.4181*1452333329', '67777155.5766*1452334132', '67777155.5766*1452334132', '13496649.2144*1452336906', '13496649.2144*1452336906', '16256766.3029*1452336993', '16256766.3029*1452336993', '74685833.9515*1452337219', '74685833.9515*1452337219', '74685833.9515*1452337219', '27825131.489*1452341096', '27825131.489*1452341096', '27825131.489*1452341096', '81033765.432*1452343556', '40540207.9327*1452345021', '92310409.2499*1452346423', '69336552.5568*1452348638', '80668996.6254*1452351161', '99647988.7359*1452353837', '30054712.3678*1452356249', '30054712.3678*1452356249', '10873878.1209*1452356280', '10873878.1209*1452356280', '10873878.1209*1452356280', '42175684.0828*1452360188', '62334265.2272*1452371127', '52910737.3575*1452372730', '74500195.6964*1452379508', '74500195.6964*1452379508', '74500195.6964*1452379508', '60615210.6449*1452383896', '60615210.6449*1452383896', '60615210.6449*1452383896', '44265928.5164*1452387967', '44265928.5164*1452387967', '36947925.2288*1452394703', '65231622.553*1452398529', '20775605.7234*1452399973', '17740169.5561*1452405394', '17740169.5561*1452405394', '17740169.5561*1452405394', '90597350.9283*1452412119', '53122125.9345*1452413435', '53122125.9345*1452413435', '84573235.4747*1452420153', '84573235.4747*1452420153', '84573235.4747*1452420153', '43280235.8697*1452424028', '1453805.77613*1452425880', '1453805.77613*1452425880', '1453805.77613*1452425880', '44707418.0022*1452430786', '8891093.67508*1452438361', '8891093.67508*1452438361', '23840078.3162*1452438952', '23840078.3162*1452438952', '23840078.3162*1452438952', '34632829.6523*1452441177', '49256392.8226*1452449327', '88744125.5219*1452451397', '88744125.5219*1452451397', '74134672.3631*1452458152', '74134672.3631*1452458152', '74134672.3631*1452458152', '74134672.3631*1452458152', '27972711.508*1452462825', '61031609.079*1452463430', '51627654.3039*1452468868', '9456209.52273*1452480396', '51475097.7973*1452492151', '18063723.2025*1452492711', '86952632.9554*1452495175', '83878924.0499*1452495313', '87071940.3248*1452501428', '87071940.3248*1452501428', '87071940.3248*1452501428', '70421412.6731*1452505965', '3897003.20025*1452507859', '88953670.0063*1452525036', '88953670.0063*1452525036', '88953670.0063*1452525036', '75537948.7366*1452528419', '75537948.7366*1452528419', '75537948.7366*1452528419', '75537948.7366*1452528419', '1640882.96693*1452531371', '1640882.96693*1452531371', '1640882.96693*1452531371', '81448006.3291*1452532683', '47544813.1257*1452536032', '48100653.2662*1452536984', '40625928.5898*1452537896', '99487080.8258*1452538235', '64903060.2595*1452539800', '64903060.2595*1452539800', '64903060.2595*1452539800', '26921614.8511*1452543678', '26921614.8511*1452543678', '31236906.3788*1452546192', '31236906.3788*1452546192', '31236906.3788*1452546192', '9982236.97957*1452552002', '9982236.97957*1452552002', '79332709.9743*1452555711', '39716932.2075*1452559315', '75378641.805*1452560864', '127776.198993*1452564626', '40579437.3889*1452566959', '40579437.3889*1452566959', '16391776.9224*1452567830', '16391776.9224*1452567830', '16391776.9224*1452567830', '14152233.9722*1452570571', '14152233.9722*1452570571', '58268388.9761*1452575051', '50881691.6899*1452576922', '87414169.4691*1452578263', '87414169.4691*1452578263', '87414169.4691*1452578263', '3355736.85765*1452581508', '79788439.5822*1452581752', '79788439.5822*1452581752', '79788439.5822*1452581752', '36473854.4953*1452586837', '37265223.3775*1452595587', '37265223.3775*1452595587', '49419451.2307*1452599920', '53682571.3346*1452604122', '53682571.3346*1452604122', '49147368.1334*1452606334', '74358986.3341*1452611559', '31790500.4877*1452615816', '8425549.64514*1452616504', '95709674.6959*1452618393', '95709674.6959*1452618393', '95709674.6959*1452618393', '62617696.8125*1452633689', '62617696.8125*1452633689', '49554413.24*1452634324', '84263198.4547*1452635199', '86657360.9525*1452635710', '86657360.9525*1452635710', '86657360.9525*1452635710', '97113187.9364*1452648070', '97113187.9364*1452648070', '97113187.9364*1452648070', '10452405.9158*1452650689', '10452405.9158*1452650689', '46722852.502*1452652212', '47718115.247*1452653775', '37030879.8365*1452655684', '37030879.8365*1452655684', '7400891.56672*1452659005', '7400891.56672*1452659005', '7400891.56672*1452659005', '84681574.1415*1452660679', '90340421.1518*1452665649', '92408710.4905*1452665946', '92408710.4905*1452665946', '92408710.4905*1452665946', '40941702.4104*1452668562', '19135350.0295*1452672902', '19135350.0295*1452672902', '31160141.378*1452677155', '71137259.6999*1452677206', '78083246.0698*1452680665', '30789660.014*1452687642', '30789660.014*1452687642', '30789660.014*1452687642', '13784662.7245*1452690354', '46886937.2298*1452692188', '46886937.2298*1452692188', '46886937.2298*1452692188', '14134390.5681*1452693028', '97410920.0956*1452694779', '97410920.0956*1452694779', '65385550.1021*1452703682', '65385550.1021*1452703682', '4615222.32957*1452707474', '4615222.32957*1452707474', '4615222.32957*1452707474', '35967751.3466*1452714322', '35967751.3466*1452714322', '35967751.3466*1452714322', '21475182.4811*1452718442', '21475182.4811*1452718442', '21475182.4811*1452718442', '86910652.9409*1452727180', '86910652.9409*1452727180', '39859470.93*1452732126', '39859470.93*1452732126', '39859470.93*1452732126', '46923295.6953*1452736674', '49236328.2958*1452741984', '49236328.2958*1452741984', '49236328.2958*1452741984', '49236328.2958*1452741984', '13242041.5526*1452747674', '13242041.5526*1452747674', '13242041.5526*1452747674', '10060277.4428*1452753019', '10060277.4428*1452753019', '10060277.4428*1452753019', '92847583.9763*1452753785', '87343715.8341*1452755328', '35716974.6802*1452756579', '22486929.0963*1452757700', '22486929.0963*1452757700', '22486929.0963*1452757700', '31049642.4266*1452763603', '31049642.4266*1452763603', '31049642.4266*1452763603', '14111064.1248*1452766808', '14111064.1248*1452766808', '71868844.0765*1452768022', '71868844.0765*1452768022', '67252794.6894*1452779418', '86655784.933*1452781794', '98063265.4336*1452782503', '98063265.4336*1452782503', '99595698.8234*1452793336', '63152933.3473*1452794520', '63152933.3473*1452794520', '89283436.2502*1452794792', '89283436.2502*1452794792', '89283436.2502*1452794792', '51862479.8439*1452796793', '50181894.4316*1452798084', '50181894.4316*1452798084', '95623657.6849*1452799626', '5620796.12616*1452801801', '48915884.1884*1452804298', '48915884.1884*1452804298', '48915884.1884*1452804298', '97013000.9486*1452807355', '97013000.9486*1452807355', '27307294.327*1452815816', '31347110.238*1452816239', '31347110.238*1452816239'], u_time_1209600=['1451607860', '1451611951', '1451614183', '1451614183', '1451616403', '1451616403', '1451623350', '1451625058', '1451642296', '1451642296', '1451646892', '1451646892', '1451647352', '1451654044', '1451656253', '1451656253', '1451656320', '1451657744', '1451657744', '1451657744', '1451684342', '1451684342', '1451687279', '1451687279', '1451697649', '1451697649', '1451700694', '1451700694', '1451700694', '1451704165', '1451710949', '1451710949', '1451713915', '1451714043', '1451714043', '1451714043', '1451714043', '1451727555', '1451730738', '1451735657', '1451735657', '1451737630', '1451739437', '1451739437', '1451739437', '1451744337', '1451744398', '1451752723', '1451754280', '1451754280', '1451754280', '1451754280', '1451762811', '1451762811', '1451762811', '1451763169', '1451763638', '1451778692', '1451783625', '1451783625', '1451783625', '1451786023', '1451798070', '1451798070', '1451798752', '1451798752', '1451800725', '1451803874', '1451803874', '1451803879', '1451813728', '1451816310', '1451816310', '1451816310', '1451816738', '1451816738', '1451816738', '1451837092', '1451842707', '1451844588', '1451844588', '1451844588', '1451849291', '1451849291', '1451858456', '1451860532', '1451860532', '1451863272', '1451866109', '1451866109', '1451866109', '1451869294', '1451871549', '1451871549', '1451874436', '1451874436', '1451874436', '1451881008', '1451881008', '1451893789', '1451893789', '1451897630', '1451897630', '1451897630', '1451905921', '1451905921', '1451905921', '1451906236', '1451906285', '1451910519', '1451910519', '1451910519', '1451910519', '1451910908', '1451921545', '1451921618', '1451921618', '1451924664', '1451937070', '1451937070', '1451937986', '1451940275', '1451940275', '1451940275', '1451940407', '1451940407', '1451947798', '1451947798', '1451947798', '1451948169', '1451948169', '1451948169', '1451949746', '1451949746', '1451950245', '1451950245', '1451954552', '1451954918', '1451955575', '1451955575', '1451955575', '1451965797', '1451965797', '1451966880', '1451966880', '1451969401', '1451969401', '1451970969', '1451972489', '1451982108', '1451982880', '1451982880', '1451986832', '1451990236', '1451994509', '1451994509', '1451995791', '1451996117', '1451996117', '1451996117', '1451997801', '1451997801', '1452000769', '1452007063', '1452008988', '1452008988', '1452008988', '1452010149', '1452010591', '1452010591', '1452010591', '1452013132', '1452028034', '1452041765', '1452047038', '1452054902', '1452054902', '1452054902', '1452054902', '1452056349', '1452056349', '1452058953', '1452058953', '1452067003', '1452067003', '1452067003', '1452067003', '1452069224', '1452073276', '1452073276', '1452076829', '1452076829', '1452076829', '1452076829', '1452080674', '1452080674', '1452099865', '1452113567', '1452113567', '1452119398', '1452119971', '1452119971', '1452125095', '1452125095', '1452125623', '1452125623', '1452125623', '1452128987', '1452129494', '1452134995', '1452134995', '1452140012', '1452140012', '1452141507', '1452141507', '1452141507', '1452142949', '1452142954', '1452142954', '1452142954', '1452144735', '1452150718', '1452156026', '1452164189', '1452167835', '1452167835', '1452168758', '1452174023', '1452174023', '1452174023', '1452174336', '1452174336', '1452175467', '1452175467', '1452175467', '1452180614', '1452180614', '1452190462', '1452195076', '1452195076', '1452195076', '1452196091', '1452196091', '1452196091', '1452202758', '1452206051', '1452208898', '1452208898', '1452208898', '1452214123', '1452214123', '1452214123', '1452216360', '1452216360', '1452218280', '1452219471', '1452219471', '1452223485', '1452227880', '1452231411', '1452231411', '1452240664', '1452244765', '1452253151', '1452260276', '1452260276', '1452273534', '1452280846', '1452280846', '1452282201', '1452287217', '1452287606', '1452287620', '1452287620', '1452287620', '1452287620', '1452288867', '1452288890', '1452301670', '1452303377', '1452303377', '1452316951', '1452324397', '1452327342', '1452327342', '1452333329', '1452333329', '1452334132', '1452334132', '1452336906', '1452336906', '1452336993', '1452336993', '1452337219', '1452337219', '1452337219', '1452341096', '1452341096', '1452341096', '1452343556', '1452345021', '1452346423', '1452348638', '1452351161', '1452353837', '1452356249', '1452356249', '1452356280', '1452356280', '1452356280', '1452360188', '1452371127', '1452372730', '1452379508', '1452379508', '1452379508', '1452383896', '1452383896', '1452383896', '1452387967', '1452387967', '1452394703', '1452398529', '1452399973', '1452405394', '1452405394', '1452405394', '1452412119', '1452413435', '1452413435', '1452420153', '1452420153', '1452420153', '1452424028', '1452425880', '1452425880', '1452425880', '1452430786', '1452438361', '1452438361', '1452438952', '1452438952', '1452438952', '1452441177', '1452449327', '1452451397', '1452451397', '1452458152', '1452458152', '1452458152', '1452458152', '1452462825', '1452463430', '1452468868', '1452480396', '1452492151', '1452492711', '1452495175', '1452495313', '1452501428', '1452501428', '1452501428', '1452505965', '1452507859', '1452525036', '1452525036', '1452525036', '1452528419', '1452528419', '1452528419', '1452528419', '1452531371', '1452531371', '1452531371', '1452532683', '1452536032', '1452536984', '1452537896', '1452538235', '1452539800', '1452539800', '1452539800', '1452543678', '1452543678', '1452546192', '1452546192', '1452546192', '1452552002', '1452552002', '1452555711', '1452559315', '1452560864', '1452564626', '1452566959', '1452566959', '1452567830', '1452567830', '1452567830', '1452570571', '1452570571', '1452575051', '1452576922', '1452578263', '1452578263', '1452578263', '1452581508', '1452581752', '1452581752', '1452581752', '1452586837', '1452595587', '1452595587', '1452599920', '1452604122', '1452604122', '1452606334', '1452611559', '1452615816', '1452616504', '1452618393', '1452618393', '1452618393', '1452633689', '1452633689', '1452634324', '1452635199', '1452635710', '1452635710', '1452635710', '1452648070', '1452648070', '1452648070', '1452650689', '1452650689', '1452652212', '1452653775', '1452655684', '1452655684', '1452659005', '1452659005', '1452659005', '1452660679', '1452665649', '1452665946', '1452665946', '1452665946', '1452668562', '1452672902', '1452672902', '1452677155', '1452677206', '1452680665', '1452687642', '1452687642', '1452687642', '1452690354', '1452692188', '1452692188', '1452692188', '1452693028', '1452694779', '1452694779', '1452703682', '1452703682', '1452707474', '1452707474', '1452707474', '1452714322', '1452714322', '1452714322', '1452718442', '1452718442', '1452718442', '1452727180', '1452727180', '1452732126', '1452732126', '1452732126', '1452736674', '1452741984', '1452741984', '1452741984', '1452741984', '1452747674', '1452747674', '1452747674', '1452753019', '1452753019', '1452753019', '1452753785', '1452755328', '1452756579', '1452757700', '1452757700', '1452757700', '1452763603', '1452763603', '1452763603', '1452766808', '1452766808', '1452768022', '1452768022', '1452779418', '1452781794', '1452782503', '1452782503', '1452793336', '1452794520', '1452794520', '1452794792', '1452794792', '1452794792', '1452796793', '1452798084', '1452798084', '1452799626', '1452801801', '1452804298', '1452804298', '1452804298', '1452807355', '1452807355', '1452815816', '1452816239', '1452816239'], crdr_0_1209600=289, crdr_1_1209600=236, cnt_2592000=1152, amt_2592000=['12471505.3357*1451607860', '78809477.6749*1451611951', '67317155.4321*1451614183', '67317155.4321*1451614183', '97672665.4793*1451616403', '97672665.4793*1451616403', '72264109.8265*1451623350', '89669411.1102*1451625058', '88556416.6689*1451642296', '88556416.6689*1451642296', '91545230.2289*1451646892', '91545230.2289*1451646892', '77146413.4146*1451647352', '22321592.7839*1451654044', '72215878.7465*1451656253', '72215878.7465*1451656253', '90251522.0652*1451656320', '73038389.1455*1451657744', '73038389.1455*1451657744', '73038389.1455*1451657744', '67834526.2523*1451684342', '67834526.2523*1451684342', '1785326.45646*1451687279', '1785326.45646*1451687279', '17136968.1802*1451697649', '17136968.1802*1451697649', '3374289.69795*1451700694', '3374289.69795*1451700694', '3374289.69795*1451700694', '7060522.08945*1451704165', '273818.318591*1451710949', '273818.318591*1451710949', '50458366.6304*1451713915', '89525055.1045*1451714043', '89525055.1045*1451714043', '89525055.1045*1451714043', '89525055.1045*1451714043', '83372957.6784*1451727555', '37589086.961*1451730738', '18852484.0632*1451735657', '18852484.0632*1451735657', '36053624.0149*1451737630', '34918676.4457*1451739437', '34918676.4457*1451739437', '34918676.4457*1451739437', '35853187.1554*1451744337', '49440869.4862*1451744398', '19196263.37*1451752723', '1784056.51043*1451754280', '1784056.51043*1451754280', '1784056.51043*1451754280', '1784056.51043*1451754280', '43363610.5017*1451762811', '43363610.5017*1451762811', '43363610.5017*1451762811', '52683411.1171*1451763169', '17191435.568*1451763638', '3563459.35301*1451778692', '33761845.3375*1451783625', '33761845.3375*1451783625', '33761845.3375*1451783625', '24876786.8751*1451786023', '23463873.3936*1451798070', '23463873.3936*1451798070', '43653135.5523*1451798752', '43653135.5523*1451798752', '29008839.1511*1451800725', '59186620.7815*1451803874', '59186620.7815*1451803874', '28345059.2314*1451803879', '70065803.4736*1451813728', '35702005.5204*1451816310', '35702005.5204*1451816310', '35702005.5204*1451816310', '13132259.2355*1451816738', '13132259.2355*1451816738', '13132259.2355*1451816738', '33002083.7483*1451837092', '60233551.7856*1451842707', '15751307.8532*1451844588', '15751307.8532*1451844588', '15751307.8532*1451844588', '77070101.1854*1451849291', '77070101.1854*1451849291', '86941290.3878*1451858456', '69744118.3656*1451860532', '69744118.3656*1451860532', '95828115.1999*1451863272', '38887644.1068*1451866109', '38887644.1068*1451866109', '38887644.1068*1451866109', '79275078.3014*1451869294', '6905548.52172*1451871549', '6905548.52172*1451871549', '34428688.5526*1451874436', '34428688.5526*1451874436', '34428688.5526*1451874436', '12938394.5478*1451881008', '12938394.5478*1451881008', '86436996.7443*1451893789', '86436996.7443*1451893789', '64657142.258*1451897630', '64657142.258*1451897630', '64657142.258*1451897630', '74486802.2976*1451905921', '74486802.2976*1451905921', '74486802.2976*1451905921', '18721749.0222*1451906236', '11325939.4693*1451906285', '25065557.914*1451910519', '25065557.914*1451910519', '25065557.914*1451910519', '25065557.914*1451910519', '22835110.0577*1451910908', '32251480.2938*1451921545', '34106307.8046*1451921618', '34106307.8046*1451921618', '48228385.3959*1451924664', '48838735.9788*1451937070', '48838735.9788*1451937070', '51118025.2697*1451937986', '53804986.1216*1451940275', '53804986.1216*1451940275', '53804986.1216*1451940275', '41781499.7678*1451940407', '41781499.7678*1451940407', '46975494.5064*1451947798', '46975494.5064*1451947798', '46975494.5064*1451947798', '95000641.5339*1451948169', '95000641.5339*1451948169', '95000641.5339*1451948169', '93165958.6984*1451949746', '93165958.6984*1451949746', '46824110.3968*1451950245', '46824110.3968*1451950245', '55588098.1674*1451954552', '60626970.398*1451954918', '17475790.8939*1451955575', '17475790.8939*1451955575', '17475790.8939*1451955575', '46481601.6725*1451965797', '46481601.6725*1451965797', '1357944.09872*1451966880', '1357944.09872*1451966880', '42678397.402*1451969401', '42678397.402*1451969401', '80798066.3313*1451970969', '65109648.6329*1451972489', '28307366.505*1451982108', '2248485.43819*1451982880', '2248485.43819*1451982880', '68772700.1787*1451986832', '9094888.5806*1451990236', '44923134.0054*1451994509', '44923134.0054*1451994509', '73652993.2372*1451995791', '88602665.39*1451996117', '88602665.39*1451996117', '88602665.39*1451996117', '51125324.7342*1451997801', '51125324.7342*1451997801', '27293598.7805*1452000769', '31182233.709*1452007063', '38557045.5662*1452008988', '38557045.5662*1452008988', '38557045.5662*1452008988', '36527483.0752*1452010149', '54765591.5953*1452010591', '54765591.5953*1452010591', '54765591.5953*1452010591', '39610214.4729*1452013132', '63073189.1827*1452028034', '93820344.5575*1452041765', '32478322.5091*1452047038', '87426316.3732*1452054902', '87426316.3732*1452054902', '87426316.3732*1452054902', '87426316.3732*1452054902', '8160560.52548*1452056349', '8160560.52548*1452056349', '27534271.8782*1452058953', '27534271.8782*1452058953', '1764485.37125*1452067003', '1764485.37125*1452067003', '1764485.37125*1452067003', '1764485.37125*1452067003', '89794309.9256*1452069224', '31905725.2591*1452073276', '31905725.2591*1452073276', '47030293.3045*1452076829', '47030293.3045*1452076829', '47030293.3045*1452076829', '47030293.3045*1452076829', '4883487.86866*1452080674', '4883487.86866*1452080674', '95153652.9705*1452099865', '68991183.7462*1452113567', '68991183.7462*1452113567', '10013694.0752*1452119398', '22757413.6641*1452119971', '22757413.6641*1452119971', '38984712.0692*1452125095', '38984712.0692*1452125095', '27001775.7996*1452125623', '27001775.7996*1452125623', '27001775.7996*1452125623', '68721710.0389*1452128987', '30451616.1309*1452129494', '43640357.817*1452134995', '43640357.817*1452134995', '42025696.9398*1452140012', '42025696.9398*1452140012', '57982742.5497*1452141507', '57982742.5497*1452141507', '57982742.5497*1452141507', '76269481.7453*1452142949', '21088702.7842*1452142954', '21088702.7842*1452142954', '21088702.7842*1452142954', '494012.891686*1452144735', '53332591.8915*1452150718', '92323689.7424*1452156026', '43797371.5834*1452164189', '61820440.5636*1452167835', '61820440.5636*1452167835', '28770485.9873*1452168758', '38574126.9816*1452174023', '38574126.9816*1452174023', '38574126.9816*1452174023', '81974798.4459*1452174336', '81974798.4459*1452174336', '95539075.4215*1452175467', '95539075.4215*1452175467', '95539075.4215*1452175467', '52889443.076*1452180614', '52889443.076*1452180614', '82881264.4604*1452190462', '8418037.73131*1452195076', '8418037.73131*1452195076', '8418037.73131*1452195076', '81139663.628*1452196091', '81139663.628*1452196091', '81139663.628*1452196091', '3324350.4493*1452202758', '16198297.0253*1452206051', '32828024.8871*1452208898', '32828024.8871*1452208898', '32828024.8871*1452208898', '39519338.2479*1452214123', '39519338.2479*1452214123', '39519338.2479*1452214123', '89991721.2313*1452216360', '89991721.2313*1452216360', '34900636.8967*1452218280', '79136577.5331*1452219471', '79136577.5331*1452219471', '83683638.2111*1452223485', '50208103.8346*1452227880', '88352795.6703*1452231411', '88352795.6703*1452231411', '7268095.7213*1452240664', '78076249.7288*1452244765', '24246248.4543*1452253151', '56245228.4508*1452260276', '56245228.4508*1452260276', '88054133.9195*1452273534', '27599359.6395*1452280846', '27599359.6395*1452280846', '78893492.4727*1452282201', '96128463.9571*1452287217', '47000955.15*1452287606', '11163625.1241*1452287620', '11163625.1241*1452287620', '11163625.1241*1452287620', '11163625.1241*1452287620', '79118365.6654*1452288867', '64575864.7724*1452288890', '82146649.5574*1452301670', '474263.060471*1452303377', '474263.060471*1452303377', '2876679.81848*1452316951', '24577662.0889*1452324397', '62093151.3611*1452327342', '62093151.3611*1452327342', '83598740.4709*1452333329', '83598740.4709*1452333329', '47586954.4082*1452334132', '47586954.4082*1452334132', '33387559.5508*1452336906', '33387559.5508*1452336906', '89307721.198*1452336993', '89307721.198*1452336993', '4324215.89677*1452337219', '4324215.89677*1452337219', '4324215.89677*1452337219', '44639511.4194*1452341096', '44639511.4194*1452341096', '44639511.4194*1452341096', '99213706.3456*1452343556', '15052537.241*1452345021', '13386742.3384*1452346423', '6558924.30821*1452348638', '3728533.28245*1452351161', '47732414.0443*1452353837', '76258893.8072*1452356249', '76258893.8072*1452356249', '21061324.0631*1452356280', '21061324.0631*1452356280', '21061324.0631*1452356280', '91327408.7351*1452360188', '95997987.1225*1452371127', '60220786.8947*1452372730', '72125373.4605*1452379508', '72125373.4605*1452379508', '72125373.4605*1452379508', '56067042.2541*1452383896', '56067042.2541*1452383896', '56067042.2541*1452383896', '49631360.6637*1452387967', '49631360.6637*1452387967', '66705576.3294*1452394703', '81408940.2247*1452398529', '55156846.4473*1452399973', '44884380.6695*1452405394', '44884380.6695*1452405394', '44884380.6695*1452405394', '46130751.5406*1452412119', '45374060.7508*1452413435', '45374060.7508*1452413435', '98219830.5049*1452420153', '98219830.5049*1452420153', '98219830.5049*1452420153', '59673433.3308*1452424028', '35944553.3003*1452425880', '35944553.3003*1452425880', '35944553.3003*1452425880', '65796453.9248*1452430786', '15805111.6006*1452438361', '15805111.6006*1452438361', '54344825.0706*1452438952', '54344825.0706*1452438952', '54344825.0706*1452438952', '7150146.41317*1452441177', '97204310.791*1452449327', '67146699.3385*1452451397', '67146699.3385*1452451397', '42086775.4655*1452458152', '42086775.4655*1452458152', '42086775.4655*1452458152', '42086775.4655*1452458152', '34065107.8024*1452462825', '86357890.9523*1452463430', '62053316.91*1452468868', '76283855.3999*1452480396', '21694077.713*1452492151', '38682793.9031*1452492711', '9744944.23327*1452495175', '20524051.2489*1452495313', '59400112.7312*1452501428', '59400112.7312*1452501428', '59400112.7312*1452501428', '79793635.291*1452505965', '7994716.35401*1452507859', '76449878.0104*1452525036', '76449878.0104*1452525036', '76449878.0104*1452525036', '35743551.8138*1452528419', '35743551.8138*1452528419', '35743551.8138*1452528419', '35743551.8138*1452528419', '18895235.187*1452531371', '18895235.187*1452531371', '18895235.187*1452531371', '2236692.50435*1452532683', '44963012.8462*1452536032', '64350517.9108*1452536984', '53270059.5622*1452537896', '55597612.4873*1452538235', '35230297.5699*1452539800', '35230297.5699*1452539800', '35230297.5699*1452539800', '66690115.8968*1452543678', '66690115.8968*1452543678', '37919081.5225*1452546192', '37919081.5225*1452546192', '37919081.5225*1452546192', '87082150.215*1452552002', '87082150.215*1452552002', '46036433.1997*1452555711', '98618098.0277*1452559315', '33456725.6204*1452560864', '88461887.4017*1452564626', '98339795.5973*1452566959', '98339795.5973*1452566959', '27251648.7792*1452567830', '27251648.7792*1452567830', '27251648.7792*1452567830', '91284914.4562*1452570571', '91284914.4562*1452570571', '64690112.7234*1452575051', '87053222.6233*1452576922', '72879583.9435*1452578263', '72879583.9435*1452578263', '72879583.9435*1452578263', '25235919.7262*1452581508', '88735224.264*1452581752', '88735224.264*1452581752', '88735224.264*1452581752', '57418741.5018*1452586837', '97974603.5916*1452595587', '97974603.5916*1452595587', '85599014.1729*1452599920', '54442582.8543*1452604122', '54442582.8543*1452604122', '94433488.6669*1452606334', '41989896.2369*1452611559', '26480862.4608*1452615816', '7676448.04519*1452616504', '63383817.9947*1452618393', '63383817.9947*1452618393', '63383817.9947*1452618393', '40067889.8285*1452633689', '40067889.8285*1452633689', '71910773.8536*1452634324', '42574884.9977*1452635199', '14994009.4493*1452635710', '14994009.4493*1452635710', '14994009.4493*1452635710', '93892116.2176*1452648070', '93892116.2176*1452648070', '93892116.2176*1452648070', '71650048.5034*1452650689', '71650048.5034*1452650689', '92564358.7065*1452652212', '64962667.5946*1452653775', '23124051.9048*1452655684', '23124051.9048*1452655684', '86465135.4894*1452659005', '86465135.4894*1452659005', '86465135.4894*1452659005', '37145871.8236*1452660679', '5427654.00078*1452665649', '59721738.9463*1452665946', '59721738.9463*1452665946', '59721738.9463*1452665946', '43357861.011*1452668562', '50177472.8671*1452672902', '50177472.8671*1452672902', '13727352.9834*1452677155', '25797748.3005*1452677206', '18007004.9258*1452680665', '1919177.12206*1452687642', '1919177.12206*1452687642', '1919177.12206*1452687642', '78833900.7855*1452690354', '91687610.5167*1452692188', '91687610.5167*1452692188', '91687610.5167*1452692188', '5226595.70546*1452693028', '8945988.96078*1452694779', '8945988.96078*1452694779', '74020466.3508*1452703682', '74020466.3508*1452703682', '5995672.96466*1452707474', '5995672.96466*1452707474', '5995672.96466*1452707474', '25184576.0823*1452714322', '25184576.0823*1452714322', '25184576.0823*1452714322', '85022200.0119*1452718442', '85022200.0119*1452718442', '85022200.0119*1452718442', '96950148.0491*1452727180', '96950148.0491*1452727180', '70147043.3536*1452732126', '70147043.3536*1452732126', '70147043.3536*1452732126', '39388379.1781*1452736674', '38395984.6584*1452741984', '38395984.6584*1452741984', '38395984.6584*1452741984', '38395984.6584*1452741984', '79812867.3068*1452747674', '79812867.3068*1452747674', '79812867.3068*1452747674', '9439599.56143*1452753019', '9439599.56143*1452753019', '9439599.56143*1452753019', '56778980.3892*1452753785', '85958842.3213*1452755328', '42730607.3656*1452756579', '37324204.148*1452757700', '37324204.148*1452757700', '37324204.148*1452757700', '29140188.6074*1452763603', '29140188.6074*1452763603', '29140188.6074*1452763603', '60956449.5503*1452766808', '60956449.5503*1452766808', '84876364.703*1452768022', '84876364.703*1452768022', '93740288.3243*1452779418', '63997417.8999*1452781794', '27167748.9295*1452782503', '27167748.9295*1452782503', '8613039.52132*1452793336', '42424486.8602*1452794520', '42424486.8602*1452794520', '11580070.643*1452794792', '11580070.643*1452794792', '11580070.643*1452794792', '61458791.0962*1452796793', '40189516.8117*1452798084', '40189516.8117*1452798084', '24075051.1592*1452799626', '79629106.1086*1452801801', '82170803.157*1452804298', '82170803.157*1452804298', '82170803.157*1452804298', '46423891.9383*1452807355', '46423891.9383*1452807355', '55959297.6985*1452815816', '22767199.3886*1452816239', '22767199.3886*1452816239', '779942.342627*1452819141', '78953393.0393*1452820570', '81419881.6467*1452821166', '81419881.6467*1452821166', '19474973.1269*1452823522', '19474973.1269*1452823522', '44215815.05*1452830255', '30459158.7373*1452831852', '62404826.346*1452837255', '62404826.346*1452837255', '62404826.346*1452837255', '66926516.258*1452841216', '61054352.869*1452845969', '61054352.869*1452845969', '80777542.539*1452848232', '80777542.539*1452848232', '80777542.539*1452848232', '67283179.7976*1452853235', '81624022.7994*1452853320', '35777494.5556*1452854090', '35777494.5556*1452854090', '10725079.6058*1452855899', '85661835.8602*1452856501', '23174676.824*1452858359', '24693612.4425*1452865700', '24693612.4425*1452865700', '24693612.4425*1452865700', '39510901.5206*1452879203', '39510901.5206*1452879203', '39510901.5206*1452879203', '67770798.3664*1452888817', '54146004.87*1452888904', '5858963.88948*1452903894', '54684866.3368*1452908307', '44830187.1691*1452909057', '44830187.1691*1452909057', '44830187.1691*1452909057', '76986399.5217*1452909693', '76986399.5217*1452909693', '76986399.5217*1452909693', '80586195.4781*1452917783', '80586195.4781*1452917783', '80586195.4781*1452917783', '20348868.3957*1452918657', '86632399.2676*1452921598', '86632399.2676*1452921598', '86632399.2676*1452921598', '21026473.0625*1452923510', '78785969.436*1452929465', '25359626.5483*1452937982', '5388959.56857*1452939253', '5388959.56857*1452939253', '5388959.56857*1452939253', '69314376.4055*1452945101', '32356293.9024*1452946274', '1848869.93776*1452947627', '40498637.1995*1452955306', '57862969.6218*1452958646', '57862969.6218*1452958646', '61907254.4027*1452959901', '79567811.3653*1452960650', '79567811.3653*1452960650', '79567811.3653*1452960650', '84498118.9018*1452965865', '84498118.9018*1452965865', '62185143.5998*1452968117', '30785508.2772*1452968455', '30785508.2772*1452968455', '30785508.2772*1452968455', '30785508.2772*1452968455', '65919428.2646*1452975707', '65919428.2646*1452975707', '19700181.2521*1452975753', '19700181.2521*1452975753', '57072175.5975*1452990876', '85180.2567351*1453000153', '85180.2567351*1453000153', '55610935.2018*1453002236', '55610935.2018*1453002236', '43881462.0366*1453007746', '43881462.0366*1453007746', '85274970.6388*1453008152', '85274970.6388*1453008152', '18673684.724*1453008418', '18673684.724*1453008418', '18673684.724*1453008418', '62419437.6214*1453016507', '62419437.6214*1453016507', '20455618.2622*1453022075', '20455618.2622*1453022075', '20455618.2622*1453022075', '26431450.5603*1453023061', '26431450.5603*1453023061', '26431450.5603*1453023061', '30331074.0239*1453025083', '30331074.0239*1453025083', '49078975.5392*1453029252', '83368744.379*1453033290', '83368744.379*1453033290', '73526371.0532*1453033841', '73526371.0532*1453033841', '40315992.0457*1453035181', '73971625.4228*1453042488', '64140074.5292*1453042834', '87335184.7038*1453045271', '742490.359921*1453047910', '742490.359921*1453047910', '84423067.983*1453049807', '32091212.1892*1453053092', '93932059.911*1453053218', '4764975.80551*1453056246', '4764975.80551*1453056246', '68384081.7518*1453062175', '67176311.629*1453064064', '67176311.629*1453064064', '67176311.629*1453064064', '71410163.6115*1453064412', '81621744.3093*1453066114', '32798939.2584*1453080014', '32798939.2584*1453080014', '64702891.2354*1453082412', '64702891.2354*1453082412', '28367780.7906*1453084155', '28367780.7906*1453084155', '66221747.0626*1453098110', '66221747.0626*1453098110', '66221747.0626*1453098110', '1303491.02451*1453098407', '8305729.23174*1453103020', '94250126.3883*1453103934', '94250126.3883*1453103934', '94250126.3883*1453103934', '170764.890583*1453106140', '91977121.9474*1453109334', '91977121.9474*1453109334', '48342737.1243*1453122069', '68063652.6229*1453134142', '68063652.6229*1453134142', '66392457.2961*1453145614', '43275289.3182*1453146650', '25378223.3002*1453147559', '21848553.2456*1453163186', '80211991.8731*1453165799', '80211991.8731*1453165799', '80211991.8731*1453165799', '72749100.1724*1453169901', '77783328.4306*1453175254', '68294832.7065*1453177484', '47438684.2096*1453180858', '74937296.6416*1453185203', '5303110.48826*1453186277', '5303110.48826*1453186277', '23234350.212*1453187208', '59012817.3237*1453194521', '59012817.3237*1453194521', '53411964.7782*1453201571', '35052325.1513*1453206175', '35052325.1513*1453206175', '35052325.1513*1453206175', '75963407.5951*1453212741', '75963407.5951*1453212741', '75963407.5951*1453212741', '47566286.565*1453215470', '25892096.4317*1453218526', '25892096.4317*1453218526', '45970849.8201*1453224654', '45970849.8201*1453224654', '45970849.8201*1453224654', '30823165.6428*1453227295', '30823165.6428*1453227295', '30823165.6428*1453227295', '1458833.65634*1453233195', '52251300.9418*1453236054', '52251300.9418*1453236054', '52251300.9418*1453236054', '6925544.09305*1453236124', '6925544.09305*1453236124', '17185311.2848*1453246994', '17185311.2848*1453246994', '79584710.1292*1453249205', '79584710.1292*1453249205', '87491193.358*1453251516', '87491193.358*1453251516', '3914782.62886*1453256965', '3914782.62886*1453256965', '78759957.8396*1453260668', '78759957.8396*1453260668', '78759957.8396*1453260668', '58362209.1652*1453265382', '65566412.4033*1453266315', '4781492.01121*1453268867', '4781492.01121*1453268867', '2452353.592*1453273281', '2452353.592*1453273281', '11076388.4648*1453274408', '26585896.4574*1453278609', '26585896.4574*1453278609', '24415169.0651*1453284558', '24415169.0651*1453284558', '52117986.662*1453290881', '52117986.662*1453290881', '52117986.662*1453290881', '968451.415474*1453294928', '968451.415474*1453294928', '968451.415474*1453294928', '35438201.9981*1453297000', '80506496.279*1453298606', '80506496.279*1453298606', '14079855.0189*1453299440', '14079855.0189*1453299440', '76820760.7924*1453299731', '76820760.7924*1453299731', '42278755.9806*1453299753', '78437955.8907*1453304242', '19378740.1554*1453305248', '8506447.09146*1453307354', '13172646.1935*1453331761', '13172646.1935*1453331761', '13172646.1935*1453331761', '32665776.7241*1453332005', '32665776.7241*1453332005', '32665776.7241*1453332005', '79832188.7915*1453332112', '79832188.7915*1453332112', '48737945.9612*1453335819', '10123189.2319*1453343936', '10123189.2319*1453343936', '10123189.2319*1453343936', '51437852.5062*1453353213', '51437852.5062*1453353213', '51437852.5062*1453353213', '273920.33823*1453361398', '273920.33823*1453361398', '94301014.8516*1453372768', '94301014.8516*1453372768', '66811509.2134*1453379610', '66811509.2134*1453379610', '89867123.0896*1453382695', '89867123.0896*1453382695', '26284134.9319*1453386744', '55023582.6886*1453390947', '55023582.6886*1453390947', '55023582.6886*1453390947', '35691143.6449*1453397681', '35691143.6449*1453397681', '20662392.1862*1453398547', '64024962.4967*1453398592', '64482358.5954*1453402630', '38442017.3459*1453411981', '38442017.3459*1453411981', '50480306.9704*1453420059', '50480306.9704*1453420059', '50480306.9704*1453420059', '30565333.7526*1453423004', '14163642.4176*1453424792', '14163642.4176*1453424792', '14163642.4176*1453424792', '36834669.5776*1453424996', '67744366.8935*1453425660', '67744366.8935*1453425660', '67744366.8935*1453425660', '32440576.4471*1453427992', '32440576.4471*1453427992', '89576942.603*1453432440', '89576942.603*1453432440', '89576942.603*1453432440', '93160763.9415*1453441705', '93160763.9415*1453441705', '93160763.9415*1453441705', '79715027.6925*1453447625', '94710845.8924*1453455358', '94710845.8924*1453455358', '94710845.8924*1453455358', '80579713.1659*1453457120', '13622329.4637*1453460459', '74633435.8115*1453461809', '44156743.7549*1453463349', '58092363.2376*1453465523', '58092363.2376*1453465523', '58092363.2376*1453465523', '73209960.0136*1453466342', '73209960.0136*1453466342', '73209960.0136*1453466342', '70911654.1825*1453466660', '70911654.1825*1453466660', '97874226.5227*1453473378', '63617979.8228*1453475784', '32153194.883*1453479488', '63508610.5467*1453482148', '63508610.5467*1453482148', '6887819.81127*1453482876', '6887819.81127*1453482876', '6887819.81127*1453482876', '90458298.2103*1453486849', '90458298.2103*1453486849', '90458298.2103*1453486849', '90458298.2103*1453486849', '99114259.5957*1453496513', '99114259.5957*1453496513', '99114259.5957*1453496513', '81345584.1302*1453501265', '90134563.5919*1453502539', '20341560.5576*1453505090', '76143608.7713*1453507345', '76143608.7713*1453507345', '76143608.7713*1453507345', '76370743.2467*1453513492', '76370743.2467*1453513492', '64257901.1901*1453513833', '64257901.1901*1453513833', '43728704.9148*1453517484', '43728704.9148*1453517484', '48817693.3698*1453519353', '6927106.50435*1453521824', '6927106.50435*1453521824', '66358000.5532*1453522238', '66358000.5532*1453522238', '9467039.99937*1453523098', '37708966.5542*1453524851', '21829382.9834*1453534129', '21829382.9834*1453534129', '21829382.9834*1453534129', '89586651.9395*1453539206', '89586651.9395*1453539206', '7269784.24235*1453547182', '6999423.0901*1453547526', '36370556.4387*1453553770', '36370556.4387*1453553770', '36370556.4387*1453553770', '95610387.9748*1453558558', '95610387.9748*1453558558', '91968134.168*1453559946', '91968134.168*1453559946', '78290672.2877*1453560825', '78290672.2877*1453560825', '58874336.0544*1453561734', '58874336.0544*1453561734', '38869401.6999*1453563835', '38869401.6999*1453563835', '38869401.6999*1453563835', '47498168.5107*1453573999', '41591658.8818*1453576375', '67431116.3109*1453576992', '34974285.102*1453585041', '34974285.102*1453585041', '34974285.102*1453585041', '4592220.20735*1453587326', '52715147.3254*1453590343', '29427269.0851*1453594313', '29427269.0851*1453594313', '13981596.8997*1453597829', '23295493.7702*1453601341', '65370246.2963*1453605272', '65370246.2963*1453605272', '54771642.9232*1453605592', '29914473.4879*1453610104', '29914473.4879*1453610104', '29914473.4879*1453610104', '7857840.47799*1453615656', '7857840.47799*1453615656', '54351614.0534*1453621894', '54351614.0534*1453621894', '54351614.0534*1453621894', '33485793.9602*1453629550', '33485793.9602*1453629550', '68222507.9983*1453631116', '75221162.9921*1453631436', '72784209.6268*1453631592', '72784209.6268*1453631592', '72784209.6268*1453631592', '99665858.7784*1453652357', '40818575.4607*1453657764', '66848813.7005*1453661080', '66848813.7005*1453661080', '76912777.4137*1453671680', '23556802.1863*1453671943', '54522397.0191*1453672945', '54522397.0191*1453672945', '14797228.3711*1453676312', '14797228.3711*1453676312', '2186306.55341*1453677354', '46011777.507*1453679448', '46011777.507*1453679448', '46011777.507*1453679448', '90250156.0706*1453680882', '90250156.0706*1453680882', '88975843.2321*1453683097', '88975843.2321*1453683097', '87857597.0579*1453684671', '4829085.66228*1453687587', '4829085.66228*1453687587', '41056200.8042*1453688183', '67937780.0158*1453689175', '67937780.0158*1453689175', '67937780.0158*1453689175', '41433562.0202*1453690171', '26298767.3111*1453698824', '96973735.017*1453700380', '98834498.0871*1453700935', '98834498.0871*1453700935', '98834498.0871*1453700935', '82544860.6579*1453701543', '90697721.6388*1453701607', '90697721.6388*1453701607', '50518745.8366*1453701652', '50518745.8366*1453701652', '50518745.8366*1453701652', '77076798.8053*1453708867', '99071328.5089*1453709033', '99071328.5089*1453709033', '19194452.9873*1453709300', '47734805.4193*1453727410', '10949794.5833*1453738708', '18078565.9097*1453745718', '18078565.9097*1453745718', '18078565.9097*1453745718', '3585621.1081*1453752395', '3585621.1081*1453752395', '40459776.4877*1453758649', '40459776.4877*1453758649', '40459776.4877*1453758649', '26626505.9439*1453770702', '26626505.9439*1453770702', '58287781.0998*1453771003', '53802954.4629*1453771251', '53802954.4629*1453771251', '53802954.4629*1453771251', '69243220.5432*1453774470', '60951456.5431*1453779361', '23831938.2282*1453782354', '23831938.2282*1453782354', '23831938.2282*1453782354', '85190231.8493*1453787516', '33225707.7061*1453789032', '33225707.7061*1453789032', '33225707.7061*1453789032', '82292145.1222*1453789740', '82292145.1222*1453789740', '12291148.2056*1453794435', '93045116.7387*1453798480', '93045116.7387*1453798480', '93045116.7387*1453798480', '89434043.9695*1453801809', '89434043.9695*1453801809', '89434043.9695*1453801809', '75280765.383*1453815049', '4648439.7244*1453816126', '4648439.7244*1453816126', '4648439.7244*1453816126', '95048148.6313*1453820622', '72181688.4798*1453822041', '72181688.4798*1453822041', '72181688.4798*1453822041', '38433168.3826*1453842931', '76874232.2294*1453844453', '92183586.3414*1453845989', '92183586.3414*1453845989', '88205255.6225*1453848928', '88205255.6225*1453848928', '88205255.6225*1453848928', '68364746.6034*1453860255', '68364746.6034*1453860255', '68364746.6034*1453860255', '50337006.8329*1453860780', '25757882.1442*1453865218', '25757882.1442*1453865218', '86902863.5819*1453868103', '62285077.3053*1453870486', '20443013.577*1453874541', '66474823.2898*1453876393', '66474823.2898*1453876393', '66474823.2898*1453876393', '66474823.2898*1453876393', '91668525.9144*1453880837', '37111295.4891*1453883399', '38622441.0914*1453890445', '38622441.0914*1453890445', '38622441.0914*1453890445', '44109377.7166*1453894016', '44109377.7166*1453894016', '44109377.7166*1453894016', '58032188.9836*1453895459', '58032188.9836*1453895459', '58032188.9836*1453895459', '90333863.4001*1453899702', '95315762.6604*1453899785', '95315762.6604*1453899785', '74754951.1455*1453900307', '81787690.3366*1453900665', '81787690.3366*1453900665', '30039559.2397*1453902404', '69064093.1649*1453908840', '69064093.1649*1453908840', '69064093.1649*1453908840', '88864850.2312*1453912891', '88864850.2312*1453912891', '88864850.2312*1453912891', '9732719.52478*1453912917', '9732719.52478*1453912917', '78133866.0753*1453917717', '42850893.407*1453918507', '42850893.407*1453918507', '42850893.407*1453918507', '71700345.8797*1453920355', '71700345.8797*1453920355', '71700345.8797*1453920355', '79144210.8316*1453922682', '65600831.9268*1453925637', '1781345.14046*1453931943', '1781345.14046*1453931943', '71652674.0599*1453932043', '52363458.6197*1453932661', '34384697.6359*1453935267', '60754855.4114*1453944332', '60754855.4114*1453944332', '60754855.4114*1453944332', '92740502.8344*1453945396', '92740502.8344*1453945396', '92740502.8344*1453945396', '92740502.8344*1453945396', '79648123.6167*1453949206', '93597303.2157*1453951117', '79350136.6694*1453953288', '31451897.294*1453955830', '31451897.294*1453955830', '98793934.7476*1453964999', '86358958.5804*1453965348', '33298335.2431*1453968432', '33298335.2431*1453968432', '40328172.1781*1453977919', '18637608.1939*1453996241', '8069548.26203*1453996456', '8069548.26203*1453996456', '3450540.23222*1453999266', '90273974.6591*1454002554', '23408131.9716*1454007716', '23408131.9716*1454007716', '77344771.252*1454009509', '47518801.5644*1454009796', '50837566.5866*1454014755', '50837566.5866*1454014755', '50837566.5866*1454014755', '44647164.9101*1454016298', '44647164.9101*1454016298', '44647164.9101*1454016298', '74008221.7088*1454018760', '35793494.2762*1454024678', '35793494.2762*1454024678', '68913234.9132*1454029420', '68913234.9132*1454029420', '68913234.9132*1454029420', '66382840.9166*1454029730', '66382840.9166*1454029730', '93726608.6838*1454036981', '37209886.3552*1454040583', '86834715.8509*1454046775', '75606499.036*1454053805', '75606499.036*1454053805', '75606499.036*1454053805', '18543535.3848*1454059035', '18543535.3848*1454059035', '18543535.3848*1454059035', '49250004.3231*1454063311', '49250004.3231*1454063311', '49250004.3231*1454063311', '69012917.5192*1454063448', '69012917.5192*1454063448', '69012917.5192*1454063448', '71640924.0487*1454070027', '13045168.4609*1454082444', '13045168.4609*1454082444', '40507076.0078*1454083118', '40507076.0078*1454083118', '40507076.0078*1454083118', '50649941.3379*1454084975', '41677532.281*1454090337', '5557888.52282*1454097399', '5557888.52282*1454097399', '96949229.1948*1454113546', '96949229.1948*1454113546', '50109793.0804*1454116165', '50109793.0804*1454116165', '37203265.6864*1454118870', '37203265.6864*1454118870', '57362317.9681*1454128720', '23424416.8327*1454130410', '73509230.5583*1454131152', '73509230.5583*1454131152', '6674593.16088*1454131563', '40841735.5868*1454133166', '40841735.5868*1454133166', '12181688.1584*1454140185', '70348898.6148*1454140557', '70348898.6148*1454140557', '70348898.6148*1454140557', '56927769.084*1454140970', '52357914.2009*1454142069', '52357914.2009*1454142069', '52357914.2009*1454142069', '10077251.0717*1454153576', '56160709.1619*1454153688', '56160709.1619*1454153688', '67666635.5828*1454159939', '67666635.5828*1454159939', '67666635.5828*1454159939', '7683915.86438*1454160043', '7683915.86438*1454160043', '5420738.54883*1454163022', '49593757.2645*1454164927', '49593757.2645*1454164927', '10885085.3918*1454178197', '85950808.548*1454179603', '22234487.7406*1454188631', '93418266.0823*1454189142', '93418266.0823*1454189142', '93418266.0823*1454189142', '95800558.1113*1454193444', '95800558.1113*1454193444', '53573937.7379*1454194033', '53573937.7379*1454194033', '53573937.7379*1454194033', '63081929.4937*1454194737', '63081929.4937*1454194737', '63081929.4937*1454194737', '15167102.6746*1454198086', '15167102.6746*1454198086', '56946612.8694*1454198440'], balance_amt_2592000=['21549400.9814*1451607860', '60401992.2174*1451611951', '70215158.8144*1451614183', '70215158.8144*1451614183', '99374367.9318*1451616403', '99374367.9318*1451616403', '91843278.6796*1451623350', '44828311.9331*1451625058', '11100036.1124*1451642296', '11100036.1124*1451642296', '839759.974617*1451646892', '839759.974617*1451646892', '90755785.0398*1451647352', '66951727.7947*1451654044', '26063083.9318*1451656253', '26063083.9318*1451656253', '29791574.9576*1451656320', '42899408.762*1451657744', '42899408.762*1451657744', '42899408.762*1451657744', '77082947.4708*1451684342', '77082947.4708*1451684342', '22563996.1981*1451687279', '22563996.1981*1451687279', '51795715.1239*1451697649', '51795715.1239*1451697649', '45527212.31*1451700694', '45527212.31*1451700694', '45527212.31*1451700694', '10682201.3541*1451704165', '54257515.4436*1451710949', '54257515.4436*1451710949', '99518743.8099*1451713915', '87598806.3534*1451714043', '87598806.3534*1451714043', '87598806.3534*1451714043', '87598806.3534*1451714043', '48166326.9459*1451727555', '22783857.6912*1451730738', '24018531.8862*1451735657', '24018531.8862*1451735657', '80830511.9624*1451737630', '94168032.0617*1451739437', '94168032.0617*1451739437', '94168032.0617*1451739437', '65497964.3675*1451744337', '48162626.1282*1451744398', '25558409.3094*1451752723', '46826589.7538*1451754280', '46826589.7538*1451754280', '46826589.7538*1451754280', '46826589.7538*1451754280', '32798096.704*1451762811', '32798096.704*1451762811', '32798096.704*1451762811', '39472240.9442*1451763169', '22517648.4138*1451763638', '17352821.9793*1451778692', '51365492.5152*1451783625', '51365492.5152*1451783625', '51365492.5152*1451783625', '68815725.0423*1451786023', '23000816.569*1451798070', '23000816.569*1451798070', '55414602.3953*1451798752', '55414602.3953*1451798752', '34506568.0672*1451800725', '4669892.60012*1451803874', '4669892.60012*1451803874', '34371388.6558*1451803879', '93636937.3579*1451813728', '64521093.5878*1451816310', '64521093.5878*1451816310', '64521093.5878*1451816310', '37831009.0785*1451816738', '37831009.0785*1451816738', '37831009.0785*1451816738', '96537698.7096*1451837092', '89316369.3734*1451842707', '46209625.4203*1451844588', '46209625.4203*1451844588', '46209625.4203*1451844588', '7514041.8878*1451849291', '7514041.8878*1451849291', '54672341.8154*1451858456', '96834449.7728*1451860532', '96834449.7728*1451860532', '69145424.2992*1451863272', '36104080.8776*1451866109', '36104080.8776*1451866109', '36104080.8776*1451866109', '41085034.3379*1451869294', '17444936.5291*1451871549', '17444936.5291*1451871549', '15354450.0987*1451874436', '15354450.0987*1451874436', '15354450.0987*1451874436', '4274383.96983*1451881008', '4274383.96983*1451881008', '48174124.149*1451893789', '48174124.149*1451893789', '36858583.3293*1451897630', '36858583.3293*1451897630', '36858583.3293*1451897630', '37597903.879*1451905921', '37597903.879*1451905921', '37597903.879*1451905921', '94829762.865*1451906236', '77684304.352*1451906285', '39112700.965*1451910519', '39112700.965*1451910519', '39112700.965*1451910519', '39112700.965*1451910519', '85430569.4661*1451910908', '19027172.2921*1451921545', '70062509.7754*1451921618', '70062509.7754*1451921618', '91432736.7086*1451924664', '47700578.5858*1451937070', '47700578.5858*1451937070', '6279255.30081*1451937986', '53481792.4888*1451940275', '53481792.4888*1451940275', '53481792.4888*1451940275', '46961417.8475*1451940407', '46961417.8475*1451940407', '52410123.2217*1451947798', '52410123.2217*1451947798', '52410123.2217*1451947798', '30491901.7747*1451948169', '30491901.7747*1451948169', '30491901.7747*1451948169', '1015330.40217*1451949746', '1015330.40217*1451949746', '73075675.6549*1451950245', '73075675.6549*1451950245', '54907992.2002*1451954552', '78446793.1618*1451954918', '80455421.9752*1451955575', '80455421.9752*1451955575', '80455421.9752*1451955575', '61777895.9127*1451965797', '61777895.9127*1451965797', '2972832.64437*1451966880', '2972832.64437*1451966880', '30492662.4652*1451969401', '30492662.4652*1451969401', '90608159.8886*1451970969', '7256928.32997*1451972489', '99425008.0265*1451982108', '45720434.4438*1451982880', '45720434.4438*1451982880', '61424188.5626*1451986832', '8860811.05248*1451990236', '68743640.5353*1451994509', '68743640.5353*1451994509', '97609260.2419*1451995791', '87110435.3697*1451996117', '87110435.3697*1451996117', '87110435.3697*1451996117', '11104671.9594*1451997801', '11104671.9594*1451997801', '54669431.4024*1452000769', '84537046.263*1452007063', '7430176.26206*1452008988', '7430176.26206*1452008988', '7430176.26206*1452008988', '47464333.857*1452010149', '31496576.9687*1452010591', '31496576.9687*1452010591', '31496576.9687*1452010591', '3707140.02928*1452013132', '10084105.1771*1452028034', '50723345.6296*1452041765', '27239091.9753*1452047038', '55974982.1024*1452054902', '55974982.1024*1452054902', '55974982.1024*1452054902', '55974982.1024*1452054902', '76021526.2399*1452056349', '76021526.2399*1452056349', '32928203.3214*1452058953', '32928203.3214*1452058953', '63719691.099*1452067003', '63719691.099*1452067003', '63719691.099*1452067003', '63719691.099*1452067003', '50694184.4346*1452069224', '76384095.2107*1452073276', '76384095.2107*1452073276', '40752421.0325*1452076829', '40752421.0325*1452076829', '40752421.0325*1452076829', '40752421.0325*1452076829', '81741865.0156*1452080674', '81741865.0156*1452080674', '79125920.7319*1452099865', '84435816.85*1452113567', '84435816.85*1452113567', '77583899.5541*1452119398', '43206954.0652*1452119971', '43206954.0652*1452119971', '25207370.228*1452125095', '25207370.228*1452125095', '49091809.3248*1452125623', '49091809.3248*1452125623', '49091809.3248*1452125623', '58145176.0326*1452128987', '81665440.1836*1452129494', '94542951.6769*1452134995', '94542951.6769*1452134995', '30211066.8193*1452140012', '30211066.8193*1452140012', '2864661.88916*1452141507', '2864661.88916*1452141507', '2864661.88916*1452141507', '47921801.6018*1452142949', '2075678.25453*1452142954', '2075678.25453*1452142954', '2075678.25453*1452142954', '23083298.4434*1452144735', '25404975.963*1452150718', '39222387.4433*1452156026', '6901151.34368*1452164189', '16623777.4843*1452167835', '16623777.4843*1452167835', '82974276.0334*1452168758', '62336814.4617*1452174023', '62336814.4617*1452174023', '62336814.4617*1452174023', '72395289.9152*1452174336', '72395289.9152*1452174336', '38059552.3137*1452175467', '38059552.3137*1452175467', '38059552.3137*1452175467', '29942766.2363*1452180614', '29942766.2363*1452180614', '63365608.0821*1452190462', '85456964.0583*1452195076', '85456964.0583*1452195076', '85456964.0583*1452195076', '64451579.2426*1452196091', '64451579.2426*1452196091', '64451579.2426*1452196091', '81539727.7963*1452202758', '13348292.3602*1452206051', '79810098.7262*1452208898', '79810098.7262*1452208898', '79810098.7262*1452208898', '7611854.49726*1452214123', '7611854.49726*1452214123', '7611854.49726*1452214123', '68244732.748*1452216360', '68244732.748*1452216360', '40776335.9087*1452218280', '93267936.8825*1452219471', '93267936.8825*1452219471', '91991271.3732*1452223485', '31894925.5808*1452227880', '10264430.0751*1452231411', '10264430.0751*1452231411', '46759144.676*1452240664', '69867176.2749*1452244765', '98419997.4652*1452253151', '36940941.8432*1452260276', '36940941.8432*1452260276', '69769720.209*1452273534', '34403348.9784*1452280846', '34403348.9784*1452280846', '51883951.3099*1452282201', '60494887.8912*1452287217', '69703616.2427*1452287606', '90740086.2578*1452287620', '90740086.2578*1452287620', '90740086.2578*1452287620', '90740086.2578*1452287620', '922673.664184*1452288867', '68278446.7984*1452288890', '53905895.3904*1452301670', '14992686.4035*1452303377', '14992686.4035*1452303377', '70342713.1009*1452316951', '91105468.9167*1452324397', '16588976.8078*1452327342', '16588976.8078*1452327342', '94848978.4181*1452333329', '94848978.4181*1452333329', '67777155.5766*1452334132', '67777155.5766*1452334132', '13496649.2144*1452336906', '13496649.2144*1452336906', '16256766.3029*1452336993', '16256766.3029*1452336993', '74685833.9515*1452337219', '74685833.9515*1452337219', '74685833.9515*1452337219', '27825131.489*1452341096', '27825131.489*1452341096', '27825131.489*1452341096', '81033765.432*1452343556', '40540207.9327*1452345021', '92310409.2499*1452346423', '69336552.5568*1452348638', '80668996.6254*1452351161', '99647988.7359*1452353837', '30054712.3678*1452356249', '30054712.3678*1452356249', '10873878.1209*1452356280', '10873878.1209*1452356280', '10873878.1209*1452356280', '42175684.0828*1452360188', '62334265.2272*1452371127', '52910737.3575*1452372730', '74500195.6964*1452379508', '74500195.6964*1452379508', '74500195.6964*1452379508', '60615210.6449*1452383896', '60615210.6449*1452383896', '60615210.6449*1452383896', '44265928.5164*1452387967', '44265928.5164*1452387967', '36947925.2288*1452394703', '65231622.553*1452398529', '20775605.7234*1452399973', '17740169.5561*1452405394', '17740169.5561*1452405394', '17740169.5561*1452405394', '90597350.9283*1452412119', '53122125.9345*1452413435', '53122125.9345*1452413435', '84573235.4747*1452420153', '84573235.4747*1452420153', '84573235.4747*1452420153', '43280235.8697*1452424028', '1453805.77613*1452425880', '1453805.77613*1452425880', '1453805.77613*1452425880', '44707418.0022*1452430786', '8891093.67508*1452438361', '8891093.67508*1452438361', '23840078.3162*1452438952', '23840078.3162*1452438952', '23840078.3162*1452438952', '34632829.6523*1452441177', '49256392.8226*1452449327', '88744125.5219*1452451397', '88744125.5219*1452451397', '74134672.3631*1452458152', '74134672.3631*1452458152', '74134672.3631*1452458152', '74134672.3631*1452458152', '27972711.508*1452462825', '61031609.079*1452463430', '51627654.3039*1452468868', '9456209.52273*1452480396', '51475097.7973*1452492151', '18063723.2025*1452492711', '86952632.9554*1452495175', '83878924.0499*1452495313', '87071940.3248*1452501428', '87071940.3248*1452501428', '87071940.3248*1452501428', '70421412.6731*1452505965', '3897003.20025*1452507859', '88953670.0063*1452525036', '88953670.0063*1452525036', '88953670.0063*1452525036', '75537948.7366*1452528419', '75537948.7366*1452528419', '75537948.7366*1452528419', '75537948.7366*1452528419', '1640882.96693*1452531371', '1640882.96693*1452531371', '1640882.96693*1452531371', '81448006.3291*1452532683', '47544813.1257*1452536032', '48100653.2662*1452536984', '40625928.5898*1452537896', '99487080.8258*1452538235', '64903060.2595*1452539800', '64903060.2595*1452539800', '64903060.2595*1452539800', '26921614.8511*1452543678', '26921614.8511*1452543678', '31236906.3788*1452546192', '31236906.3788*1452546192', '31236906.3788*1452546192', '9982236.97957*1452552002', '9982236.97957*1452552002', '79332709.9743*1452555711', '39716932.2075*1452559315', '75378641.805*1452560864', '127776.198993*1452564626', '40579437.3889*1452566959', '40579437.3889*1452566959', '16391776.9224*1452567830', '16391776.9224*1452567830', '16391776.9224*1452567830', '14152233.9722*1452570571', '14152233.9722*1452570571', '58268388.9761*1452575051', '50881691.6899*1452576922', '87414169.4691*1452578263', '87414169.4691*1452578263', '87414169.4691*1452578263', '3355736.85765*1452581508', '79788439.5822*1452581752', '79788439.5822*1452581752', '79788439.5822*1452581752', '36473854.4953*1452586837', '37265223.3775*1452595587', '37265223.3775*1452595587', '49419451.2307*1452599920', '53682571.3346*1452604122', '53682571.3346*1452604122', '49147368.1334*1452606334', '74358986.3341*1452611559', '31790500.4877*1452615816', '8425549.64514*1452616504', '95709674.6959*1452618393', '95709674.6959*1452618393', '95709674.6959*1452618393', '62617696.8125*1452633689', '62617696.8125*1452633689', '49554413.24*1452634324', '84263198.4547*1452635199', '86657360.9525*1452635710', '86657360.9525*1452635710', '86657360.9525*1452635710', '97113187.9364*1452648070', '97113187.9364*1452648070', '97113187.9364*1452648070', '10452405.9158*1452650689', '10452405.9158*1452650689', '46722852.502*1452652212', '47718115.247*1452653775', '37030879.8365*1452655684', '37030879.8365*1452655684', '7400891.56672*1452659005', '7400891.56672*1452659005', '7400891.56672*1452659005', '84681574.1415*1452660679', '90340421.1518*1452665649', '92408710.4905*1452665946', '92408710.4905*1452665946', '92408710.4905*1452665946', '40941702.4104*1452668562', '19135350.0295*1452672902', '19135350.0295*1452672902', '31160141.378*1452677155', '71137259.6999*1452677206', '78083246.0698*1452680665', '30789660.014*1452687642', '30789660.014*1452687642', '30789660.014*1452687642', '13784662.7245*1452690354', '46886937.2298*1452692188', '46886937.2298*1452692188', '46886937.2298*1452692188', '14134390.5681*1452693028', '97410920.0956*1452694779', '97410920.0956*1452694779', '65385550.1021*1452703682', '65385550.1021*1452703682', '4615222.32957*1452707474', '4615222.32957*1452707474', '4615222.32957*1452707474', '35967751.3466*1452714322', '35967751.3466*1452714322', '35967751.3466*1452714322', '21475182.4811*1452718442', '21475182.4811*1452718442', '21475182.4811*1452718442', '86910652.9409*1452727180', '86910652.9409*1452727180', '39859470.93*1452732126', '39859470.93*1452732126', '39859470.93*1452732126', '46923295.6953*1452736674', '49236328.2958*1452741984', '49236328.2958*1452741984', '49236328.2958*1452741984', '49236328.2958*1452741984', '13242041.5526*1452747674', '13242041.5526*1452747674', '13242041.5526*1452747674', '10060277.4428*1452753019', '10060277.4428*1452753019', '10060277.4428*1452753019', '92847583.9763*1452753785', '87343715.8341*1452755328', '35716974.6802*1452756579', '22486929.0963*1452757700', '22486929.0963*1452757700', '22486929.0963*1452757700', '31049642.4266*1452763603', '31049642.4266*1452763603', '31049642.4266*1452763603', '14111064.1248*1452766808', '14111064.1248*1452766808', '71868844.0765*1452768022', '71868844.0765*1452768022', '67252794.6894*1452779418', '86655784.933*1452781794', '98063265.4336*1452782503', '98063265.4336*1452782503', '99595698.8234*1452793336', '63152933.3473*1452794520', '63152933.3473*1452794520', '89283436.2502*1452794792', '89283436.2502*1452794792', '89283436.2502*1452794792', '51862479.8439*1452796793', '50181894.4316*1452798084', '50181894.4316*1452798084', '95623657.6849*1452799626', '5620796.12616*1452801801', '48915884.1884*1452804298', '48915884.1884*1452804298', '48915884.1884*1452804298', '97013000.9486*1452807355', '97013000.9486*1452807355', '27307294.327*1452815816', '31347110.238*1452816239', '31347110.238*1452816239', '67755332.1818*1452819141', '87826205.9459*1452820570', '6951677.52548*1452821166', '6951677.52548*1452821166', '21204061.5101*1452823522', '21204061.5101*1452823522', '87120900.2168*1452830255', '18071486.61*1452831852', '71981161.985*1452837255', '71981161.985*1452837255', '71981161.985*1452837255', '53807383.4417*1452841216', '88107714.393*1452845969', '88107714.393*1452845969', '24857286.7119*1452848232', '24857286.7119*1452848232', '24857286.7119*1452848232', '86038212.9054*1452853235', '79479915.6147*1452853320', '58279398.2226*1452854090', '58279398.2226*1452854090', '7825229.93052*1452855899', '95234184.983*1452856501', '82652275.5729*1452858359', '19897413.5274*1452865700', '19897413.5274*1452865700', '19897413.5274*1452865700', '74250787.1406*1452879203', '74250787.1406*1452879203', '74250787.1406*1452879203', '10776111.6784*1452888817', '83112463.5546*1452888904', '51122305.7362*1452903894', '44308477.0351*1452908307', '2931432.40423*1452909057', '2931432.40423*1452909057', '2931432.40423*1452909057', '46923187.2473*1452909693', '46923187.2473*1452909693', '46923187.2473*1452909693', '31754495.1107*1452917783', '31754495.1107*1452917783', '31754495.1107*1452917783', '79421334.5204*1452918657', '73732214.6236*1452921598', '73732214.6236*1452921598', '73732214.6236*1452921598', '92055475.7731*1452923510', '59021021.777*1452929465', '97570480.8723*1452937982', '82168230.0448*1452939253', '82168230.0448*1452939253', '82168230.0448*1452939253', '16681568.3348*1452945101', '38842653.6322*1452946274', '57551953.1555*1452947627', '19323330.4829*1452955306', '72664406.746*1452958646', '72664406.746*1452958646', '89073971.0564*1452959901', '59180767.8461*1452960650', '59180767.8461*1452960650', '59180767.8461*1452960650', '77087629.3271*1452965865', '77087629.3271*1452965865', '84336407.8736*1452968117', '59221794.181*1452968455', '59221794.181*1452968455', '59221794.181*1452968455', '59221794.181*1452968455', '31130284.5046*1452975707', '31130284.5046*1452975707', '8206255.63261*1452975753', '8206255.63261*1452975753', '32183490.9023*1452990876', '33641783.262*1453000153', '33641783.262*1453000153', '12353732.8616*1453002236', '12353732.8616*1453002236', '33445163.8853*1453007746', '33445163.8853*1453007746', '11098297.0701*1453008152', '11098297.0701*1453008152', '16671267.8857*1453008418', '16671267.8857*1453008418', '16671267.8857*1453008418', '96273295.687*1453016507', '96273295.687*1453016507', '83822477.489*1453022075', '83822477.489*1453022075', '83822477.489*1453022075', '35253585.4196*1453023061', '35253585.4196*1453023061', '35253585.4196*1453023061', '60095160.281*1453025083', '60095160.281*1453025083', '82954273.7399*1453029252', '2736329.8221*1453033290', '2736329.8221*1453033290', '81827133.4667*1453033841', '81827133.4667*1453033841', '34361292.9317*1453035181', '58362486.2747*1453042488', '10269310.8088*1453042834', '75257613.6019*1453045271', '11818906.1394*1453047910', '11818906.1394*1453047910', '22409812.4896*1453049807', '81585893.9882*1453053092', '74943636.6335*1453053218', '37846296.9214*1453056246', '37846296.9214*1453056246', '5321044.67382*1453062175', '79140630.9011*1453064064', '79140630.9011*1453064064', '79140630.9011*1453064064', '56547856.7484*1453064412', '15003670.4346*1453066114', '30633153.6555*1453080014', '30633153.6555*1453080014', '62766957.1312*1453082412', '62766957.1312*1453082412', '66691074.5616*1453084155', '66691074.5616*1453084155', '12887237.655*1453098110', '12887237.655*1453098110', '12887237.655*1453098110', '19266644.9962*1453098407', '38715024.2424*1453103020', '65991207.9335*1453103934', '65991207.9335*1453103934', '65991207.9335*1453103934', '75902619.2404*1453106140', '3120553.19323*1453109334', '3120553.19323*1453109334', '76897693.7249*1453122069', '41153182.3173*1453134142', '41153182.3173*1453134142', '17617007.1425*1453145614', '63464484.2372*1453146650', '44922615.5099*1453147559', '83777974.654*1453163186', '97712113.4449*1453165799', '97712113.4449*1453165799', '97712113.4449*1453165799', '18883728.2745*1453169901', '81500457.4943*1453175254', '67084384.2351*1453177484', '58140155.974*1453180858', '72281156.753*1453185203', '74361251.3881*1453186277', '74361251.3881*1453186277', '63602492.592*1453187208', '48622646.0157*1453194521', '48622646.0157*1453194521', '17838518.3106*1453201571', '99253365.6946*1453206175', '99253365.6946*1453206175', '99253365.6946*1453206175', '20506511.2528*1453212741', '20506511.2528*1453212741', '20506511.2528*1453212741', '4117410.20126*1453215470', '27123095.6723*1453218526', '27123095.6723*1453218526', '1311082.98624*1453224654', '1311082.98624*1453224654', '1311082.98624*1453224654', '8572735.12207*1453227295', '8572735.12207*1453227295', '8572735.12207*1453227295', '48302360.6252*1453233195', '44988375.7955*1453236054', '44988375.7955*1453236054', '44988375.7955*1453236054', '73820996.4466*1453236124', '73820996.4466*1453236124', '73829955.2598*1453246994', '73829955.2598*1453246994', '13642937.902*1453249205', '13642937.902*1453249205', '4905001.25304*1453251516', '4905001.25304*1453251516', '93259774.6114*1453256965', '93259774.6114*1453256965', '58734823.5516*1453260668', '58734823.5516*1453260668', '58734823.5516*1453260668', '18097634.8153*1453265382', '6776793.38922*1453266315', '90285628.4897*1453268867', '90285628.4897*1453268867', '26470597.5989*1453273281', '26470597.5989*1453273281', '98645230.5123*1453274408', '8577798.82841*1453278609', '8577798.82841*1453278609', '42486742.698*1453284558', '42486742.698*1453284558', '33376930.6735*1453290881', '33376930.6735*1453290881', '33376930.6735*1453290881', '59048792.6291*1453294928', '59048792.6291*1453294928', '59048792.6291*1453294928', '95131144.8965*1453297000', '17623061.286*1453298606', '17623061.286*1453298606', '8465867.40507*1453299440', '8465867.40507*1453299440', '94560418.7958*1453299731', '94560418.7958*1453299731', '17822718.3286*1453299753', '44978406.438*1453304242', '80443615.3292*1453305248', '70503341.4991*1453307354', '19277629.3925*1453331761', '19277629.3925*1453331761', '19277629.3925*1453331761', '7359046.86957*1453332005', '7359046.86957*1453332005', '7359046.86957*1453332005', '62384785.209*1453332112', '62384785.209*1453332112', '64296054.5316*1453335819', '91881321.0565*1453343936', '91881321.0565*1453343936', '91881321.0565*1453343936', '84564906.436*1453353213', '84564906.436*1453353213', '84564906.436*1453353213', '91590410.656*1453361398', '91590410.656*1453361398', '65249791.4563*1453372768', '65249791.4563*1453372768', '27366008.2974*1453379610', '27366008.2974*1453379610', '50936757.529*1453382695', '50936757.529*1453382695', '11659353.8589*1453386744', '42591545.6313*1453390947', '42591545.6313*1453390947', '42591545.6313*1453390947', '13502143.6546*1453397681', '13502143.6546*1453397681', '68289000.9998*1453398547', '53675911.398*1453398592', '83099164.5047*1453402630', '20588442.2378*1453411981', '20588442.2378*1453411981', '76772344.6105*1453420059', '76772344.6105*1453420059', '76772344.6105*1453420059', '73516685.2812*1453423004', '27739061.8828*1453424792', '27739061.8828*1453424792', '27739061.8828*1453424792', '25028337.0755*1453424996', '15071237.7897*1453425660', '15071237.7897*1453425660', '15071237.7897*1453425660', '89591683.4991*1453427992', '89591683.4991*1453427992', '83972474.7368*1453432440', '83972474.7368*1453432440', '83972474.7368*1453432440', '42002864.9403*1453441705', '42002864.9403*1453441705', '42002864.9403*1453441705', '97195950.2381*1453447625', '1423692.39109*1453455358', '1423692.39109*1453455358', '1423692.39109*1453455358', '35997494.1928*1453457120', '22273220.8399*1453460459', '47336002.3663*1453461809', '51029821.6691*1453463349', '80078020.6985*1453465523', '80078020.6985*1453465523', '80078020.6985*1453465523', '88759519.9992*1453466342', '88759519.9992*1453466342', '88759519.9992*1453466342', '70584092.8439*1453466660', '70584092.8439*1453466660', '5510497.84231*1453473378', '61672654.114*1453475784', '43336752.3458*1453479488', '96569513.1691*1453482148', '96569513.1691*1453482148', '13024300.6596*1453482876', '13024300.6596*1453482876', '13024300.6596*1453482876', '77720499.3968*1453486849', '77720499.3968*1453486849', '77720499.3968*1453486849', '77720499.3968*1453486849', '91592575.1255*1453496513', '91592575.1255*1453496513', '91592575.1255*1453496513', '17820930.5475*1453501265', '40358470.8532*1453502539', '43481686.4229*1453505090', '51674728.5157*1453507345', '51674728.5157*1453507345', '51674728.5157*1453507345', '18086123.2087*1453513492', '18086123.2087*1453513492', '75078563.0442*1453513833', '75078563.0442*1453513833', '68113166.4446*1453517484', '68113166.4446*1453517484', '48479392.6727*1453519353', '16132542.4016*1453521824', '16132542.4016*1453521824', '15718700.9949*1453522238', '15718700.9949*1453522238', '48356278.1179*1453523098', '99896811.4967*1453524851', '89678597.8527*1453534129', '89678597.8527*1453534129', '89678597.8527*1453534129', '80101751.0062*1453539206', '80101751.0062*1453539206', '5273803.73778*1453547182', '96624504.9565*1453547526', '34151618.6498*1453553770', '34151618.6498*1453553770', '34151618.6498*1453553770', '76264652.3326*1453558558', '76264652.3326*1453558558', '99469429.5039*1453559946', '99469429.5039*1453559946', '29782880.7629*1453560825', '29782880.7629*1453560825', '41420925.2076*1453561734', '41420925.2076*1453561734', '69137260.1477*1453563835', '69137260.1477*1453563835', '69137260.1477*1453563835', '48261782.6061*1453573999', '80755256.7828*1453576375', '69899600.5508*1453576992', '1431420.34572*1453585041', '1431420.34572*1453585041', '1431420.34572*1453585041', '54636569.5698*1453587326', '74117356.3186*1453590343', '17634010.5248*1453594313', '17634010.5248*1453594313', '91582954.7976*1453597829', '97784067.3767*1453601341', '38337085.8721*1453605272', '38337085.8721*1453605272', '52494755.4954*1453605592', '50963649.6318*1453610104', '50963649.6318*1453610104', '50963649.6318*1453610104', '53491701.8775*1453615656', '53491701.8775*1453615656', '35209082.2015*1453621894', '35209082.2015*1453621894', '35209082.2015*1453621894', '91280269.699*1453629550', '91280269.699*1453629550', '4865989.77523*1453631116', '43274000.3669*1453631436', '9607430.67394*1453631592', '9607430.67394*1453631592', '9607430.67394*1453631592', '40873507.36*1453652357', '64185576.6517*1453657764', '20414957.8356*1453661080', '20414957.8356*1453661080', '73913981.558*1453671680', '30374646.2081*1453671943', '63907252.1498*1453672945', '63907252.1498*1453672945', '4044969.13112*1453676312', '4044969.13112*1453676312', '59408171.7845*1453677354', '30216070.1019*1453679448', '30216070.1019*1453679448', '30216070.1019*1453679448', '98798073.2091*1453680882', '98798073.2091*1453680882', '40447822.9378*1453683097', '40447822.9378*1453683097', '70270870.5638*1453684671', '13556673.4518*1453687587', '13556673.4518*1453687587', '30736088.8419*1453688183', '44233733.0939*1453689175', '44233733.0939*1453689175', '44233733.0939*1453689175', '75805962.6895*1453690171', '74143353.2734*1453698824', '50335199.1181*1453700380', '21063226.8916*1453700935', '21063226.8916*1453700935', '21063226.8916*1453700935', '17380973.6998*1453701543', '4508121.46336*1453701607', '4508121.46336*1453701607', '33677785.467*1453701652', '33677785.467*1453701652', '33677785.467*1453701652', '24937822.6175*1453708867', '27883021.1772*1453709033', '27883021.1772*1453709033', '27898005.9732*1453709300', '62925399.605*1453727410', '20509483.1303*1453738708', '51820988.6722*1453745718', '51820988.6722*1453745718', '51820988.6722*1453745718', '69707399.6941*1453752395', '69707399.6941*1453752395', '68061986.2825*1453758649', '68061986.2825*1453758649', '68061986.2825*1453758649', '53901021.5472*1453770702', '53901021.5472*1453770702', '70542814.2353*1453771003', '66754917.5958*1453771251', '66754917.5958*1453771251', '66754917.5958*1453771251', '42270927.777*1453774470', '16686395.0142*1453779361', '9673956.77767*1453782354', '9673956.77767*1453782354', '9673956.77767*1453782354', '92769830.2594*1453787516', '4007140.47498*1453789032', '4007140.47498*1453789032', '4007140.47498*1453789032', '34484812.4058*1453789740', '34484812.4058*1453789740', '20763430.979*1453794435', '16016267.3619*1453798480', '16016267.3619*1453798480', '16016267.3619*1453798480', '74611358.0075*1453801809', '74611358.0075*1453801809', '74611358.0075*1453801809', '65835218.41*1453815049', '87262060.5073*1453816126', '87262060.5073*1453816126', '87262060.5073*1453816126', '60620608.5511*1453820622', '92254152.7069*1453822041', '92254152.7069*1453822041', '92254152.7069*1453822041', '19603138.5699*1453842931', '64054640.3568*1453844453', '79314416.5579*1453845989', '79314416.5579*1453845989', '55415562.3819*1453848928', '55415562.3819*1453848928', '55415562.3819*1453848928', '18818600.962*1453860255', '18818600.962*1453860255', '18818600.962*1453860255', '78799879.9494*1453860780', '52788985.1233*1453865218', '52788985.1233*1453865218', '39344643.7533*1453868103', '93197662.5965*1453870486', '14910013.9878*1453874541', '64226087.099*1453876393', '64226087.099*1453876393', '64226087.099*1453876393', '64226087.099*1453876393', '29441843.414*1453880837', '96184548.3261*1453883399', '52952748.1799*1453890445', '52952748.1799*1453890445', '52952748.1799*1453890445', '81354552.377*1453894016', '81354552.377*1453894016', '81354552.377*1453894016', '54986404.917*1453895459', '54986404.917*1453895459', '54986404.917*1453895459', '18760505.2221*1453899702', '32680146.902*1453899785', '32680146.902*1453899785', '5950137.36522*1453900307', '36717160.092*1453900665', '36717160.092*1453900665', '81782514.061*1453902404', '2072196.97026*1453908840', '2072196.97026*1453908840', '2072196.97026*1453908840', '34993036.0262*1453912891', '34993036.0262*1453912891', '34993036.0262*1453912891', '35624983.4423*1453912917', '35624983.4423*1453912917', '24585893.4524*1453917717', '75057216.4553*1453918507', '75057216.4553*1453918507', '75057216.4553*1453918507', '82394689.6963*1453920355', '82394689.6963*1453920355', '82394689.6963*1453920355', '7006986.12709*1453922682', '61807890.3136*1453925637', '14791806.2709*1453931943', '14791806.2709*1453931943', '87451120.4048*1453932043', '52124253.1817*1453932661', '58150978.8748*1453935267', '29185234.7669*1453944332', '29185234.7669*1453944332', '29185234.7669*1453944332', '27015809.2721*1453945396', '27015809.2721*1453945396', '27015809.2721*1453945396', '27015809.2721*1453945396', '22115226.4806*1453949206', '62172867.5377*1453951117', '1468302.46954*1453953288', '92129636.2383*1453955830', '92129636.2383*1453955830', '1923087.88895*1453964999', '6786253.51535*1453965348', '81077243.9756*1453968432', '81077243.9756*1453968432', '8948424.87601*1453977919', '43808728.7029*1453996241', '96758948.9884*1453996456', '96758948.9884*1453996456', '23427193.7099*1453999266', '7094821.85848*1454002554', '49159775.8003*1454007716', '49159775.8003*1454007716', '56116739.2669*1454009509', '20580819.6264*1454009796', '10093220.9476*1454014755', '10093220.9476*1454014755', '10093220.9476*1454014755', '4543805.18693*1454016298', '4543805.18693*1454016298', '4543805.18693*1454016298', '79133208.1736*1454018760', '1986764.2586*1454024678', '1986764.2586*1454024678', '19665394.0035*1454029420', '19665394.0035*1454029420', '19665394.0035*1454029420', '32876097.4237*1454029730', '32876097.4237*1454029730', '52777158.669*1454036981', '41010968.8491*1454040583', '92670526.4907*1454046775', '26872018.5645*1454053805', '26872018.5645*1454053805', '26872018.5645*1454053805', '45940806.8599*1454059035', '45940806.8599*1454059035', '45940806.8599*1454059035', '83756923.8723*1454063311', '83756923.8723*1454063311', '83756923.8723*1454063311', '59474313.2289*1454063448', '59474313.2289*1454063448', '59474313.2289*1454063448', '14491832.4731*1454070027', '80553770.3436*1454082444', '80553770.3436*1454082444', '7149703.13025*1454083118', '7149703.13025*1454083118', '7149703.13025*1454083118', '37028287.2045*1454084975', '33854958.1214*1454090337', '2376904.65425*1454097399', '2376904.65425*1454097399', '13675413.3762*1454113546', '13675413.3762*1454113546', '79002317.8937*1454116165', '79002317.8937*1454116165', '71602920.5652*1454118870', '71602920.5652*1454118870', '15362664.5716*1454128720', '4005836.05829*1454130410', '76125426.9766*1454131152', '76125426.9766*1454131152', '39203037.2748*1454131563', '36866475.4917*1454133166', '36866475.4917*1454133166', '81691732.7176*1454140185', '24210791.4819*1454140557', '24210791.4819*1454140557', '24210791.4819*1454140557', '23330089.8486*1454140970', '81681856.6024*1454142069', '81681856.6024*1454142069', '81681856.6024*1454142069', '84595004.6099*1454153576', '31544411.9384*1454153688', '31544411.9384*1454153688', '68302180.5875*1454159939', '68302180.5875*1454159939', '68302180.5875*1454159939', '2030180.53075*1454160043', '2030180.53075*1454160043', '58458821.7465*1454163022', '16487315.6306*1454164927', '16487315.6306*1454164927', '10629933.6167*1454178197', '44720971.8641*1454179603', '37161073.8237*1454188631', '10294931.9286*1454189142', '10294931.9286*1454189142', '10294931.9286*1454189142', '68135282.615*1454193444', '68135282.615*1454193444', '75010356.6128*1454194033', '75010356.6128*1454194033', '75010356.6128*1454194033', '44509420.309*1454194737', '44509420.309*1454194737', '44509420.309*1454194737', '36417858.8783*1454198086', '36417858.8783*1454198086', '67275135.7591*1454198440'], u_time_2592000=['1451607860', '1451611951', '1451614183', '1451614183', '1451616403', '1451616403', '1451623350', '1451625058', '1451642296', '1451642296', '1451646892', '1451646892', '1451647352', '1451654044', '1451656253', '1451656253', '1451656320', '1451657744', '1451657744', '1451657744', '1451684342', '1451684342', '1451687279', '1451687279', '1451697649', '1451697649', '1451700694', '1451700694', '1451700694', '1451704165', '1451710949', '1451710949', '1451713915', '1451714043', '1451714043', '1451714043', '1451714043', '1451727555', '1451730738', '1451735657', '1451735657', '1451737630', '1451739437', '1451739437', '1451739437', '1451744337', '1451744398', '1451752723', '1451754280', '1451754280', '1451754280', '1451754280', '1451762811', '1451762811', '1451762811', '1451763169', '1451763638', '1451778692', '1451783625', '1451783625', '1451783625', '1451786023', '1451798070', '1451798070', '1451798752', '1451798752', '1451800725', '1451803874', '1451803874', '1451803879', '1451813728', '1451816310', '1451816310', '1451816310', '1451816738', '1451816738', '1451816738', '1451837092', '1451842707', '1451844588', '1451844588', '1451844588', '1451849291', '1451849291', '1451858456', '1451860532', '1451860532', '1451863272', '1451866109', '1451866109', '1451866109', '1451869294', '1451871549', '1451871549', '1451874436', '1451874436', '1451874436', '1451881008', '1451881008', '1451893789', '1451893789', '1451897630', '1451897630', '1451897630', '1451905921', '1451905921', '1451905921', '1451906236', '1451906285', '1451910519', '1451910519', '1451910519', '1451910519', '1451910908', '1451921545', '1451921618', '1451921618', '1451924664', '1451937070', '1451937070', '1451937986', '1451940275', '1451940275', '1451940275', '1451940407', '1451940407', '1451947798', '1451947798', '1451947798', '1451948169', '1451948169', '1451948169', '1451949746', '1451949746', '1451950245', '1451950245', '1451954552', '1451954918', '1451955575', '1451955575', '1451955575', '1451965797', '1451965797', '1451966880', '1451966880', '1451969401', '1451969401', '1451970969', '1451972489', '1451982108', '1451982880', '1451982880', '1451986832', '1451990236', '1451994509', '1451994509', '1451995791', '1451996117', '1451996117', '1451996117', '1451997801', '1451997801', '1452000769', '1452007063', '1452008988', '1452008988', '1452008988', '1452010149', '1452010591', '1452010591', '1452010591', '1452013132', '1452028034', '1452041765', '1452047038', '1452054902', '1452054902', '1452054902', '1452054902', '1452056349', '1452056349', '1452058953', '1452058953', '1452067003', '1452067003', '1452067003', '1452067003', '1452069224', '1452073276', '1452073276', '1452076829', '1452076829', '1452076829', '1452076829', '1452080674', '1452080674', '1452099865', '1452113567', '1452113567', '1452119398', '1452119971', '1452119971', '1452125095', '1452125095', '1452125623', '1452125623', '1452125623', '1452128987', '1452129494', '1452134995', '1452134995', '1452140012', '1452140012', '1452141507', '1452141507', '1452141507', '1452142949', '1452142954', '1452142954', '1452142954', '1452144735', '1452150718', '1452156026', '1452164189', '1452167835', '1452167835', '1452168758', '1452174023', '1452174023', '1452174023', '1452174336', '1452174336', '1452175467', '1452175467', '1452175467', '1452180614', '1452180614', '1452190462', '1452195076', '1452195076', '1452195076', '1452196091', '1452196091', '1452196091', '1452202758', '1452206051', '1452208898', '1452208898', '1452208898', '1452214123', '1452214123', '1452214123', '1452216360', '1452216360', '1452218280', '1452219471', '1452219471', '1452223485', '1452227880', '1452231411', '1452231411', '1452240664', '1452244765', '1452253151', '1452260276', '1452260276', '1452273534', '1452280846', '1452280846', '1452282201', '1452287217', '1452287606', '1452287620', '1452287620', '1452287620', '1452287620', '1452288867', '1452288890', '1452301670', '1452303377', '1452303377', '1452316951', '1452324397', '1452327342', '1452327342', '1452333329', '1452333329', '1452334132', '1452334132', '1452336906', '1452336906', '1452336993', '1452336993', '1452337219', '1452337219', '1452337219', '1452341096', '1452341096', '1452341096', '1452343556', '1452345021', '1452346423', '1452348638', '1452351161', '1452353837', '1452356249', '1452356249', '1452356280', '1452356280', '1452356280', '1452360188', '1452371127', '1452372730', '1452379508', '1452379508', '1452379508', '1452383896', '1452383896', '1452383896', '1452387967', '1452387967', '1452394703', '1452398529', '1452399973', '1452405394', '1452405394', '1452405394', '1452412119', '1452413435', '1452413435', '1452420153', '1452420153', '1452420153', '1452424028', '1452425880', '1452425880', '1452425880', '1452430786', '1452438361', '1452438361', '1452438952', '1452438952', '1452438952', '1452441177', '1452449327', '1452451397', '1452451397', '1452458152', '1452458152', '1452458152', '1452458152', '1452462825', '1452463430', '1452468868', '1452480396', '1452492151', '1452492711', '1452495175', '1452495313', '1452501428', '1452501428', '1452501428', '1452505965', '1452507859', '1452525036', '1452525036', '1452525036', '1452528419', '1452528419', '1452528419', '1452528419', '1452531371', '1452531371', '1452531371', '1452532683', '1452536032', '1452536984', '1452537896', '1452538235', '1452539800', '1452539800', '1452539800', '1452543678', '1452543678', '1452546192', '1452546192', '1452546192', '1452552002', '1452552002', '1452555711', '1452559315', '1452560864', '1452564626', '1452566959', '1452566959', '1452567830', '1452567830', '1452567830', '1452570571', '1452570571', '1452575051', '1452576922', '1452578263', '1452578263', '1452578263', '1452581508', '1452581752', '1452581752', '1452581752', '1452586837', '1452595587', '1452595587', '1452599920', '1452604122', '1452604122', '1452606334', '1452611559', '1452615816', '1452616504', '1452618393', '1452618393', '1452618393', '1452633689', '1452633689', '1452634324', '1452635199', '1452635710', '1452635710', '1452635710', '1452648070', '1452648070', '1452648070', '1452650689', '1452650689', '1452652212', '1452653775', '1452655684', '1452655684', '1452659005', '1452659005', '1452659005', '1452660679', '1452665649', '1452665946', '1452665946', '1452665946', '1452668562', '1452672902', '1452672902', '1452677155', '1452677206', '1452680665', '1452687642', '1452687642', '1452687642', '1452690354', '1452692188', '1452692188', '1452692188', '1452693028', '1452694779', '1452694779', '1452703682', '1452703682', '1452707474', '1452707474', '1452707474', '1452714322', '1452714322', '1452714322', '1452718442', '1452718442', '1452718442', '1452727180', '1452727180', '1452732126', '1452732126', '1452732126', '1452736674', '1452741984', '1452741984', '1452741984', '1452741984', '1452747674', '1452747674', '1452747674', '1452753019', '1452753019', '1452753019', '1452753785', '1452755328', '1452756579', '1452757700', '1452757700', '1452757700', '1452763603', '1452763603', '1452763603', '1452766808', '1452766808', '1452768022', '1452768022', '1452779418', '1452781794', '1452782503', '1452782503', '1452793336', '1452794520', '1452794520', '1452794792', '1452794792', '1452794792', '1452796793', '1452798084', '1452798084', '1452799626', '1452801801', '1452804298', '1452804298', '1452804298', '1452807355', '1452807355', '1452815816', '1452816239', '1452816239', '1452819141', '1452820570', '1452821166', '1452821166', '1452823522', '1452823522', '1452830255', '1452831852', '1452837255', '1452837255', '1452837255', '1452841216', '1452845969', '1452845969', '1452848232', '1452848232', '1452848232', '1452853235', '1452853320', '1452854090', '1452854090', '1452855899', '1452856501', '1452858359', '1452865700', '1452865700', '1452865700', '1452879203', '1452879203', '1452879203', '1452888817', '1452888904', '1452903894', '1452908307', '1452909057', '1452909057', '1452909057', '1452909693', '1452909693', '1452909693', '1452917783', '1452917783', '1452917783', '1452918657', '1452921598', '1452921598', '1452921598', '1452923510', '1452929465', '1452937982', '1452939253', '1452939253', '1452939253', '1452945101', '1452946274', '1452947627', '1452955306', '1452958646', '1452958646', '1452959901', '1452960650', '1452960650', '1452960650', '1452965865', '1452965865', '1452968117', '1452968455', '1452968455', '1452968455', '1452968455', '1452975707', '1452975707', '1452975753', '1452975753', '1452990876', '1453000153', '1453000153', '1453002236', '1453002236', '1453007746', '1453007746', '1453008152', '1453008152', '1453008418', '1453008418', '1453008418', '1453016507', '1453016507', '1453022075', '1453022075', '1453022075', '1453023061', '1453023061', '1453023061', '1453025083', '1453025083', '1453029252', '1453033290', '1453033290', '1453033841', '1453033841', '1453035181', '1453042488', '1453042834', '1453045271', '1453047910', '1453047910', '1453049807', '1453053092', '1453053218', '1453056246', '1453056246', '1453062175', '1453064064', '1453064064', '1453064064', '1453064412', '1453066114', '1453080014', '1453080014', '1453082412', '1453082412', '1453084155', '1453084155', '1453098110', '1453098110', '1453098110', '1453098407', '1453103020', '1453103934', '1453103934', '1453103934', '1453106140', '1453109334', '1453109334', '1453122069', '1453134142', '1453134142', '1453145614', '1453146650', '1453147559', '1453163186', '1453165799', '1453165799', '1453165799', '1453169901', '1453175254', '1453177484', '1453180858', '1453185203', '1453186277', '1453186277', '1453187208', '1453194521', '1453194521', '1453201571', '1453206175', '1453206175', '1453206175', '1453212741', '1453212741', '1453212741', '1453215470', '1453218526', '1453218526', '1453224654', '1453224654', '1453224654', '1453227295', '1453227295', '1453227295', '1453233195', '1453236054', '1453236054', '1453236054', '1453236124', '1453236124', '1453246994', '1453246994', '1453249205', '1453249205', '1453251516', '1453251516', '1453256965', '1453256965', '1453260668', '1453260668', '1453260668', '1453265382', '1453266315', '1453268867', '1453268867', '1453273281', '1453273281', '1453274408', '1453278609', '1453278609', '1453284558', '1453284558', '1453290881', '1453290881', '1453290881', '1453294928', '1453294928', '1453294928', '1453297000', '1453298606', '1453298606', '1453299440', '1453299440', '1453299731', '1453299731', '1453299753', '1453304242', '1453305248', '1453307354', '1453331761', '1453331761', '1453331761', '1453332005', '1453332005', '1453332005', '1453332112', '1453332112', '1453335819', '1453343936', '1453343936', '1453343936', '1453353213', '1453353213', '1453353213', '1453361398', '1453361398', '1453372768', '1453372768', '1453379610', '1453379610', '1453382695', '1453382695', '1453386744', '1453390947', '1453390947', '1453390947', '1453397681', '1453397681', '1453398547', '1453398592', '1453402630', '1453411981', '1453411981', '1453420059', '1453420059', '1453420059', '1453423004', '1453424792', '1453424792', '1453424792', '1453424996', '1453425660', '1453425660', '1453425660', '1453427992', '1453427992', '1453432440', '1453432440', '1453432440', '1453441705', '1453441705', '1453441705', '1453447625', '1453455358', '1453455358', '1453455358', '1453457120', '1453460459', '1453461809', '1453463349', '1453465523', '1453465523', '1453465523', '1453466342', '1453466342', '1453466342', '1453466660', '1453466660', '1453473378', '1453475784', '1453479488', '1453482148', '1453482148', '1453482876', '1453482876', '1453482876', '1453486849', '1453486849', '1453486849', '1453486849', '1453496513', '1453496513', '1453496513', '1453501265', '1453502539', '1453505090', '1453507345', '1453507345', '1453507345', '1453513492', '1453513492', '1453513833', '1453513833', '1453517484', '1453517484', '1453519353', '1453521824', '1453521824', '1453522238', '1453522238', '1453523098', '1453524851', '1453534129', '1453534129', '1453534129', '1453539206', '1453539206', '1453547182', '1453547526', '1453553770', '1453553770', '1453553770', '1453558558', '1453558558', '1453559946', '1453559946', '1453560825', '1453560825', '1453561734', '1453561734', '1453563835', '1453563835', '1453563835', '1453573999', '1453576375', '1453576992', '1453585041', '1453585041', '1453585041', '1453587326', '1453590343', '1453594313', '1453594313', '1453597829', '1453601341', '1453605272', '1453605272', '1453605592', '1453610104', '1453610104', '1453610104', '1453615656', '1453615656', '1453621894', '1453621894', '1453621894', '1453629550', '1453629550', '1453631116', '1453631436', '1453631592', '1453631592', '1453631592', '1453652357', '1453657764', '1453661080', '1453661080', '1453671680', '1453671943', '1453672945', '1453672945', '1453676312', '1453676312', '1453677354', '1453679448', '1453679448', '1453679448', '1453680882', '1453680882', '1453683097', '1453683097', '1453684671', '1453687587', '1453687587', '1453688183', '1453689175', '1453689175', '1453689175', '1453690171', '1453698824', '1453700380', '1453700935', '1453700935', '1453700935', '1453701543', '1453701607', '1453701607', '1453701652', '1453701652', '1453701652', '1453708867', '1453709033', '1453709033', '1453709300', '1453727410', '1453738708', '1453745718', '1453745718', '1453745718', '1453752395', '1453752395', '1453758649', '1453758649', '1453758649', '1453770702', '1453770702', '1453771003', '1453771251', '1453771251', '1453771251', '1453774470', '1453779361', '1453782354', '1453782354', '1453782354', '1453787516', '1453789032', '1453789032', '1453789032', '1453789740', '1453789740', '1453794435', '1453798480', '1453798480', '1453798480', '1453801809', '1453801809', '1453801809', '1453815049', '1453816126', '1453816126', '1453816126', '1453820622', '1453822041', '1453822041', '1453822041', '1453842931', '1453844453', '1453845989', '1453845989', '1453848928', '1453848928', '1453848928', '1453860255', '1453860255', '1453860255', '1453860780', '1453865218', '1453865218', '1453868103', '1453870486', '1453874541', '1453876393', '1453876393', '1453876393', '1453876393', '1453880837', '1453883399', '1453890445', '1453890445', '1453890445', '1453894016', '1453894016', '1453894016', '1453895459', '1453895459', '1453895459', '1453899702', '1453899785', '1453899785', '1453900307', '1453900665', '1453900665', '1453902404', '1453908840', '1453908840', '1453908840', '1453912891', '1453912891', '1453912891', '1453912917', '1453912917', '1453917717', '1453918507', '1453918507', '1453918507', '1453920355', '1453920355', '1453920355', '1453922682', '1453925637', '1453931943', '1453931943', '1453932043', '1453932661', '1453935267', '1453944332', '1453944332', '1453944332', '1453945396', '1453945396', '1453945396', '1453945396', '1453949206', '1453951117', '1453953288', '1453955830', '1453955830', '1453964999', '1453965348', '1453968432', '1453968432', '1453977919', '1453996241', '1453996456', '1453996456', '1453999266', '1454002554', '1454007716', '1454007716', '1454009509', '1454009796', '1454014755', '1454014755', '1454014755', '1454016298', '1454016298', '1454016298', '1454018760', '1454024678', '1454024678', '1454029420', '1454029420', '1454029420', '1454029730', '1454029730', '1454036981', '1454040583', '1454046775', '1454053805', '1454053805', '1454053805', '1454059035', '1454059035', '1454059035', '1454063311', '1454063311', '1454063311', '1454063448', '1454063448', '1454063448', '1454070027', '1454082444', '1454082444', '1454083118', '1454083118', '1454083118', '1454084975', '1454090337', '1454097399', '1454097399', '1454113546', '1454113546', '1454116165', '1454116165', '1454118870', '1454118870', '1454128720', '1454130410', '1454131152', '1454131152', '1454131563', '1454133166', '1454133166', '1454140185', '1454140557', '1454140557', '1454140557', '1454140970', '1454142069', '1454142069', '1454142069', '1454153576', '1454153688', '1454153688', '1454159939', '1454159939', '1454159939', '1454160043', '1454160043', '1454163022', '1454164927', '1454164927', '1454178197', '1454179603', '1454188631', '1454189142', '1454189142', '1454189142', '1454193444', '1454193444', '1454194033', '1454194033', '1454194033', '1454194737', '1454194737', '1454194737', '1454198086', '1454198086', '1454198440'], crdr_0_2592000=586, crdr_1_2592000=566)]

    
   

In [9]:
# rdd处理函数
def rdd_fun_list(x_list):
    for x in x_list:
        # 金额
        amt_list = [
            ['amt_1800', x.amt_1800],  # 30min
            ['balance_amt_1800', x.balance_amt_1800],

            ['amt_3600', x.amt_3600],  # 1h
            ['balance_amt_3600', x.balance_amt_3600],

            ['amt_86400', x.amt_86400],  # 1天
            ['balance_amt_86400', x.balance_amt_86400],

            ['amt_604800', x.amt_604800],  # 7天
            ['balance_amt_604800', x.balance_amt_604800],

            ['amt_1209600', x.amt_1209600],  # 14天
            ['balance_amt_1209600', x.balance_amt_1209600],

            ['amt_2592000', x.amt_2592000],  # 30天
            ['balance_amt_2592000', x.balance_amt_2592000],

        ]
        # 时间
        time_list = [
            ['u_time_1800', x.u_time_1800],
            ['u_time_3600', x.u_time_3600],
            ['u_time_86400', x.u_time_86400],
            ['u_time_604800', x.u_time_604800],
            ['u_time_1209600', x.u_time_1209600],
            ['u_time_2592000', x.u_time_2592000],

        ]
        res_dict = {}
        a_dict = {}

        # 1. money
        for col in amt_list:
            col_name = col[0]  # 列名

            # 如果col[1]为空时执行
            if not col[1]:
                # a_dict[col_name] = []
                a_dict[col_name + '_sum'] = float(np.nan)
                a_dict[col_name + '_avg'] = float(np.nan)
                a_dict[col_name + '_std'] = float(np.nan)
                a_dict[col_name + '_max'] = float(np.nan)
                a_dict[col_name + '_min'] = float(np.nan)
                a_dict[col_name + '_q1'] = float(np.nan)
                a_dict[col_name + '_q3'] = float(np.nan)
                a_dict[col_name + '_softNoise'] = float(np.nan)
                a_dict[col_name + '_hardNoise'] = float(np.nan)

                continue

            # col[1]非空时执行
            sort_list = []
            for item in col[1]:
                each = item.split('*')
                trx_amt = each[0]
                u_trx_time = each[1]

                sort_list.append([trx_amt, u_trx_time])

                # 按照时间戳timestemp排序
            sort_list = sorted(sort_list, key=lambda x: int(x[1]))
            tmp_list = []  # 临时list
            for item in sort_list:
                tmp_list.append(float(item[0]))

            # a_dict[col_name] = tmp_list
            a_dict[col_name + '_sum'] = float(np.sum(tmp_list))
            a_dict[col_name + '_avg'] = float(np.mean(tmp_list))
            a_dict[col_name + '_std'] = float(np.std(tmp_list))
            a_dict[col_name + '_max'] = float(np.max(tmp_list))
            a_dict[col_name + '_min'] = float(np.min(tmp_list))
            a_dict[col_name + '_q1'] = float(np.sort(tmp_list)[int(0.25 * len(tmp_list))])
            a_dict[col_name + '_q3'] = float(np.sort(tmp_list)[int(0.75 * len(tmp_list))])
            a_dict[col_name + '_softNoise'] = float(2.5 * a_dict[col_name + '_q3'] - 1.5 * a_dict[col_name + '_q1'])
            a_dict[col_name + '_hardNoise'] = float(4 * a_dict[col_name + '_q3'] - 3 * a_dict[col_name + '_q1'])

        # 2. timestamp
        for col in time_list:
            col_name = col[0]  # 列名

            # 如果col[1]为空时执行
            if not col[1]:
                # a_dict[col_name] = []
                a_dict[col_name + '_sum'] = float(np.nan)
                a_dict[col_name + '_avg'] = float(np.nan)
                a_dict[col_name + '_std'] = float(np.nan)
                a_dict[col_name + '_max'] = float(np.nan)
                a_dict[col_name + '_min'] = float(np.nan)
                a_dict[col_name + '_q1'] = float(np.nan)
                a_dict[col_name + '_q3'] = float(np.nan)
                a_dict[col_name + '_softNoise'] = float(np.nan)
                a_dict[col_name + '_hardNoise'] = float(np.nan)

                continue

            # col[1]非空时执行
            sort_list = []
            for timestamp in col[1]:
                each = int(timestamp)
                sort_list.append(each)
            sort_list = sorted(sort_list)

            # a_dict[col_name] = sort_list
            a_dict[col_name + '_sum'] = float(np.sum(sort_list))
            a_dict[col_name + '_avg'] = float(np.mean(sort_list))
            a_dict[col_name + '_std'] = float(np.std(sort_list))
            a_dict[col_name + '_max'] = float(np.max(sort_list))
            a_dict[col_name + '_min'] = float(np.min(sort_list))
            a_dict[col_name + '_q1'] = float(np.sort(sort_list)[int(0.25 * len(sort_list))])
            a_dict[col_name + '_q3'] = float(np.sort(sort_list)[int(0.75 * len(sort_list))])
            a_dict[col_name + '_softNoise'] = float(2.5 * a_dict[col_name + '_q3'] - 1.5 * a_dict[col_name + '_q1'])
            a_dict[col_name + '_hardNoise'] = float(4 * a_dict[col_name + '_q3'] - 3 * a_dict[col_name + '_q1'])

        b_dict = {
            'trx_id': x.trx_id,
            'cust_id': x.cust_id,
            'target_cust_id': x.target_cust_id,
            'flag_crdr': x.flag_crdr,
            'trx_amt': x.trx_amt,
            'balance_amt': x.balance_amt,
            'trx_date': x.trx_date,
            'trx_time': x.trx_time,
            'u_trx_time': x.u_trx_time,
            'cnt_1800': x.cnt_1800,
            'cnt_3600': x.cnt_3600,
            'cnt_86400': x.cnt_86400,
            'cnt_604800': x.cnt_604800,
            'cnt_1209600': x.cnt_1209600,
            'cnt_2592000': x.cnt_2592000,
            'crdr_0_1800': x.crdr_0_1800,
            'crdr_1_1800': x.crdr_1_1800,
            'crdr_0_3600': x.crdr_0_3600,
            'crdr_1_3600': x.crdr_1_3600,
            'crdr_0_86400': x.crdr_0_86400,
            'crdr_1_86400': x.crdr_1_86400,
            'crdr_0_604800': x.crdr_0_604800,
            'crdr_1_604800': x.crdr_1_604800,
            'crdr_0_1209600': x.crdr_0_1209600,
            'crdr_1_1209600': x.crdr_1_1209600,
            'crdr_0_2592000': x.crdr_0_2592000,
            'crdr_1_2592000': x.crdr_1_2592000,
        }

        # 合并a_dict和b_dict
        res_dict = {**b_dict, **a_dict}

        keys = list(res_dict.keys())

        yield [res_dict[k] for k in keys]

        # print(len(res_dict)) #223
        # print(list(res_dict.keys())) #获取列名
        # print('*******')
        # print(res_dict)

    #return res_dict
#list(rdd_fun_list(ax).keys())


In [10]:
col_list = ['trx_id',
 'cust_id',
 'target_cust_id',
 'flag_crdr',
 'trx_amt',
 'balance_amt',
 'trx_date',
 'trx_time',
 'u_trx_time',
 'cnt_1800',
 'cnt_3600',
 'cnt_86400',
 'cnt_604800',
 'cnt_1209600',
 'cnt_2592000',
 'crdr_0_1800',
 'crdr_1_1800',
 'crdr_0_3600',
 'crdr_1_3600',
 'crdr_0_86400',
 'crdr_1_86400',
 'crdr_0_604800',
 'crdr_1_604800',
 'crdr_0_1209600',
 'crdr_1_1209600',
 'crdr_0_2592000',
 'crdr_1_2592000',
 'amt_1800_sum',
 'amt_1800_avg',
 'amt_1800_std',
 'amt_1800_max',
 'amt_1800_min',
 'amt_1800_q1',
 'amt_1800_q3',
 'amt_1800_softNoise',
 'amt_1800_hardNoise',
 'balance_amt_1800_sum',
 'balance_amt_1800_avg',
 'balance_amt_1800_std',
 'balance_amt_1800_max',
 'balance_amt_1800_min',
 'balance_amt_1800_q1',
 'balance_amt_1800_q3',
 'balance_amt_1800_softNoise',
 'balance_amt_1800_hardNoise',
 'amt_3600_sum',
 'amt_3600_avg',
 'amt_3600_std',
 'amt_3600_max',
 'amt_3600_min',
 'amt_3600_q1',
 'amt_3600_q3',
 'amt_3600_softNoise',
 'amt_3600_hardNoise',
 'balance_amt_3600_sum',
 'balance_amt_3600_avg',
 'balance_amt_3600_std',
 'balance_amt_3600_max',
 'balance_amt_3600_min',
 'balance_amt_3600_q1',
 'balance_amt_3600_q3',
 'balance_amt_3600_softNoise',
 'balance_amt_3600_hardNoise',
 'amt_86400_sum',
 'amt_86400_avg',
 'amt_86400_std',
 'amt_86400_max',
 'amt_86400_min',
 'amt_86400_q1',
 'amt_86400_q3',
 'amt_86400_softNoise',
 'amt_86400_hardNoise',
 'balance_amt_86400_sum',
 'balance_amt_86400_avg',
 'balance_amt_86400_std',
 'balance_amt_86400_max',
 'balance_amt_86400_min',
 'balance_amt_86400_q1',
 'balance_amt_86400_q3',
 'balance_amt_86400_softNoise',
 'balance_amt_86400_hardNoise',
 'amt_604800_sum',
 'amt_604800_avg',
 'amt_604800_std',
 'amt_604800_max',
 'amt_604800_min',
 'amt_604800_q1',
 'amt_604800_q3',
 'amt_604800_softNoise',
 'amt_604800_hardNoise',
 'balance_amt_604800_sum',
 'balance_amt_604800_avg',
 'balance_amt_604800_std',
 'balance_amt_604800_max',
 'balance_amt_604800_min',
 'balance_amt_604800_q1',
 'balance_amt_604800_q3',
 'balance_amt_604800_softNoise',
 'balance_amt_604800_hardNoise',
 'amt_1209600_sum',
 'amt_1209600_avg',
 'amt_1209600_std',
 'amt_1209600_max',
 'amt_1209600_min',
 'amt_1209600_q1',
 'amt_1209600_q3',
 'amt_1209600_softNoise',
 'amt_1209600_hardNoise',
 'balance_amt_1209600_sum',
 'balance_amt_1209600_avg',
 'balance_amt_1209600_std',
 'balance_amt_1209600_max',
 'balance_amt_1209600_min',
 'balance_amt_1209600_q1',
 'balance_amt_1209600_q3',
 'balance_amt_1209600_softNoise',
 'balance_amt_1209600_hardNoise',
 'amt_2592000_sum',
 'amt_2592000_avg',
 'amt_2592000_std',
 'amt_2592000_max',
 'amt_2592000_min',
 'amt_2592000_q1',
 'amt_2592000_q3',
 'amt_2592000_softNoise',
 'amt_2592000_hardNoise',
 'balance_amt_2592000_sum',
 'balance_amt_2592000_avg',
 'balance_amt_2592000_std',
 'balance_amt_2592000_max',
 'balance_amt_2592000_min',
 'balance_amt_2592000_q1',
 'balance_amt_2592000_q3',
 'balance_amt_2592000_softNoise',
 'balance_amt_2592000_hardNoise',
 'u_time_1800_sum',
 'u_time_1800_avg',
 'u_time_1800_std',
 'u_time_1800_max',
 'u_time_1800_min',
 'u_time_1800_q1',
 'u_time_1800_q3',
 'u_time_1800_softNoise',
 'u_time_1800_hardNoise',
 'u_time_3600_sum',
 'u_time_3600_avg',
 'u_time_3600_std',
 'u_time_3600_max',
 'u_time_3600_min',
 'u_time_3600_q1',
 'u_time_3600_q3',
 'u_time_3600_softNoise',
 'u_time_3600_hardNoise',
 'u_time_86400_sum',
 'u_time_86400_avg',
 'u_time_86400_std',
 'u_time_86400_max',
 'u_time_86400_min',
 'u_time_86400_q1',
 'u_time_86400_q3',
 'u_time_86400_softNoise',
 'u_time_86400_hardNoise',
 'u_time_604800_sum',
 'u_time_604800_avg',
 'u_time_604800_std',
 'u_time_604800_max',
 'u_time_604800_min',
 'u_time_604800_q1',
 'u_time_604800_q3',
 'u_time_604800_softNoise',
 'u_time_604800_hardNoise',
 'u_time_1209600_sum',
 'u_time_1209600_avg',
 'u_time_1209600_std',
 'u_time_1209600_max',
 'u_time_1209600_min',
 'u_time_1209600_q1',
 'u_time_1209600_q3',
 'u_time_1209600_softNoise',
 'u_time_1209600_hardNoise',
 'u_time_2592000_sum',
 'u_time_2592000_avg',
 'u_time_2592000_std',
 'u_time_2592000_max',
 'u_time_2592000_min',
 'u_time_2592000_q1',
 'u_time_2592000_q3',
 'u_time_2592000_softNoise',
 'u_time_2592000_hardNoise']

In [11]:
len(col_list)

189

In [12]:
rdd_full = ori_trx_new_full.where("cust_id='cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272'").limit(2).rdd.mapPartitions(lambda x: rdd_fun_list(x))

In [13]:
user_act_full = spark.createDataFrame(rdd_full, col_list)

In [14]:
user_act_full.collect()

[Row(trx_id='trx_7a3b1bb7-a14a-4f5e-8738-ea7f8941b79a', cust_id='cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272', target_cust_id='cust_persion__a2fd0a79-6d0d-4cfc-b9fa-df0879eca080', flag_crdr='0', trx_amt='12471505.3357', balance_amt='21549400.9814', trx_date='2016-01-01', trx_time='00:24:20', u_trx_time=1451579060, cnt_1800=1, cnt_3600=1, cnt_86400=24, cnt_604800=249, cnt_1209600=525, cnt_2592000=1152, crdr_0_1800=1, crdr_1_1800=0, crdr_0_3600=1, crdr_1_3600=0, crdr_0_86400=12, crdr_1_86400=12, crdr_0_604800=149, crdr_1_604800=100, crdr_0_1209600=289, crdr_1_1209600=236, crdr_0_2592000=586, crdr_1_2592000=566, amt_1800_sum=12471505.3357, amt_1800_avg=12471505.3357, amt_1800_std=0.0, amt_1800_max=12471505.3357, amt_1800_min=12471505.3357, amt_1800_q1=12471505.3357, amt_1800_q3=12471505.3357, amt_1800_softNoise=12471505.335699998, amt_1800_hardNoise=12471505.335699998, balance_amt_1800_sum=21549400.9814, balance_amt_1800_avg=21549400.9814, balance_amt_1800_std=0.0, balance_amt_1800

In [16]:
user_act_full.registerTempTable("user_act_full")

In [17]:
user_act_full2=spark.sql(

"""
select
    cust_id,
    avg(cnt_1800) as cnt_1800_avg,
    max(cnt_1800) as cnt_1800_max,
    min(cnt_1800) as cnt_1800_min,
    avg(cnt_3600) as cnt_3600_avg,
    max(cnt_3600) as cnt_3600_max,
    min(cnt_3600) as cnt_3600_min,
    avg(cnt_86400) as cnt_86400_avg,
    max(cnt_86400) as cnt_86400_max,
    min(cnt_86400) as cnt_86400_min,
    avg(cnt_604800) as cnt_604800_avg,
    max(cnt_604800) as cnt_604800_max,
    min(cnt_604800) as cnt_604800_min,
    avg(cnt_1209600) as cnt_1209600_avg,
    max(cnt_1209600) as cnt_1209600_max,
    min(cnt_1209600) as cnt_1209600_min,
    avg(cnt_2592000) as cnt_2592000_avg,
    max(cnt_2592000) as cnt_2592000_max,
    min(cnt_2592000) as cnt_2592000_min,
    avg(crdr_0_1800) as crdr_0_1800_avg,
    max(crdr_0_1800) as crdr_0_1800_max,
    min(crdr_0_1800) as crdr_0_1800_min,
    avg(crdr_1_1800) as crdr_1_1800_avg,
    max(crdr_1_1800) as crdr_1_1800_max,
    min(crdr_1_1800) as crdr_1_1800_min,
    avg(crdr_0_3600) as crdr_0_3600_avg,
    max(crdr_0_3600) as crdr_0_3600_max,
    min(crdr_0_3600) as crdr_0_3600_min,
    avg(crdr_1_3600) as crdr_1_3600_avg,
    max(crdr_1_3600) as crdr_1_3600_max,
    min(crdr_1_3600) as crdr_1_3600_min,
    avg(crdr_0_86400) as crdr_0_86400_avg,
    max(crdr_0_86400) as crdr_0_86400_max,
    min(crdr_0_86400) as crdr_0_86400_min,
    avg(crdr_1_86400) as crdr_1_86400_avg,
    max(crdr_1_86400) as crdr_1_86400_max,
    min(crdr_1_86400) as crdr_1_86400_min,
    avg(crdr_0_604800) as crdr_0_604800_avg,
    max(crdr_0_604800) as crdr_0_604800_max,
    min(crdr_0_604800) as crdr_0_604800_min,
    avg(crdr_1_604800) as crdr_1_604800_avg,
    max(crdr_1_604800) as crdr_1_604800_max,
    min(crdr_1_604800) as crdr_1_604800_min,
    avg(crdr_0_1209600) as crdr_0_1209600_avg,
    max(crdr_0_1209600) as crdr_0_1209600_max,
    min(crdr_0_1209600) as crdr_0_1209600_min,
    avg(crdr_1_1209600) as crdr_1_1209600_avg,
    max(crdr_1_1209600) as crdr_1_1209600_max,
    min(crdr_1_1209600) as crdr_1_1209600_min,
    avg(crdr_0_2592000) as crdr_0_2592000_avg,
    max(crdr_0_2592000) as crdr_0_2592000_max,
    min(crdr_0_2592000) as crdr_0_2592000_min,
    avg(crdr_1_2592000) as crdr_1_2592000_avg,
    max(crdr_1_2592000) as crdr_1_2592000_max,
    min(crdr_1_2592000) as crdr_1_2592000_min,
    avg(amt_1800_sum) as amt_1800_sum_avg,
    max(amt_1800_sum) as amt_1800_sum_max,
    min(amt_1800_sum) as amt_1800_sum_min,
    avg(amt_1800_avg) as amt_1800_avg_avg,
    max(amt_1800_avg) as amt_1800_avg_max,
    min(amt_1800_avg) as amt_1800_avg_min,
    avg(amt_1800_std) as amt_1800_std_avg,
    max(amt_1800_std) as amt_1800_std_max,
    min(amt_1800_std) as amt_1800_std_min,
    avg(amt_1800_max) as amt_1800_max_avg,
    max(amt_1800_max) as amt_1800_max_max,
    min(amt_1800_max) as amt_1800_max_min,
    avg(amt_1800_min) as amt_1800_min_avg,
    max(amt_1800_min) as amt_1800_min_max,
    min(amt_1800_min) as amt_1800_min_min,
    avg(amt_1800_q1) as amt_1800_q1_avg,
    max(amt_1800_q1) as amt_1800_q1_max,
    min(amt_1800_q1) as amt_1800_q1_min,
    avg(amt_1800_q3) as amt_1800_q3_avg,
    max(amt_1800_q3) as amt_1800_q3_max,
    min(amt_1800_q3) as amt_1800_q3_min,
    avg(amt_1800_softNoise) as amt_1800_softNoise_avg,
    max(amt_1800_softNoise) as amt_1800_softNoise_max,
    min(amt_1800_softNoise) as amt_1800_softNoise_min,
    avg(amt_1800_hardNoise) as amt_1800_hardNoise_avg,
    max(amt_1800_hardNoise) as amt_1800_hardNoise_max,
    min(amt_1800_hardNoise) as amt_1800_hardNoise_min,
    avg(balance_amt_1800_sum) as balance_amt_1800_sum_avg,
    max(balance_amt_1800_sum) as balance_amt_1800_sum_max,
    min(balance_amt_1800_sum) as balance_amt_1800_sum_min,
    avg(balance_amt_1800_avg) as balance_amt_1800_avg_avg,
    max(balance_amt_1800_avg) as balance_amt_1800_avg_max,
    min(balance_amt_1800_avg) as balance_amt_1800_avg_min,
    avg(balance_amt_1800_std) as balance_amt_1800_std_avg,
    max(balance_amt_1800_std) as balance_amt_1800_std_max,
    min(balance_amt_1800_std) as balance_amt_1800_std_min,
    avg(balance_amt_1800_max) as balance_amt_1800_max_avg,
    max(balance_amt_1800_max) as balance_amt_1800_max_max,
    min(balance_amt_1800_max) as balance_amt_1800_max_min,
    avg(balance_amt_1800_min) as balance_amt_1800_min_avg,
    max(balance_amt_1800_min) as balance_amt_1800_min_max,
    min(balance_amt_1800_min) as balance_amt_1800_min_min,
    avg(balance_amt_1800_q1) as balance_amt_1800_q1_avg,
    max(balance_amt_1800_q1) as balance_amt_1800_q1_max,
    min(balance_amt_1800_q1) as balance_amt_1800_q1_min,
    avg(balance_amt_1800_q3) as balance_amt_1800_q3_avg,
    max(balance_amt_1800_q3) as balance_amt_1800_q3_max,
    min(balance_amt_1800_q3) as balance_amt_1800_q3_min,
    avg(balance_amt_1800_softNoise) as balance_amt_1800_softNoise_avg,
    max(balance_amt_1800_softNoise) as balance_amt_1800_softNoise_max,
    min(balance_amt_1800_softNoise) as balance_amt_1800_softNoise_min,
    avg(balance_amt_1800_hardNoise) as balance_amt_1800_hardNoise_avg,
    max(balance_amt_1800_hardNoise) as balance_amt_1800_hardNoise_max,
    min(balance_amt_1800_hardNoise) as balance_amt_1800_hardNoise_min,
    avg(amt_3600_sum) as amt_3600_sum_avg,
    max(amt_3600_sum) as amt_3600_sum_max,
    min(amt_3600_sum) as amt_3600_sum_min,
    avg(amt_3600_avg) as amt_3600_avg_avg,
    max(amt_3600_avg) as amt_3600_avg_max,
    min(amt_3600_avg) as amt_3600_avg_min,
    avg(amt_3600_std) as amt_3600_std_avg,
    max(amt_3600_std) as amt_3600_std_max,
    min(amt_3600_std) as amt_3600_std_min,
    avg(amt_3600_max) as amt_3600_max_avg,
    max(amt_3600_max) as amt_3600_max_max,
    min(amt_3600_max) as amt_3600_max_min,
    avg(amt_3600_min) as amt_3600_min_avg,
    max(amt_3600_min) as amt_3600_min_max,
    min(amt_3600_min) as amt_3600_min_min,
    avg(amt_3600_q1) as amt_3600_q1_avg,
    max(amt_3600_q1) as amt_3600_q1_max,
    min(amt_3600_q1) as amt_3600_q1_min,
    avg(amt_3600_q3) as amt_3600_q3_avg,
    max(amt_3600_q3) as amt_3600_q3_max,
    min(amt_3600_q3) as amt_3600_q3_min,
    avg(amt_3600_softNoise) as amt_3600_softNoise_avg,
    max(amt_3600_softNoise) as amt_3600_softNoise_max,
    min(amt_3600_softNoise) as amt_3600_softNoise_min,
    avg(amt_3600_hardNoise) as amt_3600_hardNoise_avg,
    max(amt_3600_hardNoise) as amt_3600_hardNoise_max,
    min(amt_3600_hardNoise) as amt_3600_hardNoise_min,
    avg(balance_amt_3600_sum) as balance_amt_3600_sum_avg,
    max(balance_amt_3600_sum) as balance_amt_3600_sum_max,
    min(balance_amt_3600_sum) as balance_amt_3600_sum_min,
    avg(balance_amt_3600_avg) as balance_amt_3600_avg_avg,
    max(balance_amt_3600_avg) as balance_amt_3600_avg_max,
    min(balance_amt_3600_avg) as balance_amt_3600_avg_min,
    avg(balance_amt_3600_std) as balance_amt_3600_std_avg,
    max(balance_amt_3600_std) as balance_amt_3600_std_max,
    min(balance_amt_3600_std) as balance_amt_3600_std_min,
    avg(balance_amt_3600_max) as balance_amt_3600_max_avg,
    max(balance_amt_3600_max) as balance_amt_3600_max_max,
    min(balance_amt_3600_max) as balance_amt_3600_max_min,
    avg(balance_amt_3600_min) as balance_amt_3600_min_avg,
    max(balance_amt_3600_min) as balance_amt_3600_min_max,
    min(balance_amt_3600_min) as balance_amt_3600_min_min,
    avg(balance_amt_3600_q1) as balance_amt_3600_q1_avg,
    max(balance_amt_3600_q1) as balance_amt_3600_q1_max,
    min(balance_amt_3600_q1) as balance_amt_3600_q1_min,
    avg(balance_amt_3600_q3) as balance_amt_3600_q3_avg,
    max(balance_amt_3600_q3) as balance_amt_3600_q3_max,
    min(balance_amt_3600_q3) as balance_amt_3600_q3_min,
    avg(balance_amt_3600_softNoise) as balance_amt_3600_softNoise_avg,
    max(balance_amt_3600_softNoise) as balance_amt_3600_softNoise_max,
    min(balance_amt_3600_softNoise) as balance_amt_3600_softNoise_min,
    avg(balance_amt_3600_hardNoise) as balance_amt_3600_hardNoise_avg,
    max(balance_amt_3600_hardNoise) as balance_amt_3600_hardNoise_max,
    min(balance_amt_3600_hardNoise) as balance_amt_3600_hardNoise_min,
    avg(amt_86400_sum) as amt_86400_sum_avg,
    max(amt_86400_sum) as amt_86400_sum_max,
    min(amt_86400_sum) as amt_86400_sum_min,
    avg(amt_86400_avg) as amt_86400_avg_avg,
    max(amt_86400_avg) as amt_86400_avg_max,
    min(amt_86400_avg) as amt_86400_avg_min,
    avg(amt_86400_std) as amt_86400_std_avg,
    max(amt_86400_std) as amt_86400_std_max,
    min(amt_86400_std) as amt_86400_std_min,
    avg(amt_86400_max) as amt_86400_max_avg,
    max(amt_86400_max) as amt_86400_max_max,
    min(amt_86400_max) as amt_86400_max_min,
    avg(amt_86400_min) as amt_86400_min_avg,
    max(amt_86400_min) as amt_86400_min_max,
    min(amt_86400_min) as amt_86400_min_min,
    avg(amt_86400_q1) as amt_86400_q1_avg,
    max(amt_86400_q1) as amt_86400_q1_max,
    min(amt_86400_q1) as amt_86400_q1_min,
    avg(amt_86400_q3) as amt_86400_q3_avg,
    max(amt_86400_q3) as amt_86400_q3_max,
    min(amt_86400_q3) as amt_86400_q3_min,
    avg(amt_86400_softNoise) as amt_86400_softNoise_avg,
    max(amt_86400_softNoise) as amt_86400_softNoise_max,
    min(amt_86400_softNoise) as amt_86400_softNoise_min,
    avg(amt_86400_hardNoise) as amt_86400_hardNoise_avg,
    max(amt_86400_hardNoise) as amt_86400_hardNoise_max,
    min(amt_86400_hardNoise) as amt_86400_hardNoise_min,
    avg(balance_amt_86400_sum) as balance_amt_86400_sum_avg,
    max(balance_amt_86400_sum) as balance_amt_86400_sum_max,
    min(balance_amt_86400_sum) as balance_amt_86400_sum_min,
    avg(balance_amt_86400_avg) as balance_amt_86400_avg_avg,
    max(balance_amt_86400_avg) as balance_amt_86400_avg_max,
    min(balance_amt_86400_avg) as balance_amt_86400_avg_min,
    avg(balance_amt_86400_std) as balance_amt_86400_std_avg,
    max(balance_amt_86400_std) as balance_amt_86400_std_max,
    min(balance_amt_86400_std) as balance_amt_86400_std_min,
    avg(balance_amt_86400_max) as balance_amt_86400_max_avg,
    max(balance_amt_86400_max) as balance_amt_86400_max_max,
    min(balance_amt_86400_max) as balance_amt_86400_max_min,
    avg(balance_amt_86400_min) as balance_amt_86400_min_avg,
    max(balance_amt_86400_min) as balance_amt_86400_min_max,
    min(balance_amt_86400_min) as balance_amt_86400_min_min,
    avg(balance_amt_86400_q1) as balance_amt_86400_q1_avg,
    max(balance_amt_86400_q1) as balance_amt_86400_q1_max,
    min(balance_amt_86400_q1) as balance_amt_86400_q1_min,
    avg(balance_amt_86400_q3) as balance_amt_86400_q3_avg,
    max(balance_amt_86400_q3) as balance_amt_86400_q3_max,
    min(balance_amt_86400_q3) as balance_amt_86400_q3_min,
    avg(balance_amt_86400_softNoise) as balance_amt_86400_softNoise_avg,
    max(balance_amt_86400_softNoise) as balance_amt_86400_softNoise_max,
    min(balance_amt_86400_softNoise) as balance_amt_86400_softNoise_min,
    avg(balance_amt_86400_hardNoise) as balance_amt_86400_hardNoise_avg,
    max(balance_amt_86400_hardNoise) as balance_amt_86400_hardNoise_max,
    min(balance_amt_86400_hardNoise) as balance_amt_86400_hardNoise_min,
    avg(amt_604800_sum) as amt_604800_sum_avg,
    max(amt_604800_sum) as amt_604800_sum_max,
    min(amt_604800_sum) as amt_604800_sum_min,
    avg(amt_604800_avg) as amt_604800_avg_avg,
    max(amt_604800_avg) as amt_604800_avg_max,
    min(amt_604800_avg) as amt_604800_avg_min,
    avg(amt_604800_std) as amt_604800_std_avg,
    max(amt_604800_std) as amt_604800_std_max,
    min(amt_604800_std) as amt_604800_std_min,
    avg(amt_604800_max) as amt_604800_max_avg,
    max(amt_604800_max) as amt_604800_max_max,
    min(amt_604800_max) as amt_604800_max_min,
    avg(amt_604800_min) as amt_604800_min_avg,
    max(amt_604800_min) as amt_604800_min_max,
    min(amt_604800_min) as amt_604800_min_min,
    avg(amt_604800_q1) as amt_604800_q1_avg,
    max(amt_604800_q1) as amt_604800_q1_max,
    min(amt_604800_q1) as amt_604800_q1_min,
    avg(amt_604800_q3) as amt_604800_q3_avg,
    max(amt_604800_q3) as amt_604800_q3_max,
    min(amt_604800_q3) as amt_604800_q3_min,
    avg(amt_604800_softNoise) as amt_604800_softNoise_avg,
    max(amt_604800_softNoise) as amt_604800_softNoise_max,
    min(amt_604800_softNoise) as amt_604800_softNoise_min,
    avg(amt_604800_hardNoise) as amt_604800_hardNoise_avg,
    max(amt_604800_hardNoise) as amt_604800_hardNoise_max,
    min(amt_604800_hardNoise) as amt_604800_hardNoise_min,
    avg(balance_amt_604800_sum) as balance_amt_604800_sum_avg,
    max(balance_amt_604800_sum) as balance_amt_604800_sum_max,
    min(balance_amt_604800_sum) as balance_amt_604800_sum_min,
    avg(balance_amt_604800_avg) as balance_amt_604800_avg_avg,
    max(balance_amt_604800_avg) as balance_amt_604800_avg_max,
    min(balance_amt_604800_avg) as balance_amt_604800_avg_min,
    avg(balance_amt_604800_std) as balance_amt_604800_std_avg,
    max(balance_amt_604800_std) as balance_amt_604800_std_max,
    min(balance_amt_604800_std) as balance_amt_604800_std_min,
    avg(balance_amt_604800_max) as balance_amt_604800_max_avg,
    max(balance_amt_604800_max) as balance_amt_604800_max_max,
    min(balance_amt_604800_max) as balance_amt_604800_max_min,
    avg(balance_amt_604800_min) as balance_amt_604800_min_avg,
    max(balance_amt_604800_min) as balance_amt_604800_min_max,
    min(balance_amt_604800_min) as balance_amt_604800_min_min,
    avg(balance_amt_604800_q1) as balance_amt_604800_q1_avg,
    max(balance_amt_604800_q1) as balance_amt_604800_q1_max,
    min(balance_amt_604800_q1) as balance_amt_604800_q1_min,
    avg(balance_amt_604800_q3) as balance_amt_604800_q3_avg,
    max(balance_amt_604800_q3) as balance_amt_604800_q3_max,
    min(balance_amt_604800_q3) as balance_amt_604800_q3_min,
    avg(balance_amt_604800_softNoise) as balance_amt_604800_softNoise_avg,
    max(balance_amt_604800_softNoise) as balance_amt_604800_softNoise_max,
    min(balance_amt_604800_softNoise) as balance_amt_604800_softNoise_min,
    avg(balance_amt_604800_hardNoise) as balance_amt_604800_hardNoise_avg,
    max(balance_amt_604800_hardNoise) as balance_amt_604800_hardNoise_max,
    min(balance_amt_604800_hardNoise) as balance_amt_604800_hardNoise_min,
    avg(amt_1209600_sum) as amt_1209600_sum_avg,
    max(amt_1209600_sum) as amt_1209600_sum_max,
    min(amt_1209600_sum) as amt_1209600_sum_min,
    avg(amt_1209600_avg) as amt_1209600_avg_avg,
    max(amt_1209600_avg) as amt_1209600_avg_max,
    min(amt_1209600_avg) as amt_1209600_avg_min,
    avg(amt_1209600_std) as amt_1209600_std_avg,
    max(amt_1209600_std) as amt_1209600_std_max,
    min(amt_1209600_std) as amt_1209600_std_min,
    avg(amt_1209600_max) as amt_1209600_max_avg,
    max(amt_1209600_max) as amt_1209600_max_max,
    min(amt_1209600_max) as amt_1209600_max_min,
    avg(amt_1209600_min) as amt_1209600_min_avg,
    max(amt_1209600_min) as amt_1209600_min_max,
    min(amt_1209600_min) as amt_1209600_min_min,
    avg(amt_1209600_q1) as amt_1209600_q1_avg,
    max(amt_1209600_q1) as amt_1209600_q1_max,
    min(amt_1209600_q1) as amt_1209600_q1_min,
    avg(amt_1209600_q3) as amt_1209600_q3_avg,
    max(amt_1209600_q3) as amt_1209600_q3_max,
    min(amt_1209600_q3) as amt_1209600_q3_min,
    avg(amt_1209600_softNoise) as amt_1209600_softNoise_avg,
    max(amt_1209600_softNoise) as amt_1209600_softNoise_max,
    min(amt_1209600_softNoise) as amt_1209600_softNoise_min,
    avg(amt_1209600_hardNoise) as amt_1209600_hardNoise_avg,
    max(amt_1209600_hardNoise) as amt_1209600_hardNoise_max,
    min(amt_1209600_hardNoise) as amt_1209600_hardNoise_min,
    avg(balance_amt_1209600_sum) as balance_amt_1209600_sum_avg,
    max(balance_amt_1209600_sum) as balance_amt_1209600_sum_max,
    min(balance_amt_1209600_sum) as balance_amt_1209600_sum_min,
    avg(balance_amt_1209600_avg) as balance_amt_1209600_avg_avg,
    max(balance_amt_1209600_avg) as balance_amt_1209600_avg_max,
    min(balance_amt_1209600_avg) as balance_amt_1209600_avg_min,
    avg(balance_amt_1209600_std) as balance_amt_1209600_std_avg,
    max(balance_amt_1209600_std) as balance_amt_1209600_std_max,
    min(balance_amt_1209600_std) as balance_amt_1209600_std_min,
    avg(balance_amt_1209600_max) as balance_amt_1209600_max_avg,
    max(balance_amt_1209600_max) as balance_amt_1209600_max_max,
    min(balance_amt_1209600_max) as balance_amt_1209600_max_min,
    avg(balance_amt_1209600_min) as balance_amt_1209600_min_avg,
    max(balance_amt_1209600_min) as balance_amt_1209600_min_max,
    min(balance_amt_1209600_min) as balance_amt_1209600_min_min,
    avg(balance_amt_1209600_q1) as balance_amt_1209600_q1_avg,
    max(balance_amt_1209600_q1) as balance_amt_1209600_q1_max,
    min(balance_amt_1209600_q1) as balance_amt_1209600_q1_min,
    avg(balance_amt_1209600_q3) as balance_amt_1209600_q3_avg,
    max(balance_amt_1209600_q3) as balance_amt_1209600_q3_max,
    min(balance_amt_1209600_q3) as balance_amt_1209600_q3_min,
    avg(balance_amt_1209600_softNoise) as balance_amt_1209600_softNoise_avg,
    max(balance_amt_1209600_softNoise) as balance_amt_1209600_softNoise_max,
    min(balance_amt_1209600_softNoise) as balance_amt_1209600_softNoise_min,
    avg(balance_amt_1209600_hardNoise) as balance_amt_1209600_hardNoise_avg,
    max(balance_amt_1209600_hardNoise) as balance_amt_1209600_hardNoise_max,
    min(balance_amt_1209600_hardNoise) as balance_amt_1209600_hardNoise_min,
    avg(amt_2592000_sum) as amt_2592000_sum_avg,
    max(amt_2592000_sum) as amt_2592000_sum_max,
    min(amt_2592000_sum) as amt_2592000_sum_min,
    avg(amt_2592000_avg) as amt_2592000_avg_avg,
    max(amt_2592000_avg) as amt_2592000_avg_max,
    min(amt_2592000_avg) as amt_2592000_avg_min,
    avg(amt_2592000_std) as amt_2592000_std_avg,
    max(amt_2592000_std) as amt_2592000_std_max,
    min(amt_2592000_std) as amt_2592000_std_min,
    avg(amt_2592000_max) as amt_2592000_max_avg,
    max(amt_2592000_max) as amt_2592000_max_max,
    min(amt_2592000_max) as amt_2592000_max_min,
    avg(amt_2592000_min) as amt_2592000_min_avg,
    max(amt_2592000_min) as amt_2592000_min_max,
    min(amt_2592000_min) as amt_2592000_min_min,
    avg(amt_2592000_q1) as amt_2592000_q1_avg,
    max(amt_2592000_q1) as amt_2592000_q1_max,
    min(amt_2592000_q1) as amt_2592000_q1_min,
    avg(amt_2592000_q3) as amt_2592000_q3_avg,
    max(amt_2592000_q3) as amt_2592000_q3_max,
    min(amt_2592000_q3) as amt_2592000_q3_min,
    avg(amt_2592000_softNoise) as amt_2592000_softNoise_avg,
    max(amt_2592000_softNoise) as amt_2592000_softNoise_max,
    min(amt_2592000_softNoise) as amt_2592000_softNoise_min,
    avg(amt_2592000_hardNoise) as amt_2592000_hardNoise_avg,
    max(amt_2592000_hardNoise) as amt_2592000_hardNoise_max,
    min(amt_2592000_hardNoise) as amt_2592000_hardNoise_min,
    avg(balance_amt_2592000_sum) as balance_amt_2592000_sum_avg,
    max(balance_amt_2592000_sum) as balance_amt_2592000_sum_max,
    min(balance_amt_2592000_sum) as balance_amt_2592000_sum_min,
    avg(balance_amt_2592000_avg) as balance_amt_2592000_avg_avg,
    max(balance_amt_2592000_avg) as balance_amt_2592000_avg_max,
    min(balance_amt_2592000_avg) as balance_amt_2592000_avg_min,
    avg(balance_amt_2592000_std) as balance_amt_2592000_std_avg,
    max(balance_amt_2592000_std) as balance_amt_2592000_std_max,
    min(balance_amt_2592000_std) as balance_amt_2592000_std_min,
    avg(balance_amt_2592000_max) as balance_amt_2592000_max_avg,
    max(balance_amt_2592000_max) as balance_amt_2592000_max_max,
    min(balance_amt_2592000_max) as balance_amt_2592000_max_min,
    avg(balance_amt_2592000_min) as balance_amt_2592000_min_avg,
    max(balance_amt_2592000_min) as balance_amt_2592000_min_max,
    min(balance_amt_2592000_min) as balance_amt_2592000_min_min,
    avg(balance_amt_2592000_q1) as balance_amt_2592000_q1_avg,
    max(balance_amt_2592000_q1) as balance_amt_2592000_q1_max,
    min(balance_amt_2592000_q1) as balance_amt_2592000_q1_min,
    avg(balance_amt_2592000_q3) as balance_amt_2592000_q3_avg,
    max(balance_amt_2592000_q3) as balance_amt_2592000_q3_max,
    min(balance_amt_2592000_q3) as balance_amt_2592000_q3_min,
    avg(balance_amt_2592000_softNoise) as balance_amt_2592000_softNoise_avg,
    max(balance_amt_2592000_softNoise) as balance_amt_2592000_softNoise_max,
    min(balance_amt_2592000_softNoise) as balance_amt_2592000_softNoise_min,
    avg(balance_amt_2592000_hardNoise) as balance_amt_2592000_hardNoise_avg,
    max(balance_amt_2592000_hardNoise) as balance_amt_2592000_hardNoise_max,
    min(balance_amt_2592000_hardNoise) as balance_amt_2592000_hardNoise_min,
    avg(u_time_1800_sum) as u_time_1800_sum_avg,
    max(u_time_1800_sum) as u_time_1800_sum_max,
    min(u_time_1800_sum) as u_time_1800_sum_min,
    avg(u_time_1800_avg) as u_time_1800_avg_avg,
    max(u_time_1800_avg) as u_time_1800_avg_max,
    min(u_time_1800_avg) as u_time_1800_avg_min,
    avg(u_time_1800_std) as u_time_1800_std_avg,
    max(u_time_1800_std) as u_time_1800_std_max,
    min(u_time_1800_std) as u_time_1800_std_min,
    avg(u_time_1800_max) as u_time_1800_max_avg,
    max(u_time_1800_max) as u_time_1800_max_max,
    min(u_time_1800_max) as u_time_1800_max_min,
    avg(u_time_1800_min) as u_time_1800_min_avg,
    max(u_time_1800_min) as u_time_1800_min_max,
    min(u_time_1800_min) as u_time_1800_min_min,
    avg(u_time_1800_q1) as u_time_1800_q1_avg,
    max(u_time_1800_q1) as u_time_1800_q1_max,
    min(u_time_1800_q1) as u_time_1800_q1_min,
    avg(u_time_1800_q3) as u_time_1800_q3_avg,
    max(u_time_1800_q3) as u_time_1800_q3_max,
    min(u_time_1800_q3) as u_time_1800_q3_min,
    avg(u_time_1800_softNoise) as u_time_1800_softNoise_avg,
    max(u_time_1800_softNoise) as u_time_1800_softNoise_max,
    min(u_time_1800_softNoise) as u_time_1800_softNoise_min,
    avg(u_time_1800_hardNoise) as u_time_1800_hardNoise_avg,
    max(u_time_1800_hardNoise) as u_time_1800_hardNoise_max,
    min(u_time_1800_hardNoise) as u_time_1800_hardNoise_min,
    avg(u_time_3600_sum) as u_time_3600_sum_avg,
    max(u_time_3600_sum) as u_time_3600_sum_max,
    min(u_time_3600_sum) as u_time_3600_sum_min,
    avg(u_time_3600_avg) as u_time_3600_avg_avg,
    max(u_time_3600_avg) as u_time_3600_avg_max,
    min(u_time_3600_avg) as u_time_3600_avg_min,
    avg(u_time_3600_std) as u_time_3600_std_avg,
    max(u_time_3600_std) as u_time_3600_std_max,
    min(u_time_3600_std) as u_time_3600_std_min,
    avg(u_time_3600_max) as u_time_3600_max_avg,
    max(u_time_3600_max) as u_time_3600_max_max,
    min(u_time_3600_max) as u_time_3600_max_min,
    avg(u_time_3600_min) as u_time_3600_min_avg,
    max(u_time_3600_min) as u_time_3600_min_max,
    min(u_time_3600_min) as u_time_3600_min_min,
    avg(u_time_3600_q1) as u_time_3600_q1_avg,
    max(u_time_3600_q1) as u_time_3600_q1_max,
    min(u_time_3600_q1) as u_time_3600_q1_min,
    avg(u_time_3600_q3) as u_time_3600_q3_avg,
    max(u_time_3600_q3) as u_time_3600_q3_max,
    min(u_time_3600_q3) as u_time_3600_q3_min,
    avg(u_time_3600_softNoise) as u_time_3600_softNoise_avg,
    max(u_time_3600_softNoise) as u_time_3600_softNoise_max,
    min(u_time_3600_softNoise) as u_time_3600_softNoise_min,
    avg(u_time_3600_hardNoise) as u_time_3600_hardNoise_avg,
    max(u_time_3600_hardNoise) as u_time_3600_hardNoise_max,
    min(u_time_3600_hardNoise) as u_time_3600_hardNoise_min,
    avg(u_time_86400_sum) as u_time_86400_sum_avg,
    max(u_time_86400_sum) as u_time_86400_sum_max,
    min(u_time_86400_sum) as u_time_86400_sum_min,
    avg(u_time_86400_avg) as u_time_86400_avg_avg,
    max(u_time_86400_avg) as u_time_86400_avg_max,
    min(u_time_86400_avg) as u_time_86400_avg_min,
    avg(u_time_86400_std) as u_time_86400_std_avg,
    max(u_time_86400_std) as u_time_86400_std_max,
    min(u_time_86400_std) as u_time_86400_std_min,
    avg(u_time_86400_max) as u_time_86400_max_avg,
    max(u_time_86400_max) as u_time_86400_max_max,
    min(u_time_86400_max) as u_time_86400_max_min,
    avg(u_time_86400_min) as u_time_86400_min_avg,
    max(u_time_86400_min) as u_time_86400_min_max,
    min(u_time_86400_min) as u_time_86400_min_min,
    avg(u_time_86400_q1) as u_time_86400_q1_avg,
    max(u_time_86400_q1) as u_time_86400_q1_max,
    min(u_time_86400_q1) as u_time_86400_q1_min,
    avg(u_time_86400_q3) as u_time_86400_q3_avg,
    max(u_time_86400_q3) as u_time_86400_q3_max,
    min(u_time_86400_q3) as u_time_86400_q3_min,
    avg(u_time_86400_softNoise) as u_time_86400_softNoise_avg,
    max(u_time_86400_softNoise) as u_time_86400_softNoise_max,
    min(u_time_86400_softNoise) as u_time_86400_softNoise_min,
    avg(u_time_86400_hardNoise) as u_time_86400_hardNoise_avg,
    max(u_time_86400_hardNoise) as u_time_86400_hardNoise_max,
    min(u_time_86400_hardNoise) as u_time_86400_hardNoise_min,
    avg(u_time_604800_sum) as u_time_604800_sum_avg,
    max(u_time_604800_sum) as u_time_604800_sum_max,
    min(u_time_604800_sum) as u_time_604800_sum_min,
    avg(u_time_604800_avg) as u_time_604800_avg_avg,
    max(u_time_604800_avg) as u_time_604800_avg_max,
    min(u_time_604800_avg) as u_time_604800_avg_min,
    avg(u_time_604800_std) as u_time_604800_std_avg,
    max(u_time_604800_std) as u_time_604800_std_max,
    min(u_time_604800_std) as u_time_604800_std_min,
    avg(u_time_604800_max) as u_time_604800_max_avg,
    max(u_time_604800_max) as u_time_604800_max_max,
    min(u_time_604800_max) as u_time_604800_max_min,
    avg(u_time_604800_min) as u_time_604800_min_avg,
    max(u_time_604800_min) as u_time_604800_min_max,
    min(u_time_604800_min) as u_time_604800_min_min,
    avg(u_time_604800_q1) as u_time_604800_q1_avg,
    max(u_time_604800_q1) as u_time_604800_q1_max,
    min(u_time_604800_q1) as u_time_604800_q1_min,
    avg(u_time_604800_q3) as u_time_604800_q3_avg,
    max(u_time_604800_q3) as u_time_604800_q3_max,
    min(u_time_604800_q3) as u_time_604800_q3_min,
    avg(u_time_604800_softNoise) as u_time_604800_softNoise_avg,
    max(u_time_604800_softNoise) as u_time_604800_softNoise_max,
    min(u_time_604800_softNoise) as u_time_604800_softNoise_min,
    avg(u_time_604800_hardNoise) as u_time_604800_hardNoise_avg,
    max(u_time_604800_hardNoise) as u_time_604800_hardNoise_max,
    min(u_time_604800_hardNoise) as u_time_604800_hardNoise_min,
    avg(u_time_1209600_sum) as u_time_1209600_sum_avg,
    max(u_time_1209600_sum) as u_time_1209600_sum_max,
    min(u_time_1209600_sum) as u_time_1209600_sum_min,
    avg(u_time_1209600_avg) as u_time_1209600_avg_avg,
    max(u_time_1209600_avg) as u_time_1209600_avg_max,
    min(u_time_1209600_avg) as u_time_1209600_avg_min,
    avg(u_time_1209600_std) as u_time_1209600_std_avg,
    max(u_time_1209600_std) as u_time_1209600_std_max,
    min(u_time_1209600_std) as u_time_1209600_std_min,
    avg(u_time_1209600_max) as u_time_1209600_max_avg,
    max(u_time_1209600_max) as u_time_1209600_max_max,
    min(u_time_1209600_max) as u_time_1209600_max_min,
    avg(u_time_1209600_min) as u_time_1209600_min_avg,
    max(u_time_1209600_min) as u_time_1209600_min_max,
    min(u_time_1209600_min) as u_time_1209600_min_min,
    avg(u_time_1209600_q1) as u_time_1209600_q1_avg,
    max(u_time_1209600_q1) as u_time_1209600_q1_max,
    min(u_time_1209600_q1) as u_time_1209600_q1_min,
    avg(u_time_1209600_q3) as u_time_1209600_q3_avg,
    max(u_time_1209600_q3) as u_time_1209600_q3_max,
    min(u_time_1209600_q3) as u_time_1209600_q3_min,
    avg(u_time_1209600_softNoise) as u_time_1209600_softNoise_avg,
    max(u_time_1209600_softNoise) as u_time_1209600_softNoise_max,
    min(u_time_1209600_softNoise) as u_time_1209600_softNoise_min,
    avg(u_time_1209600_hardNoise) as u_time_1209600_hardNoise_avg,
    max(u_time_1209600_hardNoise) as u_time_1209600_hardNoise_max,
    min(u_time_1209600_hardNoise) as u_time_1209600_hardNoise_min,
    avg(u_time_2592000_sum) as u_time_2592000_sum_avg,
    max(u_time_2592000_sum) as u_time_2592000_sum_max,
    min(u_time_2592000_sum) as u_time_2592000_sum_min,
    avg(u_time_2592000_avg) as u_time_2592000_avg_avg,
    max(u_time_2592000_avg) as u_time_2592000_avg_max,
    min(u_time_2592000_avg) as u_time_2592000_avg_min,
    avg(u_time_2592000_std) as u_time_2592000_std_avg,
    max(u_time_2592000_std) as u_time_2592000_std_max,
    min(u_time_2592000_std) as u_time_2592000_std_min,
    avg(u_time_2592000_max) as u_time_2592000_max_avg,
    max(u_time_2592000_max) as u_time_2592000_max_max,
    min(u_time_2592000_max) as u_time_2592000_max_min,
    avg(u_time_2592000_min) as u_time_2592000_min_avg,
    max(u_time_2592000_min) as u_time_2592000_min_max,
    min(u_time_2592000_min) as u_time_2592000_min_min,
    avg(u_time_2592000_q1) as u_time_2592000_q1_avg,
    max(u_time_2592000_q1) as u_time_2592000_q1_max,
    min(u_time_2592000_q1) as u_time_2592000_q1_min,
    avg(u_time_2592000_q3) as u_time_2592000_q3_avg,
    max(u_time_2592000_q3) as u_time_2592000_q3_max,
    min(u_time_2592000_q3) as u_time_2592000_q3_min,
    avg(u_time_2592000_softNoise) as u_time_2592000_softNoise_avg,
    max(u_time_2592000_softNoise) as u_time_2592000_softNoise_max,
    min(u_time_2592000_softNoise) as u_time_2592000_softNoise_min,
    avg(u_time_2592000_hardNoise) as u_time_2592000_hardNoise_avg,
    max(u_time_2592000_hardNoise) as u_time_2592000_hardNoise_max,
    min(u_time_2592000_hardNoise) as u_time_2592000_hardNoise_min

  
    
from user_act_full
group by cust_id
"""
)

In [18]:
len(user_act_full2.columns)

541

In [19]:
user_act_full2.collect()

[Row(cust_id='cust_company__0e8cdba6-1eab-4b3a-b106-12553faae272', cnt_1800_avg=1.0, cnt_1800_max=1, cnt_1800_min=1, cnt_3600_avg=2.0, cnt_3600_max=3, cnt_3600_min=1, cnt_86400_avg=25.0, cnt_86400_max=26, cnt_86400_min=24, cnt_604800_avg=251.5, cnt_604800_max=254, cnt_604800_min=249, cnt_1209600_avg=527.0, cnt_1209600_max=529, cnt_1209600_min=525, cnt_2592000_avg=1152.0, cnt_2592000_max=1152, cnt_2592000_min=1152, crdr_0_1800_avg=0.5, crdr_0_1800_max=1, crdr_0_1800_min=0, crdr_1_1800_avg=0.5, crdr_1_1800_max=1, crdr_1_1800_min=0, crdr_0_3600_avg=1.5, crdr_0_3600_max=2, crdr_0_3600_min=1, crdr_1_3600_avg=0.5, crdr_1_3600_max=1, crdr_1_3600_min=0, crdr_0_86400_avg=12.0, crdr_0_86400_max=12, crdr_0_86400_min=12, crdr_1_86400_avg=13.0, crdr_1_86400_max=14, crdr_1_86400_min=12, crdr_0_604800_avg=149.0, crdr_0_604800_max=149, crdr_0_604800_min=149, crdr_1_604800_avg=102.5, crdr_1_604800_max=105, crdr_1_604800_min=100, crdr_0_1209600_avg=290.5, crdr_0_1209600_max=292, crdr_0_1209600_min=289, 

In [20]:
len(set(user_act_full2.columns))

541

In [21]:
user_act_full2.columns

['cust_id',
 'cnt_1800_avg',
 'cnt_1800_max',
 'cnt_1800_min',
 'cnt_3600_avg',
 'cnt_3600_max',
 'cnt_3600_min',
 'cnt_86400_avg',
 'cnt_86400_max',
 'cnt_86400_min',
 'cnt_604800_avg',
 'cnt_604800_max',
 'cnt_604800_min',
 'cnt_1209600_avg',
 'cnt_1209600_max',
 'cnt_1209600_min',
 'cnt_2592000_avg',
 'cnt_2592000_max',
 'cnt_2592000_min',
 'crdr_0_1800_avg',
 'crdr_0_1800_max',
 'crdr_0_1800_min',
 'crdr_1_1800_avg',
 'crdr_1_1800_max',
 'crdr_1_1800_min',
 'crdr_0_3600_avg',
 'crdr_0_3600_max',
 'crdr_0_3600_min',
 'crdr_1_3600_avg',
 'crdr_1_3600_max',
 'crdr_1_3600_min',
 'crdr_0_86400_avg',
 'crdr_0_86400_max',
 'crdr_0_86400_min',
 'crdr_1_86400_avg',
 'crdr_1_86400_max',
 'crdr_1_86400_min',
 'crdr_0_604800_avg',
 'crdr_0_604800_max',
 'crdr_0_604800_min',
 'crdr_1_604800_avg',
 'crdr_1_604800_max',
 'crdr_1_604800_min',
 'crdr_0_1209600_avg',
 'crdr_0_1209600_max',
 'crdr_0_1209600_min',
 'crdr_1_1209600_avg',
 'crdr_1_1209600_max',
 'crdr_1_1209600_min',
 'crdr_0_2592000_av